# Title
[]()

In [1]:
from pandas import json_normalize  
import pandas as pd
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\custom_python")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\src")
from silvhua import *
from datetime import datetime
# import json
# from plotly.subplots import make_subplots
# import requests

# Fix column alignment

In [8]:
from tabula import read_pdf
import json
import re
import warnings
warnings.simplefilter("ignore", category=FutureWarning)

def merge_rows(df, file, id_column=['Unnamed: 0'], fill_method='bfill', groupby_column=['HA']):
    """
    Merges rows in a pandas dataframe that were parsed incorrectly into multiple rows.
    Assumes that the rows that need to be merged have the same value in a unique ID column.
    """
    print(f'\tRemoving duplicate rows from {file}...')
    print('\t\tOriginal df shape: ', df.shape)
    print(f'\t\tOriginal columns:', df.columns)
    df[id_column] = df[id_column].fillna(method=fill_method)
    
    # Group the rows by the unique ID column and aggregate the values in each group

    columns_to_fill = df.drop(columns=groupby_column).columns
    grouped_df = df.groupby(id_column+groupby_column, as_index=False).agg(
        lambda x: ''.join(str(v) for v in x if (type(v)==str) & (x.name in columns_to_fill)))
    grouped_df = grouped_df[grouped_df[groupby_column[0]] != groupby_column[0]]
    # Return the merged dataframe
    print('\t\tTransformed df shape:', grouped_df.shape)
    return grouped_df.replace('', None)



def parse_pdf_tolist(pdf_filenames, filepath=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\BC-Covid-workplace-closures\data\raw', 
    pages='all', last_id_column='HA', value_to_ignore='s.22(1)',
    blank_columns=['workplace_name', 'workplace_address'],
    filename=None,
    path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\BC-Covid-workplace-closures\data\interim'):
    """
    Create dataframe from a PDF file using 
    """
    print(f'Extracting the following files:')
    if type(pdf_filenames) != list:
        pdf_filenames = [pdf_filenames]
    dfs_list = []
    reference_dict = dict()
    corrected_pages = dict()
    for index, file in enumerate(pdf_filenames):
        print(file)
        filepath = f'{filepath}/'.replace('\\','/') if filepath else ''
        path_and_file = filepath+file
        try:
            df_list = read_pdf(path_and_file, pages=pages)
        except:
            print(f'\nError parsing {file}')
        file_dfs = []
        page1_columns = df_list[0].columns.str.replace('\r', '').str.replace(r'(workplace)([a-zA-Z]+)', r'\1_\2', regex=True).to_list()
        print('number of columns:', len(page1_columns))
        if last_id_column in page1_columns:
            page1_columns_sorted = sorted(page1_columns)
        else:
            previous_columns = dfs_list[index-1].columns
            print(f'\t\tNo headers found; using headers from previous document:', previous_columns)
            if len(df_list[page_number].columns) > len(previous_columns):
                n_cols_to_pad = len(df_list[page_number].columns) - len(previous_columns)
                new_columns_padding = [f'blank{index}' for index, col in enumerate(range(n_cols_to_pad))]
                page1_columns_sorted = sorted(page1_columns) + new_columns_padding
                page1_columns = page1_columns + new_columns_padding

            elif len(df_list[page_number].columns) < len(previous_columns):
                page1_columns = (previous_columns[:len(df_list[page_number].columns)].to_list())
                page1_columns_sorted = sorted(page1_columns)
            else:
                page1_columns_sorted = (dfs_list[index-1].columns.to_list())
            page1_columns = ['Unnamed: 0' if column=='index' else column for column in page1_columns]
            print(f'\t\t**Current df columns:',page1_columns)
        
        for page_number in range(len(df_list)):
            print(f'\tpage {page_number+1}...')
            df_list[page_number].columns = df_list[page_number].columns.str.replace('\r', '').str.replace(r'(workplace)([a-zA-Z]+)', r'\1_\2', regex=True)
            print('\t\tCurrent number of columns:', len(df_list[page_number].columns), len(page1_columns_sorted))
            print(f'\t\t{len(page1_columns_sorted)}, {len(page1_columns)}')
            if len(df_list[page_number].columns.to_list()) == len(page1_columns_sorted):
                if sorted(df_list[page_number].columns.to_list()) == page1_columns_sorted:
                    pass
                else:
                    df_list[page_number].columns = page1_columns 
                    print('\t\tKeeping page 1 columns')
            else:
                if len(df_list[page_number].columns) > len(page1_columns_sorted):
                    n_cols_to_pad = len(df_list[page_number].columns) - len(page1_columns_sorted)
                    new_columns_padding = [f'blank{index}' for index, col in enumerate(range(n_cols_to_pad))]
                    df_list[page_number].columns = page1_columns + new_columns_padding
                    print('\t\tPadding columns')
                elif len(df_list[page_number].columns) < len(page1_columns_sorted):
                    df_list[page_number].columns = page1_columns[:len(df_list[page_number].columns)]
                    print('\t\tDropping columns')
                else:
                    df_list[page_number].columns = page1_columns
                corrected_pages[file] = page_number+1
                print(f'\n\t**Fixing column parsing for page {page_number+1} of {file}**')
                print(f'\t\t{df_list[page_number].columns}')
            # Consolidate rows that were incorrectly split
            if (df_list[page_number]['Unnamed: 0'].isna().sum() > 0):
                # print('\tAttempting to consolidate rows that were incorrectly split: ', df_list[page_number].columns)
                print('\tAttempting to consolidate rows that were incorrectly split: ')
                try:
                    df_list[page_number] = merge_rows(df_list[page_number], file)
                except:
                    print(f'\t\tUnable to consolidate rows on page {page_number+1} of {file}')
                    reference_dict['unparsed documents'] = file
                    continue

            # Align the columns
            if value_to_ignore in df_list[page_number].iloc[0].values:
                value_to_ignore_column = df_list[page_number].iloc[0].values.tolist().index(value_to_ignore)
                print(f'\tRemoving {value_to_ignore} from column {value_to_ignore_column}')
                df_list[page_number].iloc[0, value_to_ignore_column:-1] = df_list[page_number].iloc[0, value_to_ignore_column+1:].values

            for column in blank_columns:
                try:
                    if df_list[page_number][column].isna().sum() != len(df_list[page_number]): # 2023-03-30 22:45:
                        blank_column_index = df_list[page_number].columns.tolist().index(column)
                        print(f'\tShifting data from column {blank_column_index}({column})')
                        df_list[page_number].iloc[:,blank_column_index+1:] = df_list[page_number].iloc[:,blank_column_index:-1].values 
                        df_list[page_number].iloc[:,blank_column_index] = None
                except:
                    pass

            file_dfs.append(df_list[page_number]) # 2023-03-30 23:04 Decrease indent
        dfs_list.append(file_dfs) # SH 2023-04-01 20:58
        
        # try:
            # final_file_df = pd.concat(file_dfs)
            # final_file_df['file'] = file
            # final_file_df.columns = final_file_df.columns.str.replace('cosed', 'closed')
            # final_file_df.reset_index(inplace=True, drop=True)
            # final_file_df.rename(columns={'Unnamed: 0': 'index'}, inplace=True)
            # dfs_list.append(final_file_df)

        #     print('\n\tDone')
        # except:
        #     print(f'Error concatenating dataframes from {file}')
        #     return file_dfs, {file: 'DataFrame concatenation error'}
    # unique_cols_dict = {}
    # for i, df in enumerate(dfs_list):
    #     other_dfs = dfs_list[:i] + dfs_list[i+1:]
    #     other_cols = set().union(*[set(other_df.columns) for other_df in other_dfs])
    #     unique_columns = set(df.columns) - other_cols
    #     unique_cols_dict[file] = unique_columns if len(unique_columns)>0 else "none"
    #     print(f'Columns unique to part{pdf_filenames[i]}: {unique_cols_dict[file]}')
    # reference_dict['corrected pages'] = corrected_pages
    # reference_dict['unique columns'] = unique_cols_dict
    # if filename:
    #     try:
    #         savepickle(dfs_list, filename=filename, path=path)
    #         with open(f'{filepath}{filename}_reference.json','w') as json_file:
    #             json.dump(reference_dict, json_file)
    #     except:
    #         print('Unable to save outputs')

    return dfs_list, reference_dict




def parse_pdf(pdf_filenames,pages='all', last_id_column='HA', value_to_ignore='s.22(1)', 
    value_to_ignore_start_index=1, value_to_ignore_end_index=None,
    blank_columns=['workplace_name', 'workplace_address'],
    filename=None,filepath=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\BC-Covid-workplace-closures\data\raw', 
    path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\BC-Covid-workplace-closures\data\interim'):
    """
    Create dataframe from a PDF file using 
    """
    print(f'Extracting the following files:')
    if type(pdf_filenames) != list:
        pdf_filenames = [pdf_filenames]
    dfs_list = []
    reference_dict = dict()
    corrected_pages = dict()
    for index, file in enumerate(pdf_filenames):
        print(file)
        filepath = f'{filepath}/'.replace('\\','/') if filepath else ''
        path_and_file = filepath+file
        try:
            df_list = read_pdf(path_and_file, pages=pages)
        except:
            print(f'\nError parsing {file}')
        file_dfs = []
        page1_columns = df_list[0].columns.str.replace('\r', '').str.replace(r'(workplace)([a-zA-Z]+)', r'\1_\2', regex=True).to_list()
        print('number of columns:', len(page1_columns))
        if last_id_column in page1_columns:
            page1_columns_sorted = sorted(page1_columns)
        else:
            previous_columns = dfs_list[index-1].columns
            print(f'\t\tNo headers found; using headers from previous document:', previous_columns)
            if len(df_list[page_number].columns) > len(previous_columns):
                n_cols_to_pad = len(df_list[page_number].columns) - len(previous_columns)
                new_columns_padding = [f'blank{index}' for index, col in enumerate(range(n_cols_to_pad))]
                page1_columns_sorted = sorted(page1_columns) + new_columns_padding
                page1_columns = page1_columns + new_columns_padding

            elif len(df_list[page_number].columns) < len(previous_columns):
                page1_columns = (previous_columns[:len(df_list[page_number].columns)].to_list())
                page1_columns_sorted = sorted(page1_columns)
            else:
                page1_columns_sorted = (dfs_list[index-1].columns.to_list())
            page1_columns = ['Unnamed: 0' if column=='index' else column for column in page1_columns]
            print(f'\t\t**Current df columns:',page1_columns)
        
        for page_number in range(len(df_list)):
            print(f'\tpage {page_number+1}...')
            df_list[page_number].columns = df_list[page_number].columns.str.replace('\r', '').str.replace(r'(workplace)([a-zA-Z]+)', r'\1_\2', regex=True)
            # print('\t\tCurrent number of columns:', len(df_list[page_number].columns), len(page1_columns_sorted))
            # print(f'\t\t{len(page1_columns_sorted)}, {len(page1_columns)}')
            if len(df_list[page_number].columns.to_list()) == len(page1_columns_sorted):
                if sorted(df_list[page_number].columns.to_list()) == page1_columns_sorted:
                    pass
                else:
                    df_list[page_number].columns = page1_columns 
                    print('\t\tKeeping page 1 columns')
            else:
                if len(df_list[page_number].columns) > len(page1_columns_sorted):
                    n_cols_to_pad = len(df_list[page_number].columns) - len(page1_columns_sorted)
                    new_columns_padding = [f'blank{index}' for index, col in enumerate(range(n_cols_to_pad))]
                    df_list[page_number].columns = page1_columns + new_columns_padding
                    print('\t\tPadding columns')
                elif len(df_list[page_number].columns) < len(page1_columns_sorted):
                    df_list[page_number].columns = page1_columns[:len(df_list[page_number].columns)]
                    print('\t\tDropping columns')
                else:
                    df_list[page_number].columns = page1_columns
                corrected_pages[file] = page_number+1
                print(f'\n\t**Fixing column parsing for page {page_number+1} of {file}**')
                print(f'\t\t{df_list[page_number].columns}')
            # Consolidate rows that were incorrectly split
            if (df_list[page_number]['Unnamed: 0'].isna().sum() > 0):
                # print('\tAttempting to consolidate rows that were incorrectly split: ', df_list[page_number].columns)
                print('\tAttempting to consolidate rows that were incorrectly split: ')
                try:
                    df_list[page_number] = merge_rows(df_list[page_number], file)
                except:
                    print(f'\t\tUnable to consolidate rows on page {page_number+1} of {file}')
                    reference_dict['unparsed documents'] = file
                    continue

            # Align the columns
            value_to_ignore_regex = rf'.*{re.escape(value_to_ignore[value_to_ignore_start_index:value_to_ignore_end_index])}.*'
            if df_list[page_number].iloc[0].str.contains(value_to_ignore_regex).any():
                row_values = df_list[page_number].fillna(0).iloc[0].values.tolist()
                for index, item in enumerate(row_values):
                    if re.match(value_to_ignore_regex, str(item)):
                        # print('found regex')
                        value_to_ignore_column = index
                        break

                print(f'\tRemoving {value_to_ignore} from column {value_to_ignore_column}')
                df_list[page_number].iloc[0, value_to_ignore_column:-1] = df_list[page_number].iloc[0, value_to_ignore_column+1:].values

            for column in blank_columns:
                try:
                    if df_list[page_number][column].isna().sum() != len(df_list[page_number]): # 2023-03-30 22:45:
                        blank_column_index = df_list[page_number].columns.tolist().index(column)
                        print(f'\tShifting data from column {blank_column_index}({column})')
                        df_list[page_number].iloc[:,blank_column_index+1:] = df_list[page_number].iloc[:,blank_column_index:-1].values 
                        df_list[page_number].iloc[:,blank_column_index] = None
                except:
                    pass
            df_list[page_number]['doc_page_number'] = page_number + 1 # 2023-03-30 23:15
            file_dfs.append(df_list[page_number])  # 2023-03-30 23:04 Decrease indent
        
        try:
            final_file_df = pd.concat(file_dfs)
            final_file_df['file'] = file
            final_file_df.columns = final_file_df.columns.str.replace('cosed', 'closed')
            final_file_df.reset_index(inplace=True, drop=True)
            final_file_df.rename(columns={'Unnamed: 0': 'index'}, inplace=True)
            dfs_list.append(final_file_df)

            print('\n\tDone')
        except:
            print(f'Error concatenating dataframes from {file}')
            return file_dfs, {file: 'DataFrame concatenation error'}
    unique_cols_dict = {}
    for i, df in enumerate(dfs_list):
        other_dfs = dfs_list[:i] + dfs_list[i+1:]
        other_cols = set().union(*[set(other_df.columns) for other_df in other_dfs])
        unique_columns = set(df.columns) - other_cols
        unique_cols_dict[file] = unique_columns if len(unique_columns)>0 else "none"
        print(f'Columns unique to part{pdf_filenames[i]}: {unique_cols_dict[file]}')
    reference_dict['corrected pages'] = corrected_pages
    reference_dict['unique columns'] = unique_cols_dict
    if filename:
        try:
            savepickle(dfs_list, filename=filename, path=path)
            with open(f'{filepath}{filename}_reference.json','w') as json_file:
                json.dump(reference_dict, json_file)
            save_csv(pd.concat(dfs_list), filename=filename, path=path)
        except:
            print('Unable to save outputs')
    try:
        return pd.concat(dfs_list), reference_dict
    except:
        print('Unable to concatenate dataframes from different documents')
        return dfs_list, reference_dict

n_docs = 7
start = 1
filenames = [f'F22-1445 Records_Part{i}.pdf' for i in range(start,start+n_docs)]
part = 1
filename = f'F22-1445 Records_Part{part}.pdf'

dfs_list1, reference1 = parse_pdf_tolist(filename, pages='1,118', filename=None)
# dfs, reference = parse_pdf(filenames, pages='all', filename='pdfs_parsed_2023-03-30')


Extracting the following files:
F22-1445 Records_Part1.pdf
number of columns: 22
	page 1...
		Current number of columns: 22 22
		22, 22
	Shifting data from column 2(workplace_name)
	Shifting data from column 3(workplace_address)
	page 2...
		Current number of columns: 25 22
		22, 22
		Padding columns

	**Fixing column parsing for page 2 of F22-1445 Records_Part1.pdf**
		Index(['Unnamed: 0', 'HA', 'workplace_name', 'workplace_address', 'city',
       'date_cluster_confirmed_public_health', 'date_cluster_no_longer_active',
       'earliest_reported_cases', 'latest_reported_cases',
       'workplace_classification', 'cases_when_cluster_confirmed',
       'total_cases', 'workplace_status', 'closure_y_n', 'closed_by_order',
       'voluntary_closure', 'closure_date', 'partially_cosed', 'open',
       'days_since_last_case', 'cluster_confirmed_epidate',
       'cluster_closed_epidate', 'blank0', 'blank1', 'blank2'],
      dtype='object')
	Attempting to consolidate rows that were incorrectly 

In [11]:
dfs_list1[0][1]

,Unnamed: 0,HA,workplace_name,workplace_address,city,date_cluster_confirmed_public_health,date_cluster_no_longer_active,earliest_reported_cases,latest_reported_cases,workplace_classification,...,voluntary_closure,closure_date,partially_cosed,open,days_since_last_case,cluster_confirmed_epidate,cluster_closed_epidate,blank0,blank1,blank2
0,455.0,FHA,None,None,None,None,SURREY,5/17/2021,5/28/2021,None,...,No,No,Unknown,None,No,Yes,None,No,20-2021,None
2,456.0,FHA,None,None,None,None,ABBOTSFORD,5/17/2021,5/28/2021,None,...,No,No,Unknown,None,No,Yes,None,No,20-2021,None
3,457.0,FHA,None,None,None,None,SURREY,5/17/2021,6/3/2021,None,...,No,No,Unknown,None,No,Yes,None,No,20-2021,None
4,458.0,FHA,None,None,None,None,MAPLE RIDGE,5/17/2021,6/2/2021,None,...,Yes,Yes,Unknown,5/18/2021,No,No,None,No,20-2021,20-2021
5,459.0,FHA,None,None,None,None,SURREY,5/17/2021,5/31/2021,None,...,No,No,Unknown,None,No,Yes,None,No,20-2021,None
6,460.0,FHA,None,None,None,None,DELTA,5/17/2021,5/27/2021,None,...,No,No,Unknown,None,No,Yes,None,No,20-2021,None
7,461.0,FHA,None,None,None,None,WESTMINSTER,5/17/2021,5/25/2021,None,...,No,No,Unknown,None,No,Yes,None,No,20-2021,None
8,462.0,NHA,None,None,None,None,Unknown,5/17/2021,None,None,...,No,No,No,None,n,No,None,Yes,20-2021,None
9,463.0,FHA,None,None,None,None,BURNABY,5/18/2021,6/7/2021,None,...,Yes,Yes,Unknown,5/19/2021,No,No,None,No,20-2021,20-2021
10,464.0,VIHA,None,None,None,None,Nanaimo,5/18/2021,None,5/16/2021,...,No,No,No,None,n,Yes,35,No,20-2021,None


## Iteration 2

In [32]:
from tabula import read_pdf
import json
import re
import warnings
warnings.simplefilter("ignore", category=FutureWarning)

def merge_rows(df, file, id_column=['Unnamed: 0'], fill_method='bfill', groupby_column=['HA']):
    """
    Merges rows in a pandas dataframe that were parsed incorrectly into multiple rows.
    Assumes that the rows that need to be merged have the same value in a unique ID column.
    """
    print(f'\tRemoving duplicate rows from {file}...')
    print('\t\tOriginal df shape: ', df.shape)
    print(f'\t\tOriginal columns:', df.columns)
    df[id_column] = df[id_column].fillna(method=fill_method)
    
    # Group the rows by the unique ID column and aggregate the values in each group

    columns_to_fill = df.drop(columns=groupby_column).columns
    grouped_df = df.groupby(id_column+groupby_column, as_index=False).agg(
        lambda x: ''.join(str(v) for v in x if (type(v)==str) & (x.name in columns_to_fill)))
    grouped_df = grouped_df[grouped_df[groupby_column[0]] != groupby_column[0]]
    # Return the merged dataframe
    print('\t\tTransformed df shape:', grouped_df.shape)
    return grouped_df.replace('', None)



def parse_pdf_tolist(pdf_filenames, filepath=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\BC-Covid-workplace-closures\data\raw', 
    pages='all', last_id_column='HA', value_to_ignore='s.22(1)',
    blank_columns=['workplace_name', 'workplace_address'],
    filename=None,
    path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\BC-Covid-workplace-closures\data\interim'):
    """
    Create dataframe from a PDF file using 
    """
    print(f'Extracting the following files:')
    if type(pdf_filenames) != list:
        pdf_filenames = [pdf_filenames]
    dfs_list = []
    reference_dict = dict()
    corrected_pages = dict()
    for index, file in enumerate(pdf_filenames):
        print(file)
        filepath = f'{filepath}/'.replace('\\','/') if filepath else ''
        path_and_file = filepath+file
        try:
            df_list = read_pdf(path_and_file, pages=pages)
        except:
            print(f'\nError parsing {file}')
        file_dfs = []
        page1_columns = df_list[0].columns.str.replace('\r', '').str.replace(r'(workplace)([a-zA-Z]+)', r'\1_\2', regex=True).to_list()
        print('number of columns:', len(page1_columns))
        if last_id_column in page1_columns:
            page1_columns_sorted = sorted(page1_columns)
        else:
            previous_columns = dfs_list[index-1].columns
            print(f'\t\tNo headers found; using headers from previous document:', previous_columns)
            if len(df_list[page_number].columns) > len(previous_columns):
                n_cols_to_pad = len(df_list[page_number].columns) - len(previous_columns)
                new_columns_padding = [f'blank{index}' for index, col in enumerate(range(n_cols_to_pad))]
                page1_columns_sorted = sorted(page1_columns) + new_columns_padding
                page1_columns = page1_columns + new_columns_padding

            elif len(df_list[page_number].columns) < len(previous_columns):
                page1_columns = (previous_columns[:len(df_list[page_number].columns)].to_list())
                page1_columns_sorted = sorted(page1_columns)
            else:
                page1_columns_sorted = (dfs_list[index-1].columns.to_list())
            page1_columns = ['Unnamed: 0' if column=='index' else column for column in page1_columns]
            print(f'\t\t**Current df columns:',page1_columns)
        
        for page_number in range(len(df_list)):
            print(f'\tpage {page_number+1}...')
            df_list[page_number].columns = df_list[page_number].columns.str.replace('\r', '').str.replace(r'(workplace)([a-zA-Z]+)', r'\1_\2', regex=True)
            print('\t\tCurrent number of columns:', len(df_list[page_number].columns), len(page1_columns_sorted))
            print(f'\t\t{len(page1_columns_sorted)}, {len(page1_columns)}')
            if len(df_list[page_number].columns.to_list()) == len(page1_columns_sorted):
                if sorted(df_list[page_number].columns.to_list()) == page1_columns_sorted:
                    pass
                else:
                    df_list[page_number].columns = page1_columns 
                    print('\t\tKeeping page 1 columns')
            else:
                if len(df_list[page_number].columns) > len(page1_columns_sorted):
                    n_cols_to_pad = len(df_list[page_number].columns) - len(page1_columns_sorted)
                    new_columns_padding = [f'blank{index}' for index, col in enumerate(range(n_cols_to_pad))]
                    df_list[page_number].columns = page1_columns + new_columns_padding
                    print('\t\tPadding columns')
                elif len(df_list[page_number].columns) < len(page1_columns_sorted):
                    df_list[page_number].columns = page1_columns[:len(df_list[page_number].columns)]
                    print('\t\tDropping columns')
                else:
                    df_list[page_number].columns = page1_columns
                corrected_pages[file] = page_number+1
                print(f'\n\t**Fixing column parsing for page {page_number+1} of {file}**')
                print(f'\t\t{df_list[page_number].columns}')
            # Consolidate rows that were incorrectly split
            if (df_list[page_number]['Unnamed: 0'].isna().sum() > 0):
                # print('\tAttempting to consolidate rows that were incorrectly split: ', df_list[page_number].columns)
                print('\tAttempting to consolidate rows that were incorrectly split: ')
                try:
                    df_list[page_number] = merge_rows(df_list[page_number], file)
                except:
                    print(f'\t\tUnable to consolidate rows on page {page_number+1} of {file}')
                    reference_dict['unparsed documents'] = file
                    continue

            # Align the columns
            if value_to_ignore in df_list[page_number].iloc[0].values:
                value_to_ignore_column = df_list[page_number].iloc[0].values.tolist().index(value_to_ignore)
                print(f'\tRemoving {value_to_ignore} from column {value_to_ignore_column}')
                df_list[page_number].iloc[0, value_to_ignore_column:-1] = df_list[page_number].iloc[0, value_to_ignore_column+1:].values

            for column in blank_columns:
                try:
                    if df_list[page_number][column].isna().sum() != len(df_list[page_number]): # 2023-03-30 22:45:
                        blank_column_index = df_list[page_number].columns.tolist().index(column)
                        print(f'\tShifting data from column {blank_column_index}({column})')
                        df_list[page_number].iloc[:,blank_column_index+1:] = df_list[page_number].iloc[:,blank_column_index:-1].values 
                        df_list[page_number].iloc[:,blank_column_index] = None
                except:
                    pass
            try:
                if n_cols_to_pad: # SH 2023-04-01 21:06
                    n_cols_to_drop = n_cols_to_pad - 1
                    column_after_blank_columns_index = blank_column_index + 1
                    print(f'\tShifting data {n_cols_to_drop} to the right until column {column_after_blank_columns_index}')
                    df_list[page_number].iloc[:,column_after_blank_columns_index:-n_cols_to_drop] = df_list[
                        page_number].iloc[:,column_after_blank_columns_index + n_cols_to_drop:].values 
                    df_list[page_number].iloc[:,blank_column_index] = None
            except:
                pass

            file_dfs.append(df_list[page_number]) # 2023-03-30 23:04 Decrease indent
        dfs_list.append(file_dfs) # SH 2023-04-01 20:58
        
        # try:
            # final_file_df = pd.concat(file_dfs)
            # final_file_df['file'] = file
            # final_file_df.columns = final_file_df.columns.str.replace('cosed', 'closed')
            # final_file_df.reset_index(inplace=True, drop=True)
            # final_file_df.rename(columns={'Unnamed: 0': 'index'}, inplace=True)
            # dfs_list.append(final_file_df)

        #     print('\n\tDone')
        # except:
        #     print(f'Error concatenating dataframes from {file}')
        #     return file_dfs, {file: 'DataFrame concatenation error'}
    # unique_cols_dict = {}
    # for i, df in enumerate(dfs_list):
    #     other_dfs = dfs_list[:i] + dfs_list[i+1:]
    #     other_cols = set().union(*[set(other_df.columns) for other_df in other_dfs])
    #     unique_columns = set(df.columns) - other_cols
    #     unique_cols_dict[file] = unique_columns if len(unique_columns)>0 else "none"
    #     print(f'Columns unique to part{pdf_filenames[i]}: {unique_cols_dict[file]}')
    # reference_dict['corrected pages'] = corrected_pages
    # reference_dict['unique columns'] = unique_cols_dict
    # if filename:
    #     try:
    #         savepickle(dfs_list, filename=filename, path=path)
    #         with open(f'{filepath}{filename}_reference.json','w') as json_file:
    #             json.dump(reference_dict, json_file)
    #     except:
    #         print('Unable to save outputs')

    return dfs_list, reference_dict




def parse_pdf(pdf_filenames,pages='all', last_id_column='HA', value_to_ignore='s.22(1)', 
    value_to_ignore_start_index=1, value_to_ignore_end_index=None,
    blank_columns=['workplace_name', 'workplace_address'],
    filename=None,filepath=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\BC-Covid-workplace-closures\data\raw', 
    path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\BC-Covid-workplace-closures\data\interim'):
    """
    Create dataframe from a PDF file using 
    """
    print(f'Extracting the following files:')
    if type(pdf_filenames) != list:
        pdf_filenames = [pdf_filenames]
    dfs_list = []
    reference_dict = dict()
    corrected_pages = dict()
    for index, file in enumerate(pdf_filenames):
        print(file)
        filepath = f'{filepath}/'.replace('\\','/') if filepath else ''
        path_and_file = filepath+file
        try:
            df_list = read_pdf(path_and_file, pages=pages)
        except:
            print(f'\nError parsing {file}')
        file_dfs = []
        page1_columns = df_list[0].columns.str.replace('\r', '').str.replace(r'(workplace)([a-zA-Z]+)', r'\1_\2', regex=True).to_list()
        print('number of columns:', len(page1_columns))
        if last_id_column in page1_columns:
            page1_columns_sorted = sorted(page1_columns)
        else:
            previous_columns = dfs_list[index-1].columns
            print(f'\t\tNo headers found; using headers from previous document:', previous_columns)
            if len(df_list[page_number].columns) > len(previous_columns):
                n_cols_to_pad = len(df_list[page_number].columns) - len(previous_columns)
                new_columns_padding = [f'blank{index}' for index, col in enumerate(range(n_cols_to_pad))]
                page1_columns_sorted = sorted(page1_columns) + new_columns_padding
                page1_columns = page1_columns + new_columns_padding

            elif len(df_list[page_number].columns) < len(previous_columns):
                page1_columns = (previous_columns[:len(df_list[page_number].columns)].to_list())
                page1_columns_sorted = sorted(page1_columns)
            else:
                page1_columns_sorted = (dfs_list[index-1].columns.to_list())
            page1_columns = ['Unnamed: 0' if column=='index' else column for column in page1_columns]
            print(f'\t\t**Current df columns:',page1_columns)
        
        for page_number in range(len(df_list)):
            print(f'\tpage {page_number+1}...')
            df_list[page_number].columns = df_list[page_number].columns.str.replace('\r', '').str.replace(r'(workplace)([a-zA-Z]+)', r'\1_\2', regex=True)
            # print('\t\tCurrent number of columns:', len(df_list[page_number].columns), len(page1_columns_sorted))
            # print(f'\t\t{len(page1_columns_sorted)}, {len(page1_columns)}')
            if len(df_list[page_number].columns.to_list()) == len(page1_columns_sorted):
                if sorted(df_list[page_number].columns.to_list()) == page1_columns_sorted:
                    pass
                else:
                    df_list[page_number].columns = page1_columns 
                    print('\t\tKeeping page 1 columns')
            else:
                if len(df_list[page_number].columns) > len(page1_columns_sorted):
                    n_cols_to_pad = len(df_list[page_number].columns) - len(page1_columns_sorted)
                    new_columns_padding = [f'blank{index}' for index, col in enumerate(range(n_cols_to_pad))]
                    df_list[page_number].columns = page1_columns + new_columns_padding
                    print('\t\tPadding columns')
                elif len(df_list[page_number].columns) < len(page1_columns_sorted):
                    df_list[page_number].columns = page1_columns[:len(df_list[page_number].columns)]
                    print('\t\tDropping columns')
                else:
                    df_list[page_number].columns = page1_columns
                corrected_pages[file] = page_number+1
                print(f'\n\t**Fixing column parsing for page {page_number+1} of {file}**')
                print(f'\t\t{df_list[page_number].columns}')
            # Consolidate rows that were incorrectly split
            if (df_list[page_number]['Unnamed: 0'].isna().sum() > 0):
                # print('\tAttempting to consolidate rows that were incorrectly split: ', df_list[page_number].columns)
                print('\tAttempting to consolidate rows that were incorrectly split: ')
                try:
                    df_list[page_number] = merge_rows(df_list[page_number], file)
                except:
                    print(f'\t\tUnable to consolidate rows on page {page_number+1} of {file}')
                    reference_dict['unparsed documents'] = file
                    continue

            # Align the columns
            value_to_ignore_regex = rf'.*{re.escape(value_to_ignore[value_to_ignore_start_index:value_to_ignore_end_index])}.*'
            if df_list[page_number].iloc[0].str.contains(value_to_ignore_regex).any():
                row_values = df_list[page_number].fillna(0).iloc[0].values.tolist()
                for index, item in enumerate(row_values):
                    if re.match(value_to_ignore_regex, str(item)):
                        # print('found regex')
                        value_to_ignore_column = index
                        break

                print(f'\tRemoving {value_to_ignore} from column {value_to_ignore_column}')
                df_list[page_number].iloc[0, value_to_ignore_column:-1] = df_list[page_number].iloc[0, value_to_ignore_column+1:].values

            for column in blank_columns:
                try:
                    if df_list[page_number][column].isna().sum() != len(df_list[page_number]): # 2023-03-30 22:45:
                        blank_column_index = df_list[page_number].columns.tolist().index(column)
                        print(f'\tShifting data from column {blank_column_index}({column})')
                        df_list[page_number].iloc[:,blank_column_index+1:] = df_list[page_number].iloc[:,blank_column_index:-1].values 
                        df_list[page_number].iloc[:,blank_column_index] = None
                except:
                    pass
            df_list[page_number]['doc_page_number'] = page_number + 1 # 2023-03-30 23:15
            file_dfs.append(df_list[page_number])  # 2023-03-30 23:04 Decrease indent
        
        try:
            final_file_df = pd.concat(file_dfs)
            final_file_df['file'] = file
            final_file_df.columns = final_file_df.columns.str.replace('cosed', 'closed')
            final_file_df.reset_index(inplace=True, drop=True)
            final_file_df.rename(columns={'Unnamed: 0': 'index'}, inplace=True)
            dfs_list.append(final_file_df)

            print('\n\tDone')
        except:
            print(f'Error concatenating dataframes from {file}')
            return file_dfs, {file: 'DataFrame concatenation error'}
    unique_cols_dict = {}
    for i, df in enumerate(dfs_list):
        other_dfs = dfs_list[:i] + dfs_list[i+1:]
        other_cols = set().union(*[set(other_df.columns) for other_df in other_dfs])
        unique_columns = set(df.columns) - other_cols
        unique_cols_dict[file] = unique_columns if len(unique_columns)>0 else "none"
        print(f'Columns unique to part{pdf_filenames[i]}: {unique_cols_dict[file]}')
    reference_dict['corrected pages'] = corrected_pages
    reference_dict['unique columns'] = unique_cols_dict
    if filename:
        try:
            savepickle(dfs_list, filename=filename, path=path)
            with open(f'{filepath}{filename}_reference.json','w') as json_file:
                json.dump(reference_dict, json_file)
            save_csv(pd.concat(dfs_list), filename=filename, path=path)
        except:
            print('Unable to save outputs')
    try:
        return pd.concat(dfs_list), reference_dict
    except:
        print('Unable to concatenate dataframes from different documents')
        return dfs_list, reference_dict

n_docs = 7
start = 1
filenames = [f'F22-1445 Records_Part{i}.pdf' for i in range(start,start+n_docs)]
part = 1
filename = f'F22-1445 Records_Part{part}.pdf'

dfs_list1, reference1 = parse_pdf_tolist(filename, pages='1,118', filename=None)
# dfs, reference = parse_pdf(filenames, pages='all', filename='pdfs_parsed_2023-03-30')


Extracting the following files:
F22-1445 Records_Part1.pdf
number of columns: 22
	page 1...
		Current number of columns: 22 22
		22, 22
	Shifting data from column 2(workplace_name)
	Shifting data from column 3(workplace_address)
	page 2...
		Current number of columns: 25 22
		22, 22
		Padding columns

	**Fixing column parsing for page 2 of F22-1445 Records_Part1.pdf**
		Index(['Unnamed: 0', 'HA', 'workplace_name', 'workplace_address', 'city',
       'date_cluster_confirmed_public_health', 'date_cluster_no_longer_active',
       'earliest_reported_cases', 'latest_reported_cases',
       'workplace_classification', 'cases_when_cluster_confirmed',
       'total_cases', 'workplace_status', 'closure_y_n', 'closed_by_order',
       'voluntary_closure', 'closure_date', 'partially_cosed', 'open',
       'days_since_last_case', 'cluster_confirmed_epidate',
       'cluster_closed_epidate', 'blank0', 'blank1', 'blank2'],
      dtype='object')
	Attempting to consolidate rows that were incorrectly 

In [33]:
print(dfs_list1[0][1].columns)
dfs_list1[0][1]

Index(['Unnamed: 0', 'HA', 'workplace_name', 'workplace_address', 'city',
       'date_cluster_confirmed_public_health', 'date_cluster_no_longer_active',
       'earliest_reported_cases', 'latest_reported_cases',
       'workplace_classification', 'cases_when_cluster_confirmed',
       'total_cases', 'workplace_status', 'closure_y_n', 'closed_by_order',
       'voluntary_closure', 'closure_date', 'partially_cosed', 'open',
       'days_since_last_case', 'cluster_confirmed_epidate',
       'cluster_closed_epidate', 'blank0', 'blank1', 'blank2'],
      dtype='object')


,Unnamed: 0,HA,workplace_name,workplace_address,city,date_cluster_confirmed_public_health,date_cluster_no_longer_active,earliest_reported_cases,latest_reported_cases,workplace_classification,...,voluntary_closure,closure_date,partially_cosed,open,days_since_last_case,cluster_confirmed_epidate,cluster_closed_epidate,blank0,blank1,blank2
0,455.0,FHA,None,None,SURREY,5/17/2021,5/28/2021,None,None,n,...,Unknown,None,No,Yes,None,No,20-2021,None,20-2021,None
2,456.0,FHA,None,None,ABBOTSFORD,5/17/2021,5/28/2021,None,None,uction,...,Unknown,None,No,Yes,None,No,20-2021,None,20-2021,None
3,457.0,FHA,None,None,SURREY,5/17/2021,6/3/2021,None,None,Office,...,Unknown,None,No,Yes,None,No,20-2021,None,20-2021,None
4,458.0,FHA,None,None,MAPLE RIDGE,5/17/2021,6/2/2021,None,None,g/Industry,...,Unknown,5/18/2021,No,No,None,No,20-2021,20-2021,20-2021,20-2021
5,459.0,FHA,None,None,SURREY,5/17/2021,5/31/2021,None,None,n,...,Unknown,None,No,Yes,None,No,20-2021,None,20-2021,None
6,460.0,FHA,None,None,DELTA,5/17/2021,5/27/2021,None,None,g/Industry,...,Unknown,None,No,Yes,None,No,20-2021,None,20-2021,None
7,461.0,FHA,None,None,WESTMINSTER,5/17/2021,5/25/2021,None,None,Retail,...,Unknown,None,No,Yes,None,No,20-2021,None,20-2021,None
8,462.0,NHA,None,None,Unknown,5/17/2021,None,None,None,Housing,...,No,None,n,No,None,Yes,20-2021,None,20-2021,None
9,463.0,FHA,None,None,BURNABY,5/18/2021,6/7/2021,None,None,ar/Lounge,...,Unknown,5/19/2021,No,No,None,No,20-2021,20-2021,20-2021,20-2021
10,464.0,VIHA,None,None,Nanaimo,5/18/2021,None,5/16/2021,5/18/2021,uction,...,No,None,n,Yes,35,No,20-2021,None,20-2021,None


## Iteration 3

In [37]:
from tabula import read_pdf
import json
import re
import warnings
warnings.simplefilter("ignore", category=FutureWarning)

def merge_rows(df, file, id_column=['Unnamed: 0'], fill_method='bfill', groupby_column=['HA']):
    """
    Merges rows in a pandas dataframe that were parsed incorrectly into multiple rows.
    Assumes that the rows that need to be merged have the same value in a unique ID column.
    """
    print(f'\tRemoving duplicate rows from {file}...')
    print('\t\tOriginal df shape: ', df.shape)
    print(f'\t\tOriginal columns:', df.columns)
    df[id_column] = df[id_column].fillna(method=fill_method)
    
    # Group the rows by the unique ID column and aggregate the values in each group

    columns_to_fill = df.drop(columns=groupby_column).columns
    grouped_df = df.groupby(id_column+groupby_column, as_index=False).agg(
        lambda x: ''.join(str(v) for v in x if (type(v)==str) & (x.name in columns_to_fill)))
    grouped_df = grouped_df[grouped_df[groupby_column[0]] != groupby_column[0]]
    # Return the merged dataframe
    print('\t\tTransformed df shape:', grouped_df.shape)
    return grouped_df.replace('', None)



def parse_pdf_tolist(pdf_filenames, filepath=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\BC-Covid-workplace-closures\data\raw', 
    pages='all', last_id_column='HA', value_to_ignore='s.22(1)',
    blank_columns=['workplace_name', 'workplace_address'],
    filename=None,
    path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\BC-Covid-workplace-closures\data\interim'):
    """
    Create dataframe from a PDF file using 
    """
    print(f'Extracting the following files:')
    if type(pdf_filenames) != list:
        pdf_filenames = [pdf_filenames]
    dfs_list = []
    reference_dict = dict()
    corrected_pages = dict()
    for index, file in enumerate(pdf_filenames):
        print(file)
        filepath = f'{filepath}/'.replace('\\','/') if filepath else ''
        path_and_file = filepath+file
        try:
            df_list = read_pdf(path_and_file, pages=pages)
        except:
            print(f'\nError parsing {file}')
        file_dfs = []
        page1_columns = df_list[0].columns.str.replace('\r', '').str.replace(r'(workplace)([a-zA-Z]+)', r'\1_\2', regex=True).to_list()
        print('number of columns:', len(page1_columns))
        if last_id_column in page1_columns:
            page1_columns_sorted = sorted(page1_columns)
        else:
            previous_columns = dfs_list[index-1].columns
            print(f'\t\tNo headers found; using headers from previous document:', previous_columns)
            if len(df_list[page_number].columns) > len(previous_columns):
                n_cols_to_pad = len(df_list[page_number].columns) - len(previous_columns)
                new_columns_padding = [f'blank{index}' for index, col in enumerate(range(n_cols_to_pad))]
                page1_columns_sorted = sorted(page1_columns) + new_columns_padding
                page1_columns = page1_columns + new_columns_padding

            elif len(df_list[page_number].columns) < len(previous_columns):
                page1_columns = (previous_columns[:len(df_list[page_number].columns)].to_list())
                page1_columns_sorted = sorted(page1_columns)
            else:
                page1_columns_sorted = (dfs_list[index-1].columns.to_list())
            page1_columns = ['Unnamed: 0' if column=='index' else column for column in page1_columns]
            print(f'\t\t**Current df columns:',page1_columns)
        
        for page_number in range(len(df_list)):
            print(f'\tpage {page_number+1}...')
            df_list[page_number].columns = df_list[page_number].columns.str.replace('\r', '').str.replace(r'(workplace)([a-zA-Z]+)', r'\1_\2', regex=True)
            print('\t\tCurrent number of columns:', len(df_list[page_number].columns), len(page1_columns_sorted))
            print(f'\t\t{len(page1_columns_sorted)}, {len(page1_columns)}')
            if len(df_list[page_number].columns.to_list()) == len(page1_columns_sorted):
                if sorted(df_list[page_number].columns.to_list()) == page1_columns_sorted:
                    pass
                else:
                    df_list[page_number].columns = page1_columns 
                    print('\t\tKeeping page 1 columns')
            else:
                if len(df_list[page_number].columns) > len(page1_columns_sorted):
                    n_cols_to_pad = len(df_list[page_number].columns) - len(page1_columns_sorted)
                    new_columns_padding = [f'blank{index}' for index, col in enumerate(range(n_cols_to_pad))]
                    df_list[page_number].columns = page1_columns + new_columns_padding
                    print('\t\tPadding columns')
                elif len(df_list[page_number].columns) < len(page1_columns_sorted):
                    df_list[page_number].columns = page1_columns[:len(df_list[page_number].columns)]
                    print('\t\tDropping columns')
                else:
                    df_list[page_number].columns = page1_columns
                corrected_pages[file] = page_number+1
                print(f'\n\t**Fixing column parsing for page {page_number+1} of {file}**')
                print(f'\t\t{df_list[page_number].columns}')
            # Consolidate rows that were incorrectly split
            if (df_list[page_number]['Unnamed: 0'].isna().sum() > 0):
                # print('\tAttempting to consolidate rows that were incorrectly split: ', df_list[page_number].columns)
                print('\tAttempting to consolidate rows that were incorrectly split: ')
                try:
                    df_list[page_number] = merge_rows(df_list[page_number], file)
                except:
                    print(f'\t\tUnable to consolidate rows on page {page_number+1} of {file}')
                    reference_dict['unparsed documents'] = file
                    continue

            # Align the columns
            if value_to_ignore in df_list[page_number].iloc[0].values:
                value_to_ignore_column = df_list[page_number].iloc[0].values.tolist().index(value_to_ignore)
                print(f'\tRemoving {value_to_ignore} from column {value_to_ignore_column}')
                df_list[page_number].iloc[0, value_to_ignore_column:-1] = df_list[page_number].iloc[0, value_to_ignore_column+1:].values

            for column in blank_columns:
                try:
                    if df_list[page_number][column].isna().sum() != len(df_list[page_number]): # 2023-03-30 22:45:
                        blank_column_index = df_list[page_number].columns.tolist().index(column)
                        print(f'\tShifting data from column {blank_column_index}({column})')
                        df_list[page_number].iloc[:,blank_column_index+1:] = df_list[page_number].iloc[:,blank_column_index:-1].values 
                        df_list[page_number].iloc[:,blank_column_index] = None
                except:
                    pass
            try:
                if n_cols_to_pad: # SH 2023-04-01 21:06
                    n_cols_to_drop = n_cols_to_pad - 1
                    column_after_blank_columns_index = blank_column_index + 1
                    print(f'\tShifting data {n_cols_to_drop} to the right until column {column_after_blank_columns_index}')
                    df_list[page_number].iloc[:,column_after_blank_columns_index:-n_cols_to_drop] = df_list[
                        page_number].iloc[:,column_after_blank_columns_index + n_cols_to_drop:].values 
                    df_list[page_number].drop(columns=new_columns_padding, inplace=True)
            except:
                pass

            file_dfs.append(df_list[page_number]) # 2023-03-30 23:04 Decrease indent
        dfs_list.append(file_dfs) # SH 2023-04-01 20:58
        
        # try:
            # final_file_df = pd.concat(file_dfs)
            # final_file_df['file'] = file
            # final_file_df.columns = final_file_df.columns.str.replace('cosed', 'closed')
            # final_file_df.reset_index(inplace=True, drop=True)
            # final_file_df.rename(columns={'Unnamed: 0': 'index'}, inplace=True)
            # dfs_list.append(final_file_df)

        #     print('\n\tDone')
        # except:
        #     print(f'Error concatenating dataframes from {file}')
        #     return file_dfs, {file: 'DataFrame concatenation error'}
    # unique_cols_dict = {}
    # for i, df in enumerate(dfs_list):
    #     other_dfs = dfs_list[:i] + dfs_list[i+1:]
    #     other_cols = set().union(*[set(other_df.columns) for other_df in other_dfs])
    #     unique_columns = set(df.columns) - other_cols
    #     unique_cols_dict[file] = unique_columns if len(unique_columns)>0 else "none"
    #     print(f'Columns unique to part{pdf_filenames[i]}: {unique_cols_dict[file]}')
    # reference_dict['corrected pages'] = corrected_pages
    # reference_dict['unique columns'] = unique_cols_dict
    # if filename:
    #     try:
    #         savepickle(dfs_list, filename=filename, path=path)
    #         with open(f'{filepath}{filename}_reference.json','w') as json_file:
    #             json.dump(reference_dict, json_file)
    #     except:
    #         print('Unable to save outputs')

    return dfs_list, reference_dict




def parse_pdf(pdf_filenames,pages='all', last_id_column='HA', value_to_ignore='s.22(1)', 
    value_to_ignore_start_index=1, value_to_ignore_end_index=None,
    blank_columns=['workplace_name', 'workplace_address'],
    filename=None,filepath=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\BC-Covid-workplace-closures\data\raw', 
    path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\BC-Covid-workplace-closures\data\interim'):
    """
    Create dataframe from a PDF file using 
    """
    print(f'Extracting the following files:')
    if type(pdf_filenames) != list:
        pdf_filenames = [pdf_filenames]
    dfs_list = []
    reference_dict = dict()
    corrected_pages = dict()
    for index, file in enumerate(pdf_filenames):
        print(file)
        filepath = f'{filepath}/'.replace('\\','/') if filepath else ''
        path_and_file = filepath+file
        try:
            df_list = read_pdf(path_and_file, pages=pages)
        except:
            print(f'\nError parsing {file}')
        file_dfs = []
        page1_columns = df_list[0].columns.str.replace('\r', '').str.replace(r'(workplace)([a-zA-Z]+)', r'\1_\2', regex=True).to_list()
        print('number of columns:', len(page1_columns))
        if last_id_column in page1_columns:
            page1_columns_sorted = sorted(page1_columns)
        else:
            previous_columns = dfs_list[index-1].columns
            print(f'\t\tNo headers found; using headers from previous document:', previous_columns)
            if len(df_list[page_number].columns) > len(previous_columns):
                n_cols_to_pad = len(df_list[page_number].columns) - len(previous_columns)
                new_columns_padding = [f'blank{index}' for index, col in enumerate(range(n_cols_to_pad))]
                page1_columns_sorted = sorted(page1_columns) + new_columns_padding
                page1_columns = page1_columns + new_columns_padding

            elif len(df_list[page_number].columns) < len(previous_columns):
                page1_columns = (previous_columns[:len(df_list[page_number].columns)].to_list())
                page1_columns_sorted = sorted(page1_columns)
            else:
                page1_columns_sorted = (dfs_list[index-1].columns.to_list())
            page1_columns = ['Unnamed: 0' if column=='index' else column for column in page1_columns]
            print(f'\t\t**Current df columns:',page1_columns)
        
        for page_number in range(len(df_list)):
            print(f'\tpage {page_number+1}...')
            df_list[page_number].columns = df_list[page_number].columns.str.replace('\r', '').str.replace(r'(workplace)([a-zA-Z]+)', r'\1_\2', regex=True)
            # print('\t\tCurrent number of columns:', len(df_list[page_number].columns), len(page1_columns_sorted))
            # print(f'\t\t{len(page1_columns_sorted)}, {len(page1_columns)}')
            if len(df_list[page_number].columns.to_list()) == len(page1_columns_sorted):
                if sorted(df_list[page_number].columns.to_list()) == page1_columns_sorted:
                    pass
                else:
                    df_list[page_number].columns = page1_columns 
                    print('\t\tKeeping page 1 columns')
            else:
                if len(df_list[page_number].columns) > len(page1_columns_sorted):
                    n_cols_to_pad = len(df_list[page_number].columns) - len(page1_columns_sorted)
                    new_columns_padding = [f'blank{index}' for index, col in enumerate(range(n_cols_to_pad))]
                    df_list[page_number].columns = page1_columns + new_columns_padding
                    print('\t\tPadding columns')
                elif len(df_list[page_number].columns) < len(page1_columns_sorted):
                    df_list[page_number].columns = page1_columns[:len(df_list[page_number].columns)]
                    print('\t\tDropping columns')
                else:
                    df_list[page_number].columns = page1_columns
                corrected_pages[file] = page_number+1
                print(f'\n\t**Fixing column parsing for page {page_number+1} of {file}**')
                print(f'\t\t{df_list[page_number].columns}')
            # Consolidate rows that were incorrectly split
            if (df_list[page_number]['Unnamed: 0'].isna().sum() > 0):
                # print('\tAttempting to consolidate rows that were incorrectly split: ', df_list[page_number].columns)
                print('\tAttempting to consolidate rows that were incorrectly split: ')
                try:
                    df_list[page_number] = merge_rows(df_list[page_number], file)
                except:
                    print(f'\t\tUnable to consolidate rows on page {page_number+1} of {file}')
                    reference_dict['unparsed documents'] = file
                    continue

            # Align the columns
            value_to_ignore_regex = rf'.*{re.escape(value_to_ignore[value_to_ignore_start_index:value_to_ignore_end_index])}.*'
            if df_list[page_number].iloc[0].str.contains(value_to_ignore_regex).any():
                row_values = df_list[page_number].fillna(0).iloc[0].values.tolist()
                for index, item in enumerate(row_values):
                    if re.match(value_to_ignore_regex, str(item)):
                        # print('found regex')
                        value_to_ignore_column = index
                        break

                print(f'\tRemoving {value_to_ignore} from column {value_to_ignore_column}')
                df_list[page_number].iloc[0, value_to_ignore_column:-1] = df_list[page_number].iloc[0, value_to_ignore_column+1:].values

            for column in blank_columns:
                try:
                    if df_list[page_number][column].isna().sum() != len(df_list[page_number]): # 2023-03-30 22:45:
                        blank_column_index = df_list[page_number].columns.tolist().index(column)
                        print(f'\tShifting data from column {blank_column_index}({column})')
                        df_list[page_number].iloc[:,blank_column_index+1:] = df_list[page_number].iloc[:,blank_column_index:-1].values 
                        df_list[page_number].iloc[:,blank_column_index] = None
                except:
                    pass
            df_list[page_number]['doc_page_number'] = page_number + 1 # 2023-03-30 23:15
            file_dfs.append(df_list[page_number])  # 2023-03-30 23:04 Decrease indent
        
        try:
            final_file_df = pd.concat(file_dfs)
            final_file_df['file'] = file
            final_file_df.columns = final_file_df.columns.str.replace('cosed', 'closed')
            final_file_df.reset_index(inplace=True, drop=True)
            final_file_df.rename(columns={'Unnamed: 0': 'index'}, inplace=True)
            dfs_list.append(final_file_df)

            print('\n\tDone')
        except:
            print(f'Error concatenating dataframes from {file}')
            return file_dfs, {file: 'DataFrame concatenation error'}
    unique_cols_dict = {}
    for i, df in enumerate(dfs_list):
        other_dfs = dfs_list[:i] + dfs_list[i+1:]
        other_cols = set().union(*[set(other_df.columns) for other_df in other_dfs])
        unique_columns = set(df.columns) - other_cols
        unique_cols_dict[file] = unique_columns if len(unique_columns)>0 else "none"
        print(f'Columns unique to part{pdf_filenames[i]}: {unique_cols_dict[file]}')
    reference_dict['corrected pages'] = corrected_pages
    reference_dict['unique columns'] = unique_cols_dict
    if filename:
        try:
            savepickle(dfs_list, filename=filename, path=path)
            with open(f'{filepath}{filename}_reference.json','w') as json_file:
                json.dump(reference_dict, json_file)
            save_csv(pd.concat(dfs_list), filename=filename, path=path)
        except:
            print('Unable to save outputs')
    try:
        return pd.concat(dfs_list), reference_dict
    except:
        print('Unable to concatenate dataframes from different documents')
        return dfs_list, reference_dict

n_docs = 7
start = 1
filenames = [f'F22-1445 Records_Part{i}.pdf' for i in range(start,start+n_docs)]
part = 1
filename = f'F22-1445 Records_Part{part}.pdf'

dfs_list1_i2, reference1_i2 = parse_pdf_tolist(filename, pages='1,118', filename=None)
# dfs, reference = parse_pdf(filenames, pages='all', filename='pdfs_parsed_2023-03-30')


Extracting the following files:
F22-1445 Records_Part1.pdf
number of columns: 22
	page 1...
		Current number of columns: 22 22
		22, 22
	Shifting data from column 2(workplace_name)
	Shifting data from column 3(workplace_address)
	page 2...
		Current number of columns: 25 22
		22, 22
		Padding columns

	**Fixing column parsing for page 2 of F22-1445 Records_Part1.pdf**
		Index(['Unnamed: 0', 'HA', 'workplace_name', 'workplace_address', 'city',
       'date_cluster_confirmed_public_health', 'date_cluster_no_longer_active',
       'earliest_reported_cases', 'latest_reported_cases',
       'workplace_classification', 'cases_when_cluster_confirmed',
       'total_cases', 'workplace_status', 'closure_y_n', 'closed_by_order',
       'voluntary_closure', 'closure_date', 'partially_cosed', 'open',
       'days_since_last_case', 'cluster_confirmed_epidate',
       'cluster_closed_epidate', 'blank0', 'blank1', 'blank2'],
      dtype='object')
	Attempting to consolidate rows that were incorrectly 

In [38]:
dfs_list1_i2[0][1]

,Unnamed: 0,HA,workplace_name,workplace_address,city,date_cluster_confirmed_public_health,date_cluster_no_longer_active,earliest_reported_cases,latest_reported_cases,workplace_classification,...,workplace_status,closure_y_n,closed_by_order,voluntary_closure,closure_date,partially_cosed,open,days_since_last_case,cluster_confirmed_epidate,cluster_closed_epidate
0,455.0,FHA,None,None,SURREY,5/17/2021,5/28/2021,None,None,n,...,measures,No,No,Unknown,None,No,Yes,None,No,20-2021
2,456.0,FHA,None,None,ABBOTSFORD,5/17/2021,5/28/2021,None,None,uction,...,n,No,No,Unknown,None,No,Yes,None,No,20-2021
3,457.0,FHA,None,None,SURREY,5/17/2021,6/3/2021,None,None,Office,...,open,No,No,Unknown,None,No,Yes,None,No,20-2021
4,458.0,FHA,None,None,MAPLE RIDGE,5/17/2021,6/2/2021,None,None,g/Industry,...,Closed,Yes,Yes,Unknown,5/18/2021,No,No,None,No,20-2021
5,459.0,FHA,None,None,SURREY,5/17/2021,5/31/2021,None,None,n,...,n,No,No,Unknown,None,No,Yes,None,No,20-2021
6,460.0,FHA,None,None,DELTA,5/17/2021,5/27/2021,None,None,g/Industry,...,open,No,No,Unknown,None,No,Yes,None,No,20-2021
7,461.0,FHA,None,None,WESTMINSTER,5/17/2021,5/25/2021,None,None,Retail,...,n,No,No,Unknown,None,No,Yes,None,No,20-2021
8,462.0,NHA,None,None,Unknown,5/17/2021,None,None,None,Housing,...,closure),No,No,No,None,n,No,None,Yes,20-2021
9,463.0,FHA,None,None,BURNABY,5/18/2021,6/7/2021,None,None,ar/Lounge,...,Closed,Yes,Yes,Unknown,5/19/2021,No,No,None,No,20-2021
10,464.0,VIHA,None,None,Nanaimo,5/18/2021,None,5/16/2021,5/18/2021,uction,...,Open,No,No,No,None,n,Yes,35,No,20-2021


In [39]:
dfs_list1_i2[0][1].columns[0]

'Unnamed: 0'

In [42]:
dfs_list1_i2[0][1]['HA'].isna().sum()

0

## Iteration 4
Previous method won't work because subsquent pages may have valid new data columns

In [57]:
from tabula import read_pdf
import json
import re
import warnings
warnings.simplefilter("ignore", category=FutureWarning)

def merge_rows(df, file, id_column=['Unnamed: 0'], fill_method='bfill', groupby_column=['HA']):
    """
    Merges rows in a pandas dataframe that were parsed incorrectly into multiple rows.
    Assumes that the rows that need to be merged have the same value in a unique ID column.
    """
    print(f'\tRemoving duplicate rows from {file}...')
    print('\t\tOriginal df shape: ', df.shape)
    print(f'\t\tOriginal columns:', df.columns)
    df[id_column] = df[id_column].fillna(method=fill_method)
    
    # Group the rows by the unique ID column and aggregate the values in each group

    columns_to_fill = df.drop(columns=groupby_column).columns
    grouped_df = df.groupby(id_column+groupby_column, as_index=False).agg(
        lambda x: ''.join(str(v) for v in x if (type(v)==str) & (x.name in columns_to_fill)))
    grouped_df = grouped_df[grouped_df[groupby_column[0]] != groupby_column[0]]
    # Return the merged dataframe
    print('\t\tTransformed df shape:', grouped_df.shape)
    return grouped_df.replace('', None)



def parse_pdf_tolist(pdf_filenames, filepath=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\BC-Covid-workplace-closures\data\raw', 
    pages='all', last_id_column='HA', value_to_ignore='s.22(1)',
    blank_columns=['workplace_name', 'workplace_address'],
    filename=None,
    path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\BC-Covid-workplace-closures\data\interim'):
    """
    Create dataframe from a PDF file using 
    """
    print(f'Extracting the following files:')
    if type(pdf_filenames) != list:
        pdf_filenames = [pdf_filenames]
    dfs_list = []
    reference_dict = dict()
    corrected_pages = dict()
    for index, file in enumerate(pdf_filenames):
        print(file)
        filepath = f'{filepath}/'.replace('\\','/') if filepath else ''
        path_and_file = filepath+file
        try:
            df_list = read_pdf(path_and_file, pages=pages)
        except:
            print(f'\nError parsing {file}')
        file_dfs = []
        page1_columns = df_list[0].columns.str.replace('\r', '').str.replace(r'(workplace)([a-zA-Z]+)', r'\1_\2', regex=True).to_list()
        print('number of columns:', len(page1_columns))
        if last_id_column in page1_columns:
            page1_columns_sorted = sorted(page1_columns)
        else:
            previous_columns = dfs_list[index-1].columns
            print(f'\t\tNo headers found; using headers from previous document:', previous_columns)
            if len(df_list[page_number].columns) > len(previous_columns):
                n_cols_to_pad = len(df_list[page_number].columns) - len(previous_columns)
                new_columns_padding = [f'blank{index}' for index, col in enumerate(range(n_cols_to_pad))]
                page1_columns_sorted = sorted(page1_columns) + new_columns_padding
                page1_columns = page1_columns + new_columns_padding

            elif len(df_list[page_number].columns) < len(previous_columns):
                page1_columns = (previous_columns[:len(df_list[page_number].columns)].to_list())
                page1_columns_sorted = sorted(page1_columns)
            else:
                page1_columns_sorted = (dfs_list[index-1].columns.to_list())
            page1_columns = ['Unnamed: 0' if column=='index' else column for column in page1_columns]
            print(f'\t\t**Current df columns:',page1_columns)
        
        for page_number in range(len(df_list)):
            print(f'\tpage {page_number+1}...')
            df_list[page_number].columns = df_list[page_number].columns.str.replace('\r', '').str.replace(r'(workplace)([a-zA-Z]+)', r'\1_\2', regex=True)
            print('\t\tCurrent number of columns:', len(df_list[page_number].columns), len(page1_columns_sorted))
            print(f'\t\t{len(page1_columns_sorted)}, {len(page1_columns)}')
            if len(df_list[page_number].columns.to_list()) == len(page1_columns_sorted):
                if sorted(df_list[page_number].columns.to_list()) == page1_columns_sorted:
                    pass
                else:
                    df_list[page_number].columns = page1_columns 
                    print('\t\tKeeping page 1 columns')
            else:
                if len(df_list[page_number].columns) > len(page1_columns_sorted):
                    n_cols_to_pad = len(df_list[page_number].columns) - len(page1_columns_sorted)
                    new_columns_padding = [f'blank{index}' for index, col in enumerate(range(n_cols_to_pad))]
                    df_list[page_number].columns = page1_columns + new_columns_padding
                    print('\t\tPadding columns')
                elif len(df_list[page_number].columns) < len(page1_columns_sorted):
                    df_list[page_number].columns = page1_columns[:len(df_list[page_number].columns)]
                    print('\t\tDropping columns')
                else:
                    df_list[page_number].columns = page1_columns
                corrected_pages[file] = page_number+1
                print(f'\n\t**Fixing column parsing for page {page_number+1} of {file}**')
                print(f'\t\t{df_list[page_number].columns}')
            # Consolidate rows that were incorrectly split
            if (df_list[page_number]['Unnamed: 0'].isna().sum() > 0):
                # print('\tAttempting to consolidate rows that were incorrectly split: ', df_list[page_number].columns)
                print('\tAttempting to consolidate rows that were incorrectly split: ')
                try:
                    df_list[page_number] = merge_rows(df_list[page_number], file)
                except:
                    print(f'\t\tUnable to consolidate rows on page {page_number+1} of {file}')
                    reference_dict['unparsed documents'] = file
                    continue

            # Align the columns
            if value_to_ignore in df_list[page_number].iloc[0].values:
                value_to_ignore_column = df_list[page_number].iloc[0].values.tolist().index(value_to_ignore)
                print(f'\tRemoving {value_to_ignore} from column {value_to_ignore_column}')
                df_list[page_number].iloc[0, value_to_ignore_column:-1] = df_list[page_number].iloc[0, value_to_ignore_column+1:].values

            for column in blank_columns:
                try:
                    if df_list[page_number][column].isna().sum() != len(df_list[page_number]): # 2023-03-30 22:45:
                        blank_column_index = df_list[page_number].columns.tolist().index(column)
                        print(f'\tShifting data from column {blank_column_index}({column})')
                        df_list[page_number].iloc[:,blank_column_index+1:] = df_list[page_number].iloc[:,blank_column_index:-1].values 
                        df_list[page_number].iloc[:,blank_column_index] = None
                except:
                    pass
            column_after_blank_columns_index = blank_column_index + 1
            column_after_blank_columns = df_list[page_number].columns[column_after_blank_columns_index]
            try:
                while df_list[page_number][column_after_blank_columns].isna().sum() == len(df_list[page_number]):
                    print(f'\tShifting data to the right until column {column_after_blank_columns}')
                    df_list[page_number].iloc[:,column_after_blank_columns_index:-1] = df_list[
                        page_number].iloc[:,column_after_blank_columns_index+1:].values 
                    df_list[page_number].drop(columns=df_list[page_number].columns[-1], inplace=True)
            except:
                pass

            file_dfs.append(df_list[page_number]) # 2023-03-30 23:04 Decrease indent
        dfs_list.append(file_dfs) # SH 2023-04-01 20:58
        
        # try:
            # final_file_df = pd.concat(file_dfs)
            # final_file_df['file'] = file
            # final_file_df.columns = final_file_df.columns.str.replace('cosed', 'closed')
            # final_file_df.reset_index(inplace=True, drop=True)
            # final_file_df.rename(columns={'Unnamed: 0': 'index'}, inplace=True)
            # dfs_list.append(final_file_df)

        #     print('\n\tDone')
        # except:
        #     print(f'Error concatenating dataframes from {file}')
        #     return file_dfs, {file: 'DataFrame concatenation error'}
    # unique_cols_dict = {}
    # for i, df in enumerate(dfs_list):
    #     other_dfs = dfs_list[:i] + dfs_list[i+1:]
    #     other_cols = set().union(*[set(other_df.columns) for other_df in other_dfs])
    #     unique_columns = set(df.columns) - other_cols
    #     unique_cols_dict[file] = unique_columns if len(unique_columns)>0 else "none"
    #     print(f'Columns unique to part{pdf_filenames[i]}: {unique_cols_dict[file]}')
    # reference_dict['corrected pages'] = corrected_pages
    # reference_dict['unique columns'] = unique_cols_dict
    # if filename:
    #     try:
    #         savepickle(dfs_list, filename=filename, path=path)
    #         with open(f'{filepath}{filename}_reference.json','w') as json_file:
    #             json.dump(reference_dict, json_file)
    #     except:
    #         print('Unable to save outputs')

    return dfs_list, reference_dict




def parse_pdf(pdf_filenames,pages='all', last_id_column='HA', value_to_ignore='s.22(1)', 
    value_to_ignore_start_index=1, value_to_ignore_end_index=None,
    blank_columns=['workplace_name', 'workplace_address'],
    filename=None,filepath=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\BC-Covid-workplace-closures\data\raw', 
    path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\BC-Covid-workplace-closures\data\interim'):
    """
    Create dataframe from a PDF file using 
    """
    print(f'Extracting the following files:')
    if type(pdf_filenames) != list:
        pdf_filenames = [pdf_filenames]
    dfs_list = []
    reference_dict = dict()
    corrected_pages = dict()
    for index, file in enumerate(pdf_filenames):
        print(file)
        filepath = f'{filepath}/'.replace('\\','/') if filepath else ''
        path_and_file = filepath+file
        try:
            df_list = read_pdf(path_and_file, pages=pages)
        except:
            print(f'\nError parsing {file}')
        file_dfs = []
        page1_columns = df_list[0].columns.str.replace('\r', '').str.replace(r'(workplace)([a-zA-Z]+)', r'\1_\2', regex=True).to_list()
        print('number of columns:', len(page1_columns))
        if last_id_column in page1_columns:
            page1_columns_sorted = sorted(page1_columns)
        else:
            previous_columns = dfs_list[index-1].columns
            print(f'\t\tNo headers found; using headers from previous document:', previous_columns)
            if len(df_list[page_number].columns) > len(previous_columns):
                n_cols_to_pad = len(df_list[page_number].columns) - len(previous_columns)
                new_columns_padding = [f'blank{index}' for index, col in enumerate(range(n_cols_to_pad))]
                page1_columns_sorted = sorted(page1_columns) + new_columns_padding
                page1_columns = page1_columns + new_columns_padding

            elif len(df_list[page_number].columns) < len(previous_columns):
                page1_columns = (previous_columns[:len(df_list[page_number].columns)].to_list())
                page1_columns_sorted = sorted(page1_columns)
            else:
                page1_columns_sorted = (dfs_list[index-1].columns.to_list())
            page1_columns = ['Unnamed: 0' if column=='index' else column for column in page1_columns]
            print(f'\t\t**Current df columns:',page1_columns)
        
        for page_number in range(len(df_list)):
            print(f'\tpage {page_number+1}...')
            df_list[page_number].columns = df_list[page_number].columns.str.replace('\r', '').str.replace(r'(workplace)([a-zA-Z]+)', r'\1_\2', regex=True)
            # print('\t\tCurrent number of columns:', len(df_list[page_number].columns), len(page1_columns_sorted))
            # print(f'\t\t{len(page1_columns_sorted)}, {len(page1_columns)}')
            if len(df_list[page_number].columns.to_list()) == len(page1_columns_sorted):
                if sorted(df_list[page_number].columns.to_list()) == page1_columns_sorted:
                    pass
                else:
                    df_list[page_number].columns = page1_columns 
                    print('\t\tKeeping page 1 columns')
            else:
                if len(df_list[page_number].columns) > len(page1_columns_sorted):
                    n_cols_to_pad = len(df_list[page_number].columns) - len(page1_columns_sorted)
                    new_columns_padding = [f'blank{index}' for index, col in enumerate(range(n_cols_to_pad))]
                    df_list[page_number].columns = page1_columns + new_columns_padding
                    print('\t\tPadding columns')
                elif len(df_list[page_number].columns) < len(page1_columns_sorted):
                    df_list[page_number].columns = page1_columns[:len(df_list[page_number].columns)]
                    print('\t\tDropping columns')
                else:
                    df_list[page_number].columns = page1_columns
                corrected_pages[file] = page_number+1
                print(f'\n\t**Fixing column parsing for page {page_number+1} of {file}**')
                print(f'\t\t{df_list[page_number].columns}')
            # Consolidate rows that were incorrectly split
            if (df_list[page_number]['Unnamed: 0'].isna().sum() > 0):
                # print('\tAttempting to consolidate rows that were incorrectly split: ', df_list[page_number].columns)
                print('\tAttempting to consolidate rows that were incorrectly split: ')
                try:
                    df_list[page_number] = merge_rows(df_list[page_number], file)
                except:
                    print(f'\t\tUnable to consolidate rows on page {page_number+1} of {file}')
                    reference_dict['unparsed documents'] = file
                    continue

            # Align the columns
            value_to_ignore_regex = rf'.*{re.escape(value_to_ignore[value_to_ignore_start_index:value_to_ignore_end_index])}.*'
            if df_list[page_number].iloc[0].str.contains(value_to_ignore_regex).any():
                row_values = df_list[page_number].fillna(0).iloc[0].values.tolist()
                for index, item in enumerate(row_values):
                    if re.match(value_to_ignore_regex, str(item)):
                        # print('found regex')
                        value_to_ignore_column = index
                        break

                print(f'\tRemoving {value_to_ignore} from column {value_to_ignore_column}')
                df_list[page_number].iloc[0, value_to_ignore_column:-1] = df_list[page_number].iloc[0, value_to_ignore_column+1:].values

            for column in blank_columns:
                try:
                    if df_list[page_number][column].isna().sum() != len(df_list[page_number]): # 2023-03-30 22:45:
                        blank_column_index = df_list[page_number].columns.tolist().index(column)
                        print(f'\tShifting data from column {blank_column_index}({column})')
                        df_list[page_number].iloc[:,blank_column_index+1:] = df_list[page_number].iloc[:,blank_column_index:-1].values 
                        df_list[page_number].iloc[:,blank_column_index] = None
                except:
                    pass
            df_list[page_number]['doc_page_number'] = page_number + 1 # 2023-03-30 23:15
            file_dfs.append(df_list[page_number])  # 2023-03-30 23:04 Decrease indent
        
        try:
            final_file_df = pd.concat(file_dfs)
            final_file_df['file'] = file
            final_file_df.columns = final_file_df.columns.str.replace('cosed', 'closed')
            final_file_df.reset_index(inplace=True, drop=True)
            final_file_df.rename(columns={'Unnamed: 0': 'index'}, inplace=True)
            dfs_list.append(final_file_df)

            print('\n\tDone')
        except:
            print(f'Error concatenating dataframes from {file}')
            return file_dfs, {file: 'DataFrame concatenation error'}
    unique_cols_dict = {}
    for i, df in enumerate(dfs_list):
        other_dfs = dfs_list[:i] + dfs_list[i+1:]
        other_cols = set().union(*[set(other_df.columns) for other_df in other_dfs])
        unique_columns = set(df.columns) - other_cols
        unique_cols_dict[file] = unique_columns if len(unique_columns)>0 else "none"
        print(f'Columns unique to part{pdf_filenames[i]}: {unique_cols_dict[file]}')
    reference_dict['corrected pages'] = corrected_pages
    reference_dict['unique columns'] = unique_cols_dict
    if filename:
        try:
            savepickle(dfs_list, filename=filename, path=path)
            with open(f'{filepath}{filename}_reference.json','w') as json_file:
                json.dump(reference_dict, json_file)
            save_csv(pd.concat(dfs_list), filename=filename, path=path)
        except:
            print('Unable to save outputs')
    try:
        return pd.concat(dfs_list), reference_dict
    except:
        print('Unable to concatenate dataframes from different documents')
        return dfs_list, reference_dict

n_docs = 7
start = 1
filenames = [f'F22-1445 Records_Part{i}.pdf' for i in range(start,start+n_docs)]
part = 1
filename = f'F22-1445 Records_Part{part}.pdf'

dfs_list1_i2, reference1_i2 = parse_pdf_tolist(filename, pages='1,118', filename=None)
# dfs, reference = parse_pdf(filenames, pages='all', filename='pdfs_parsed_2023-03-30')


Extracting the following files:
F22-1445 Records_Part1.pdf
number of columns: 22
	page 1...
		Current number of columns: 22 22
		22, 22
	Shifting data from column 2(workplace_name)
	Shifting data from column 3(workplace_address)
	page 2...
		Current number of columns: 25 22
		22, 22
		Padding columns

	**Fixing column parsing for page 2 of F22-1445 Records_Part1.pdf**
		Index(['Unnamed: 0', 'HA', 'workplace_name', 'workplace_address', 'city',
       'date_cluster_confirmed_public_health', 'date_cluster_no_longer_active',
       'earliest_reported_cases', 'latest_reported_cases',
       'workplace_classification', 'cases_when_cluster_confirmed',
       'total_cases', 'workplace_status', 'closure_y_n', 'closed_by_order',
       'voluntary_closure', 'closure_date', 'partially_cosed', 'open',
       'days_since_last_case', 'cluster_confirmed_epidate',
       'cluster_closed_epidate', 'blank0', 'blank1', 'blank2'],
      dtype='object')
	Attempting to consolidate rows that were incorrectly 

In [58]:
dfs_list1_i2[0][1]

,Unnamed: 0,HA,workplace_name,workplace_address,city,date_cluster_confirmed_public_health,date_cluster_no_longer_active,earliest_reported_cases,latest_reported_cases,workplace_classification,...,closure_y_n,closed_by_order,voluntary_closure,closure_date,partially_cosed,open,days_since_last_case,cluster_confirmed_epidate,cluster_closed_epidate,blank0
0,455.0,FHA,None,None,SURREY,5/17/2021,5/28/2021,None,None,n,...,No,No,Unknown,None,No,Yes,None,No,20-2021,None
2,456.0,FHA,None,None,ABBOTSFORD,5/17/2021,5/28/2021,None,None,uction,...,No,No,Unknown,None,No,Yes,None,No,20-2021,None
3,457.0,FHA,None,None,SURREY,5/17/2021,6/3/2021,None,None,Office,...,No,No,Unknown,None,No,Yes,None,No,20-2021,None
4,458.0,FHA,None,None,MAPLE RIDGE,5/17/2021,6/2/2021,None,None,g/Industry,...,Yes,Yes,Unknown,5/18/2021,No,No,None,No,20-2021,20-2021
5,459.0,FHA,None,None,SURREY,5/17/2021,5/31/2021,None,None,n,...,No,No,Unknown,None,No,Yes,None,No,20-2021,None
6,460.0,FHA,None,None,DELTA,5/17/2021,5/27/2021,None,None,g/Industry,...,No,No,Unknown,None,No,Yes,None,No,20-2021,None
7,461.0,FHA,None,None,WESTMINSTER,5/17/2021,5/25/2021,None,None,Retail,...,No,No,Unknown,None,No,Yes,None,No,20-2021,None
8,462.0,NHA,None,None,Unknown,5/17/2021,None,None,None,Housing,...,No,No,No,None,n,No,None,Yes,20-2021,None
9,463.0,FHA,None,None,BURNABY,5/18/2021,6/7/2021,None,None,ar/Lounge,...,Yes,Yes,Unknown,5/19/2021,No,No,None,No,20-2021,20-2021
10,464.0,VIHA,None,None,Nanaimo,5/18/2021,None,5/16/2021,5/18/2021,uction,...,No,No,No,None,n,Yes,35,No,20-2021,None


In [62]:
set(dfs_list1_i2[0][1].columns) - set(dfs_list1_i2[0][0].columns) 

{'blank0'}

## Iteration 5

In [59]:
from tabula import read_pdf
import json
import re
import warnings
warnings.simplefilter("ignore", category=FutureWarning)

def merge_rows(df, file, id_column=['Unnamed: 0'], fill_method='bfill', groupby_column=['HA']):
    """
    Merges rows in a pandas dataframe that were parsed incorrectly into multiple rows.
    Assumes that the rows that need to be merged have the same value in a unique ID column.
    """
    print(f'\tRemoving duplicate rows from {file}...')
    print('\t\tOriginal df shape: ', df.shape)
    print(f'\t\tOriginal columns:', df.columns)
    df[id_column] = df[id_column].fillna(method=fill_method)
    
    # Group the rows by the unique ID column and aggregate the values in each group

    columns_to_fill = df.drop(columns=groupby_column).columns
    grouped_df = df.groupby(id_column+groupby_column, as_index=False).agg(
        lambda x: ''.join(str(v) for v in x if (type(v)==str) & (x.name in columns_to_fill)))
    grouped_df = grouped_df[grouped_df[groupby_column[0]] != groupby_column[0]]
    # Return the merged dataframe
    print('\t\tTransformed df shape:', grouped_df.shape)
    return grouped_df.replace('', None)



def parse_pdf_tolist(pdf_filenames, filepath=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\BC-Covid-workplace-closures\data\raw', 
    pages='all', last_id_column='HA', value_to_ignore='s.22(1)',
    blank_columns=['workplace_name', 'workplace_address'],
    filename=None,
    path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\BC-Covid-workplace-closures\data\interim'):
    """
    Create dataframe from a PDF file using 
    """
    print(f'Extracting the following files:')
    if type(pdf_filenames) != list:
        pdf_filenames = [pdf_filenames]
    dfs_list = []
    reference_dict = dict()
    corrected_pages = dict()
    for index, file in enumerate(pdf_filenames):
        print(file)
        filepath = f'{filepath}/'.replace('\\','/') if filepath else ''
        path_and_file = filepath+file
        try:
            df_list = read_pdf(path_and_file, pages=pages)
        except:
            print(f'\nError parsing {file}')
        file_dfs = []
        page1_columns = df_list[0].columns.str.replace('\r', '').str.replace(r'(workplace)([a-zA-Z]+)', r'\1_\2', regex=True).to_list()
        print('number of columns:', len(page1_columns))
        if last_id_column in page1_columns:
            page1_columns_sorted = sorted(page1_columns)
        else:
            previous_columns = dfs_list[index-1].columns
            print(f'\t\tNo headers found; using headers from previous document:', previous_columns)
            if len(df_list[page_number].columns) > len(previous_columns):
                n_cols_to_pad = len(df_list[page_number].columns) - len(previous_columns)
                new_columns_padding = [f'blank{index}' for index, col in enumerate(range(n_cols_to_pad))]
                page1_columns_sorted = sorted(page1_columns) + new_columns_padding
                page1_columns = page1_columns + new_columns_padding

            elif len(df_list[page_number].columns) < len(previous_columns):
                page1_columns = (previous_columns[:len(df_list[page_number].columns)].to_list())
                page1_columns_sorted = sorted(page1_columns)
            else:
                page1_columns_sorted = (dfs_list[index-1].columns.to_list())
            page1_columns = ['Unnamed: 0' if column=='index' else column for column in page1_columns]
            print(f'\t\t**Current df columns:',page1_columns)
        
        for page_number in range(len(df_list)):
            print(f'\tpage {page_number+1}...')
            df_list[page_number].columns = df_list[page_number].columns.str.replace('\r', '').str.replace(r'(workplace)([a-zA-Z]+)', r'\1_\2', regex=True)
            print('\t\tCurrent number of columns:', len(df_list[page_number].columns), len(page1_columns_sorted))
            print(f'\t\t{len(page1_columns_sorted)}, {len(page1_columns)}')
            if len(df_list[page_number].columns.to_list()) == len(page1_columns_sorted):
                if sorted(df_list[page_number].columns.to_list()) == page1_columns_sorted:
                    pass
                else:
                    df_list[page_number].columns = page1_columns 
                    print('\t\tKeeping page 1 columns')
            else:
                if len(df_list[page_number].columns) > len(page1_columns_sorted):
                    n_cols_to_pad = len(df_list[page_number].columns) - len(page1_columns_sorted)
                    new_columns_padding = [f'blank{index}' for index, col in enumerate(range(n_cols_to_pad))]
                    df_list[page_number].columns = page1_columns + new_columns_padding
                    print('\t\tPadding columns')
                elif len(df_list[page_number].columns) < len(page1_columns_sorted):
                    df_list[page_number].columns = page1_columns[:len(df_list[page_number].columns)]
                    print('\t\tDropping columns')
                else:
                    df_list[page_number].columns = page1_columns
                corrected_pages[file] = page_number+1
                print(f'\n\t**Fixing column parsing for page {page_number+1} of {file}**')
                print(f'\t\t{df_list[page_number].columns}')
            # Consolidate rows that were incorrectly split
            if (df_list[page_number]['Unnamed: 0'].isna().sum() > 0):
                # print('\tAttempting to consolidate rows that were incorrectly split: ', df_list[page_number].columns)
                print('\tAttempting to consolidate rows that were incorrectly split: ')
                try:
                    df_list[page_number] = merge_rows(df_list[page_number], file)
                except:
                    print(f'\t\tUnable to consolidate rows on page {page_number+1} of {file}')
                    reference_dict['unparsed documents'] = file
                    continue

            # Align the columns
            if value_to_ignore in df_list[page_number].iloc[0].values:
                value_to_ignore_column = df_list[page_number].iloc[0].values.tolist().index(value_to_ignore)
                print(f'\tRemoving {value_to_ignore} from column {value_to_ignore_column}')
                df_list[page_number].iloc[0, value_to_ignore_column:-1] = df_list[page_number].iloc[0, value_to_ignore_column+1:].values

            for column in blank_columns:
                try:
                    if df_list[page_number][column].isna().sum() != len(df_list[page_number]): # 2023-03-30 22:45:
                        blank_column_index = df_list[page_number].columns.tolist().index(column)
                        print(f'\tShifting data from column {blank_column_index}({column})')
                        df_list[page_number].iloc[:,blank_column_index+1:] = df_list[page_number].iloc[:,blank_column_index:-1].values 
                        df_list[page_number].iloc[:,blank_column_index] = None
                except:
                    pass
            column_after_blank_columns_index = blank_column_index + 1
            column_after_blank_columns = df_list[page_number].columns[column_after_blank_columns_index]
            try:
                while df_list[page_number][column_after_blank_columns].isna().sum() == len(df_list[page_number]):
                    print(f'\tShifting data to the right until `{column_after_blank_columns}` column')
                    df_list[page_number].iloc[:,column_after_blank_columns_index:-1] = df_list[
                        page_number].iloc[:,column_after_blank_columns_index+1:].values 
                    df_list[page_number].drop(columns=df_list[page_number].columns[-1], inplace=True)
            except:
                pass

            file_dfs.append(df_list[page_number]) # 2023-03-30 23:04 Decrease indent
        dfs_list.append(file_dfs) # SH 2023-04-01 20:58
        
        # try:
            # final_file_df = pd.concat(file_dfs)
            # final_file_df['file'] = file
            # final_file_df.columns = final_file_df.columns.str.replace('cosed', 'closed')
            # final_file_df.reset_index(inplace=True, drop=True)
            # final_file_df.rename(columns={'Unnamed: 0': 'index'}, inplace=True)
            # dfs_list.append(final_file_df)

        #     print('\n\tDone')
        # except:
        #     print(f'Error concatenating dataframes from {file}')
        #     return file_dfs, {file: 'DataFrame concatenation error'}
    # unique_cols_dict = {}
    # for i, df in enumerate(dfs_list):
    #     other_dfs = dfs_list[:i] + dfs_list[i+1:]
    #     other_cols = set().union(*[set(other_df.columns) for other_df in other_dfs])
    #     unique_columns = set(df.columns) - other_cols
    #     unique_cols_dict[file] = unique_columns if len(unique_columns)>0 else "none"
    #     print(f'Columns unique to part{pdf_filenames[i]}: {unique_cols_dict[file]}')
    # reference_dict['corrected pages'] = corrected_pages
    # reference_dict['unique columns'] = unique_cols_dict
    # if filename:
    #     try:
    #         savepickle(dfs_list, filename=filename, path=path)
    #         with open(f'{filepath}{filename}_reference.json','w') as json_file:
    #             json.dump(reference_dict, json_file)
    #     except:
    #         print('Unable to save outputs')

    return dfs_list, reference_dict




def parse_pdf(pdf_filenames,pages='all', last_id_column='HA', value_to_ignore='s.22(1)', 
    value_to_ignore_start_index=1, value_to_ignore_end_index=None,
    blank_columns=['workplace_name', 'workplace_address'],
    filename=None,filepath=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\BC-Covid-workplace-closures\data\raw', 
    path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\BC-Covid-workplace-closures\data\interim'):
    """
    Create dataframe from a PDF file using 
    """
    print(f'Extracting the following files:')
    if type(pdf_filenames) != list:
        pdf_filenames = [pdf_filenames]
    dfs_list = []
    reference_dict = dict()
    corrected_pages = dict()
    for index, file in enumerate(pdf_filenames):
        print(file)
        filepath = f'{filepath}/'.replace('\\','/') if filepath else ''
        path_and_file = filepath+file
        try:
            df_list = read_pdf(path_and_file, pages=pages)
        except:
            print(f'\nError parsing {file}')
        file_dfs = []
        page1_columns = df_list[0].columns.str.replace('\r', '').str.replace(r'(workplace)([a-zA-Z]+)', r'\1_\2', regex=True).to_list()
        print('number of columns:', len(page1_columns))
        if last_id_column in page1_columns:
            page1_columns_sorted = sorted(page1_columns)
        else:
            previous_columns = dfs_list[index-1].columns
            print(f'\t\tNo headers found; using headers from previous document:', previous_columns)
            if len(df_list[page_number].columns) > len(previous_columns):
                n_cols_to_pad = len(df_list[page_number].columns) - len(previous_columns)
                new_columns_padding = [f'blank{index}' for index, col in enumerate(range(n_cols_to_pad))]
                page1_columns_sorted = sorted(page1_columns) + new_columns_padding
                page1_columns = page1_columns + new_columns_padding

            elif len(df_list[page_number].columns) < len(previous_columns):
                page1_columns = (previous_columns[:len(df_list[page_number].columns)].to_list())
                page1_columns_sorted = sorted(page1_columns)
            else:
                page1_columns_sorted = (dfs_list[index-1].columns.to_list())
            page1_columns = ['Unnamed: 0' if column=='index' else column for column in page1_columns]
            print(f'\t\t**Current df columns:',page1_columns)
        
        for page_number in range(len(df_list)):
            print(f'\tpage {page_number+1}...')
            df_list[page_number].columns = df_list[page_number].columns.str.replace('\r', '').str.replace(r'(workplace)([a-zA-Z]+)', r'\1_\2', regex=True)
            if len(df_list[page_number].columns.to_list()) == len(page1_columns_sorted):
                if sorted(df_list[page_number].columns.to_list()) == page1_columns_sorted:
                    pass
                else:
                    df_list[page_number].columns = page1_columns 
                    print('\t\tKeeping page 1 columns')
            else:
                if len(df_list[page_number].columns) > len(page1_columns_sorted):
                    n_cols_to_pad = len(df_list[page_number].columns) - len(page1_columns_sorted)
                    new_columns_padding = [f'blank{index}' for index, col in enumerate(range(n_cols_to_pad))]
                    df_list[page_number].columns = page1_columns + new_columns_padding
                    print('\t\tPadding columns')
                elif len(df_list[page_number].columns) < len(page1_columns_sorted):
                    df_list[page_number].columns = page1_columns[:len(df_list[page_number].columns)]
                    print('\t\tDropping columns')
                else:
                    df_list[page_number].columns = page1_columns
                corrected_pages[file] = page_number+1
                print(f'\n\t**Fixing column parsing for page {page_number+1} of {file}**')
                print(f'\t\t{df_list[page_number].columns}')
            # Consolidate rows that were incorrectly split
            if (df_list[page_number]['Unnamed: 0'].isna().sum() > 0):
                # print('\tAttempting to consolidate rows that were incorrectly split: ', df_list[page_number].columns)
                print('\tAttempting to consolidate rows that were incorrectly split: ')
                try:
                    df_list[page_number] = merge_rows(df_list[page_number], file)
                except:
                    print(f'\t\tUnable to consolidate rows on page {page_number+1} of {file}')
                    reference_dict['unparsed documents'] = file
                    continue

            # Align the columns
            value_to_ignore_regex = rf'.*{re.escape(value_to_ignore[value_to_ignore_start_index:value_to_ignore_end_index])}.*'
            if df_list[page_number].iloc[0].str.contains(value_to_ignore_regex).any():
                row_values = df_list[page_number].fillna(0).iloc[0].values.tolist()
                for index, item in enumerate(row_values):
                    if re.match(value_to_ignore_regex, str(item)):
                        # print('found regex')
                        value_to_ignore_column = index
                        break

                print(f'\tRemoving {value_to_ignore} from column {value_to_ignore_column}')
                df_list[page_number].iloc[0, value_to_ignore_column:-1] = df_list[page_number].iloc[0, value_to_ignore_column+1:].values

            for column in blank_columns:
                try:
                    if df_list[page_number][column].isna().sum() != len(df_list[page_number]): # 2023-03-30 22:45:
                        blank_column_index = df_list[page_number].columns.tolist().index(column)
                        print(f'\tShifting data from column {blank_column_index}({column})')
                        df_list[page_number].iloc[:,blank_column_index+1:] = df_list[page_number].iloc[:,blank_column_index:-1].values 
                        df_list[page_number].iloc[:,blank_column_index] = None
                except:
                    pass
            column_after_blank_columns_index = blank_column_index + 1
            column_after_blank_columns = df_list[page_number].columns[column_after_blank_columns_index]
            try:
                while df_list[page_number][column_after_blank_columns].isna().sum() == len(df_list[page_number]):
                    print(f'\tShifting data to the right until `{column_after_blank_columns}` column')
                    df_list[page_number].iloc[:,column_after_blank_columns_index:-1] = df_list[
                        page_number].iloc[:,column_after_blank_columns_index+1:].values 
                    df_list[page_number].drop(columns=df_list[page_number].columns[-1], inplace=True)
            except:
                pass
            df_list[page_number]['doc_page_number'] = page_number + 1 # 2023-03-30 23:15
            file_dfs.append(df_list[page_number])  # 2023-03-30 23:04 Decrease indent
        
        try:
            final_file_df = pd.concat(file_dfs)
            final_file_df['file'] = file
            final_file_df.columns = final_file_df.columns.str.replace('cosed', 'closed')
            final_file_df.reset_index(inplace=True, drop=True)
            final_file_df.rename(columns={'Unnamed: 0': 'index'}, inplace=True)
            dfs_list.append(final_file_df)

            print('\n\tDone')
        except:
            print(f'Error concatenating dataframes from {file}')
            return file_dfs, {file: 'DataFrame concatenation error'}
    unique_cols_dict = {}
    for i, df in enumerate(dfs_list):
        other_dfs = dfs_list[:i] + dfs_list[i+1:]
        other_cols = set().union(*[set(other_df.columns) for other_df in other_dfs])
        unique_columns = set(df.columns) - other_cols
        unique_cols_dict[file] = unique_columns if len(unique_columns)>0 else "none"
        print(f'Columns unique to part{pdf_filenames[i]}: {unique_cols_dict[file]}')
    reference_dict['corrected pages'] = corrected_pages
    reference_dict['unique columns'] = unique_cols_dict
    if filename:
        try:
            savepickle(dfs_list, filename=filename, path=path)
            with open(f'{filepath}{filename}_reference.json','w') as json_file:
                json.dump(reference_dict, json_file)
            save_csv(pd.concat(dfs_list), filename=filename, path=path)
        except:
            print('Unable to save outputs')
    try:
        return pd.concat(dfs_list), reference_dict
    except:
        print('Unable to concatenate dataframes from different documents')
        return dfs_list, reference_dict

n_docs = 7
start = 1
filenames = [f'F22-1445 Records_Part{i}.pdf' for i in range(start,start+n_docs)]
part = 1
filename = f'F22-1445 Records_Part{part}.pdf'

dfs1, reference1_i5 = parse_pdf(filename, pages='1,118', filename=None)
# dfs, reference = parse_pdf(filenames, pages='all', filename='pdfs_parsed_2023-03-30')


Extracting the following files:
F22-1445 Records_Part1.pdf
number of columns: 22
	page 1...
	Shifting data from column 2(workplace_name)
	Shifting data from column 3(workplace_address)
	page 2...
		Padding columns

	**Fixing column parsing for page 2 of F22-1445 Records_Part1.pdf**
		Index(['Unnamed: 0', 'HA', 'workplace_name', 'workplace_address', 'city',
       'date_cluster_confirmed_public_health', 'date_cluster_no_longer_active',
       'earliest_reported_cases', 'latest_reported_cases',
       'workplace_classification', 'cases_when_cluster_confirmed',
       'total_cases', 'workplace_status', 'closure_y_n', 'closed_by_order',
       'voluntary_closure', 'closure_date', 'partially_cosed', 'open',
       'days_since_last_case', 'cluster_confirmed_epidate',
       'cluster_closed_epidate', 'blank0', 'blank1', 'blank2'],
      dtype='object')
	Attempting to consolidate rows that were incorrectly split: 
	Removing duplicate rows from F22-1445 Records_Part1.pdf...
		Original df shape:

In [60]:
dfs1

,index,HA,workplace_name,workplace_address,city,date_cluster_confirmed_public_health,date_cluster_no_longer_active,earliest_reported_cases,latest_reported_cases,workplace_classification,...,voluntary_closure,closure_date,partially_closed,open,days_since_last_case,cluster_confirmed_epidate,cluster_closed_epidate,doc_page_number,blank0,file
0,1.0,FHA,None,None,SURREY,4/1/2021,4/17/2021,NaN,NaN,Restaurant/Bar/L\rounge,...,Unknown,NaN,No,No,NaN,13-2021,NaN,1,NaN,F22-1445 Records_Part1.pdf
1,2.0,FHA,None,None,LANGLEY,4/1/2021,4/13/2021,NaN,NaN,Other,...,Unknown,NaN,No,No,NaN,13-2021,NaN,1,NaN,F22-1445 Records_Part1.pdf
2,3.0,FHA,None,None,SURREY,4/1/2021,4/13/2021,NaN,NaN,Restaurant/Bar/L\rounge,...,Unknown,4/4/2021,No,No,NaN,13-2021,14-2021,1,NaN,F22-1445 Records_Part1.pdf
3,4.0,FHA,None,None,BURNABY,4/1/2021,4/21/2021,NaN,NaN,Gym/Fitness,...,Unknown,4/7/2021,No,No,NaN,13-2021,14-2021,1,NaN,F22-1445 Records_Part1.pdf
4,5.0,FHA,None,None,BURNABY,4/1/2021,4/14/2021,NaN,NaN,Restaurant/Bar/L\rounge,...,Unknown,4/5/2021,No,No,NaN,13-2021,14-2021,1,NaN,F22-1445 Records_Part1.pdf
5,6.0,FHA,None,None,CHILLIWACK,4/1/2021,4/17/2021,NaN,NaN,Maintenance/Tr\rades/Constructio\rn,...,Unknown,NaN,No,No,NaN,13-2021,NaN,1,NaN,F22-1445 Records_Part1.pdf
6,7.0,FHA,None,None,COQUITLAM,4/1/2021,4/14/2021,NaN,NaN,Office,...,Unknown,NaN,No,No,NaN,13-2021,NaN,1,NaN,F22-1445 Records_Part1.pdf
7,8.0,FHA,None,None,DELTA,4/1/2021,4/6/2021,NaN,NaN,Transportation,...,Unknown,NaN,No,No,NaN,13-2021,NaN,1,NaN,F22-1445 Records_Part1.pdf
8,9.0,FHA,None,None,LANGLEY,4/1/2021,4/30/2021,NaN,NaN,Farm/Agriculture,...,Unknown,NaN,No,No,NaN,13-2021,NaN,1,NaN,F22-1445 Records_Part1.pdf
9,10.0,FHA,None,None,LANGLEY,4/1/2021,NaN,NaN,NaN,Warehouse,...,Unknown,NaN,No,No,NaN,13-2021,NaN,1,NaN,F22-1445 Records_Part1.pdf


## Iteration 6
Keep columns if difference from page is only 1 new colum

In [66]:
from tabula import read_pdf
import json
import re
import warnings
warnings.simplefilter("ignore", category=FutureWarning)

def merge_rows(df, file, id_column=['Unnamed: 0'], fill_method='bfill', groupby_column=['HA']):
    """
    Merges rows in a pandas dataframe that were parsed incorrectly into multiple rows.
    Assumes that the rows that need to be merged have the same value in a unique ID column.
    """
    print(f'\tRemoving duplicate rows from {file}...')
    print('\t\tOriginal df shape: ', df.shape)
    print(f'\t\tOriginal columns:', df.columns)
    df[id_column] = df[id_column].fillna(method=fill_method)
    
    # Group the rows by the unique ID column and aggregate the values in each group

    columns_to_fill = df.drop(columns=groupby_column).columns
    grouped_df = df.groupby(id_column+groupby_column, as_index=False).agg(
        lambda x: ''.join(str(v) for v in x if (type(v)==str) & (x.name in columns_to_fill)))
    grouped_df = grouped_df[grouped_df[groupby_column[0]] != groupby_column[0]]
    # Return the merged dataframe
    print('\t\tTransformed df shape:', grouped_df.shape)
    return grouped_df.replace('', None)



def parse_pdf_tolist(pdf_filenames, filepath=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\BC-Covid-workplace-closures\data\raw', 
    pages='all', last_id_column='HA', value_to_ignore='s.22(1)',
    blank_columns=['workplace_name', 'workplace_address'],
    filename=None,
    path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\BC-Covid-workplace-closures\data\interim'):
    """
    Create dataframe from a PDF file using 
    """
    print(f'Extracting the following files:')
    if type(pdf_filenames) != list:
        pdf_filenames = [pdf_filenames]
    dfs_list = []
    reference_dict = dict()
    corrected_pages = dict()
    for index, file in enumerate(pdf_filenames):
        print(file)
        filepath = f'{filepath}/'.replace('\\','/') if filepath else ''
        path_and_file = filepath+file
        try:
            df_list = read_pdf(path_and_file, pages=pages)
        except:
            print(f'\nError parsing {file}')
        file_dfs = []
        page1_columns = df_list[0].columns.str.replace('\r', '').str.replace(r'(workplace)([a-zA-Z]+)', r'\1_\2', regex=True).to_list()
        print('number of columns:', len(page1_columns))
        if last_id_column in page1_columns:
            page1_columns_sorted = sorted(page1_columns)
        else:
            previous_columns = dfs_list[index-1].columns
            print(f'\t\tNo headers found; using headers from previous document:', previous_columns)
            if len(df_list[page_number].columns) > len(previous_columns):
                n_cols_to_pad = len(df_list[page_number].columns) - len(previous_columns)
                new_columns_padding = [f'blank{index}' for index, col in enumerate(range(n_cols_to_pad))]
                page1_columns_sorted = sorted(page1_columns) + new_columns_padding
                page1_columns = page1_columns + new_columns_padding

            elif len(df_list[page_number].columns) < len(previous_columns):
                page1_columns = (previous_columns[:len(df_list[page_number].columns)].to_list())
                page1_columns_sorted = sorted(page1_columns)
            else:
                page1_columns_sorted = (dfs_list[index-1].columns.to_list())
            page1_columns = ['Unnamed: 0' if column=='index' else column for column in page1_columns]
            print(f'\t\t**Current df columns:',page1_columns)
        
        for page_number in range(len(df_list)):
            print(f'\tpage {page_number+1}...')
            df_list[page_number].columns = df_list[page_number].columns.str.replace('\r', '').str.replace(r'(workplace)([a-zA-Z]+)', r'\1_\2', regex=True)
            print('\t\tCurrent number of columns:', len(df_list[page_number].columns), len(page1_columns_sorted))
            print(f'\t\t{len(page1_columns_sorted)}, {len(page1_columns)}')
            if len(df_list[page_number].columns.to_list()) == len(page1_columns_sorted):
                if sorted(df_list[page_number].columns.to_list()) == page1_columns_sorted:
                    pass
                else:
                    df_list[page_number].columns = page1_columns 
                    print('\t\tKeeping page 1 columns')
            else:
                if len(df_list[page_number].columns) > len(page1_columns_sorted):
                    n_cols_to_pad = len(df_list[page_number].columns) - len(page1_columns_sorted)
                    new_columns_padding = [f'blank{index}' for index, col in enumerate(range(n_cols_to_pad))]
                    new_columns = set(df_list[page_number].columns) - set(page1_columns_sorted)
                    print(f'\t\tFound new columns: {new_columns}')
                    if len(new_columns) < len(page1_columns_sorted):
                        df_list[page_number].columns = df_list[page_number].columns + new_columns_padding
                    else:
                        df_list[page_number].columns = page1_columns + new_columns_padding
                    print(f'\t\tPadding {n_cols_to_pad} columns')
                elif len(df_list[page_number].columns) < len(page1_columns_sorted):
                    df_list[page_number].columns = page1_columns[:len(df_list[page_number].columns)]
                    print('\t\tDropping columns')
                else:
                    df_list[page_number].columns = page1_columns
                corrected_pages[file] = page_number+1
                print(f'\n\t**Fixing column parsing for page {page_number+1} of {file}**')
                print(f'\t\t{df_list[page_number].columns}')
            # Consolidate rows that were incorrectly split
            if (df_list[page_number]['Unnamed: 0'].isna().sum() > 0):
                # print('\tAttempting to consolidate rows that were incorrectly split: ', df_list[page_number].columns)
                print('\tAttempting to consolidate rows that were incorrectly split: ')
                try:
                    df_list[page_number] = merge_rows(df_list[page_number], file)
                except:
                    print(f'\t\tUnable to consolidate rows on page {page_number+1} of {file}')
                    reference_dict['unparsed documents'] = file
                    continue

            # Align the columns
            if value_to_ignore in df_list[page_number].iloc[0].values:
                value_to_ignore_column = df_list[page_number].iloc[0].values.tolist().index(value_to_ignore)
                print(f'\tRemoving {value_to_ignore} from column {value_to_ignore_column}')
                df_list[page_number].iloc[0, value_to_ignore_column:-1] = df_list[page_number].iloc[0, value_to_ignore_column+1:].values

            for column in blank_columns:
                try:
                    if df_list[page_number][column].isna().sum() != len(df_list[page_number]): # 2023-03-30 22:45:
                        blank_column_index = df_list[page_number].columns.tolist().index(column)
                        print(f'\tShifting data from column {blank_column_index}({column})')
                        df_list[page_number].iloc[:,blank_column_index+1:] = df_list[page_number].iloc[:,blank_column_index:-1].values 
                        df_list[page_number].iloc[:,blank_column_index] = None
                except:
                    pass
            column_after_blank_columns_index = blank_column_index + 1
            column_after_blank_columns = df_list[page_number].columns[column_after_blank_columns_index]
            try:
                while df_list[page_number][column_after_blank_columns].isna().sum() == len(df_list[page_number]):
                    print(f'\tShifting data to the right until `{column_after_blank_columns}` column')
                    df_list[page_number].iloc[:,column_after_blank_columns_index:-1] = df_list[
                        page_number].iloc[:,column_after_blank_columns_index+1:].values 
                    df_list[page_number].drop(columns=df_list[page_number].columns[-1], inplace=True)
            except:
                pass

            file_dfs.append(df_list[page_number]) # 2023-03-30 23:04 Decrease indent
        dfs_list.append(file_dfs) # SH 2023-04-01 20:58
        

    return dfs_list, reference_dict




def parse_pdf(pdf_filenames,pages='all', last_id_column='HA', value_to_ignore='s.22(1)', 
    value_to_ignore_start_index=1, value_to_ignore_end_index=None,
    blank_columns=['workplace_name', 'workplace_address'],
    filename=None,filepath=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\BC-Covid-workplace-closures\data\raw', 
    path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\BC-Covid-workplace-closures\data\interim'):
    """
    Create dataframe from a PDF file using 
    """
    print(f'Extracting the following files:')
    if type(pdf_filenames) != list:
        pdf_filenames = [pdf_filenames]
    dfs_list = []
    reference_dict = dict()
    corrected_pages = dict()
    for index, file in enumerate(pdf_filenames):
        print(file)
        filepath = f'{filepath}/'.replace('\\','/') if filepath else ''
        path_and_file = filepath+file
        try:
            df_list = read_pdf(path_and_file, pages=pages)
        except:
            print(f'\nError parsing {file}')
        file_dfs = []
        page1_columns = df_list[0].columns.str.replace('\r', '').str.replace(r'(workplace)([a-zA-Z]+)', r'\1_\2', regex=True).to_list()
        print('number of columns:', len(page1_columns))
        if last_id_column in page1_columns:
            page1_columns_sorted = sorted(page1_columns)
        else:
            previous_columns = dfs_list[index-1].columns
            print(f'\t\tNo headers found; using headers from previous document:', previous_columns)
            if len(df_list[page_number].columns) > len(previous_columns):
                n_cols_to_pad = len(df_list[page_number].columns) - len(previous_columns)
                new_columns_padding = [f'blank{index}' for index, col in enumerate(range(n_cols_to_pad))]
                page1_columns_sorted = sorted(page1_columns) + new_columns_padding
                page1_columns = page1_columns + new_columns_padding

            elif len(df_list[page_number].columns) < len(previous_columns):
                page1_columns = (previous_columns[:len(df_list[page_number].columns)].to_list())
                page1_columns_sorted = sorted(page1_columns)
            else:
                page1_columns_sorted = (dfs_list[index-1].columns.to_list())
            page1_columns = ['Unnamed: 0' if column=='index' else column for column in page1_columns]
            print(f'\t\t**Current df columns:',page1_columns)
        
        for page_number in range(len(df_list)):
            print(f'\tpage {page_number+1}...')
            df_list[page_number].columns = df_list[page_number].columns.str.replace('\r', '').str.replace(r'(workplace)([a-zA-Z]+)', r'\1_\2', regex=True)
            if len(df_list[page_number].columns.to_list()) == len(page1_columns_sorted):
                if sorted(df_list[page_number].columns.to_list()) == page1_columns_sorted:
                    pass
                else:
                    df_list[page_number].columns = page1_columns 
                    print('\t\tKeeping page 1 columns')
            else:
                if len(df_list[page_number].columns) > len(page1_columns_sorted):
                    n_cols_to_pad = len(df_list[page_number].columns) - len(page1_columns_sorted)
                    new_columns_padding = [f'blank{index}' for index, col in enumerate(range(n_cols_to_pad))]
                    df_list[page_number].columns = page1_columns + new_columns_padding
                    print('\t\tPadding columns')
                elif len(df_list[page_number].columns) < len(page1_columns_sorted):
                    df_list[page_number].columns = page1_columns[:len(df_list[page_number].columns)]
                    print('\t\tDropping columns')
                else:
                    df_list[page_number].columns = page1_columns
                corrected_pages[file] = page_number+1
                print(f'\n\t**Fixing column parsing for page {page_number+1} of {file}**')
                print(f'\t\t{df_list[page_number].columns}')
            # Consolidate rows that were incorrectly split
            if (df_list[page_number]['Unnamed: 0'].isna().sum() > 0):
                # print('\tAttempting to consolidate rows that were incorrectly split: ', df_list[page_number].columns)
                print('\tAttempting to consolidate rows that were incorrectly split: ')
                try:
                    df_list[page_number] = merge_rows(df_list[page_number], file)
                except:
                    print(f'\t\tUnable to consolidate rows on page {page_number+1} of {file}')
                    reference_dict['unparsed documents'] = file
                    continue

            # Align the columns
            value_to_ignore_regex = rf'.*{re.escape(value_to_ignore[value_to_ignore_start_index:value_to_ignore_end_index])}.*'
            if df_list[page_number].iloc[0].str.contains(value_to_ignore_regex).any():
                row_values = df_list[page_number].fillna(0).iloc[0].values.tolist()
                for index, item in enumerate(row_values):
                    if re.match(value_to_ignore_regex, str(item)):
                        # print('found regex')
                        value_to_ignore_column = index
                        break

                print(f'\tRemoving {value_to_ignore} from column {value_to_ignore_column}')
                df_list[page_number].iloc[0, value_to_ignore_column:-1] = df_list[page_number].iloc[0, value_to_ignore_column+1:].values

            for column in blank_columns:
                try:
                    if df_list[page_number][column].isna().sum() != len(df_list[page_number]): # 2023-03-30 22:45:
                        blank_column_index = df_list[page_number].columns.tolist().index(column)
                        print(f'\tShifting data from column {blank_column_index}({column})')
                        df_list[page_number].iloc[:,blank_column_index+1:] = df_list[page_number].iloc[:,blank_column_index:-1].values 
                        df_list[page_number].iloc[:,blank_column_index] = None
                except:
                    pass
            column_after_blank_columns_index = blank_column_index + 1
            column_after_blank_columns = df_list[page_number].columns[column_after_blank_columns_index]
            try:
                while df_list[page_number][column_after_blank_columns].isna().sum() == len(df_list[page_number]):
                    print(f'\tShifting data to the right until `{column_after_blank_columns}` column')
                    df_list[page_number].iloc[:,column_after_blank_columns_index:-1] = df_list[
                        page_number].iloc[:,column_after_blank_columns_index+1:].values 
                    df_list[page_number].drop(columns=df_list[page_number].columns[-1], inplace=True)
            except:
                pass
            df_list[page_number]['doc_page_number'] = page_number + 1 # 2023-03-30 23:15
            file_dfs.append(df_list[page_number])  # 2023-03-30 23:04 Decrease indent
        
        try:
            final_file_df = pd.concat(file_dfs)
            final_file_df['file'] = file
            final_file_df.columns = final_file_df.columns.str.replace('cosed', 'closed')
            final_file_df.reset_index(inplace=True, drop=True)
            final_file_df.rename(columns={'Unnamed: 0': 'index'}, inplace=True)
            dfs_list.append(final_file_df)

            print('\n\tDone')
        except:
            print(f'Error concatenating dataframes from {file}')
            return file_dfs, {file: 'DataFrame concatenation error'}
    unique_cols_dict = {}
    for i, df in enumerate(dfs_list):
        other_dfs = dfs_list[:i] + dfs_list[i+1:]
        other_cols = set().union(*[set(other_df.columns) for other_df in other_dfs])
        unique_columns = set(df.columns) - other_cols
        unique_cols_dict[file] = unique_columns if len(unique_columns)>0 else "none"
        print(f'Columns unique to part{pdf_filenames[i]}: {unique_cols_dict[file]}')
    reference_dict['corrected pages'] = corrected_pages
    reference_dict['unique columns'] = unique_cols_dict
    if filename:
        try:
            savepickle(dfs_list, filename=filename, path=path)
            with open(f'{filepath}{filename}_reference.json','w') as json_file:
                json.dump(reference_dict, json_file)
            save_csv(pd.concat(dfs_list), filename=filename, path=path)
        except:
            print('Unable to save outputs')
    try:
        return pd.concat(dfs_list), reference_dict
    except:
        print('Unable to concatenate dataframes from different documents')
        return dfs_list, reference_dict

n_docs = 7
start = 1
filenames = [f'F22-1445 Records_Part{i}.pdf' for i in range(start,start+n_docs)]
part = 1
filename = f'F22-1445 Records_Part{part}.pdf'

dfs_list1_i6, reference1_i6 = parse_pdf_tolist(filename, pages='1,118', filename=None)
# dfs, reference = parse_pdf(filenames, pages='all', filename='pdfs_parsed_2023-03-30')


Extracting the following files:
F22-1445 Records_Part1.pdf
number of columns: 22
	page 1...
		Current number of columns: 22 22
		22, 22
	Shifting data from column 2(workplace_name)
	Shifting data from column 3(workplace_address)
	page 2...
		Current number of columns: 25 22
		22, 22
		Found new columns: {'Unnamed: 6', 'Unnamed: 20', 'Unnamed: 13', 'Unnamed: 1', 'cases_', 'Unnamed: 5', 'Unnamed: 10', 'Unnamed: 14', 'Unnamed: 22', 'Unnamed: 2', 'Unnamed: 7', 'Unnamed: 19', 'Unnamed: 8', 'Unnamed: 4', 'Unnamed: 23', 'Unnamed: 12', 'Unnamed: 3', 'Unnamed: 18', 'Unnamed: 15', 'Unnamed: 9', 'Unnamed: 21', 'Unnamed: 11', 'Unnamed: 17', 'Unnamed: 16'}
		Padding 3 columns

	**Fixing column parsing for page 2 of F22-1445 Records_Part1.pdf**
		Index(['Unnamed: 0', 'HA', 'workplace_name', 'workplace_address', 'city',
       'date_cluster_confirmed_public_health', 'date_cluster_no_longer_active',
       'earliest_reported_cases', 'latest_reported_cases',
       'workplace_classification', 'cases_wh

In [67]:
dfs_list1_i6[0][1]

,Unnamed: 0,HA,workplace_name,workplace_address,city,date_cluster_confirmed_public_health,date_cluster_no_longer_active,earliest_reported_cases,latest_reported_cases,workplace_classification,...,closure_y_n,closed_by_order,voluntary_closure,closure_date,partially_cosed,open,days_since_last_case,cluster_confirmed_epidate,cluster_closed_epidate,blank0
0,455.0,FHA,None,None,SURREY,5/17/2021,5/28/2021,None,None,n,...,No,No,Unknown,None,No,Yes,None,No,20-2021,None
2,456.0,FHA,None,None,ABBOTSFORD,5/17/2021,5/28/2021,None,None,uction,...,No,No,Unknown,None,No,Yes,None,No,20-2021,None
3,457.0,FHA,None,None,SURREY,5/17/2021,6/3/2021,None,None,Office,...,No,No,Unknown,None,No,Yes,None,No,20-2021,None
4,458.0,FHA,None,None,MAPLE RIDGE,5/17/2021,6/2/2021,None,None,g/Industry,...,Yes,Yes,Unknown,5/18/2021,No,No,None,No,20-2021,20-2021
5,459.0,FHA,None,None,SURREY,5/17/2021,5/31/2021,None,None,n,...,No,No,Unknown,None,No,Yes,None,No,20-2021,None
6,460.0,FHA,None,None,DELTA,5/17/2021,5/27/2021,None,None,g/Industry,...,No,No,Unknown,None,No,Yes,None,No,20-2021,None
7,461.0,FHA,None,None,WESTMINSTER,5/17/2021,5/25/2021,None,None,Retail,...,No,No,Unknown,None,No,Yes,None,No,20-2021,None
8,462.0,NHA,None,None,Unknown,5/17/2021,None,None,None,Housing,...,No,No,No,None,n,No,None,Yes,20-2021,None
9,463.0,FHA,None,None,BURNABY,5/18/2021,6/7/2021,None,None,ar/Lounge,...,Yes,Yes,Unknown,5/19/2021,No,No,None,No,20-2021,20-2021
10,464.0,VIHA,None,None,Nanaimo,5/18/2021,None,5/16/2021,5/18/2021,uction,...,No,No,No,None,n,Yes,35,No,20-2021,None


## Iteration 7
part1 columns change starting on page 34; best to parse it in 2 batches to avoid column misalignment

In [84]:
from tabula import read_pdf
import json
import re
import warnings
warnings.simplefilter("ignore", category=FutureWarning)

def merge_rows(df, file, id_column=['Unnamed: 0'], fill_method='bfill', groupby_column=['HA']):
    """
    Merges rows in a pandas dataframe that were parsed incorrectly into multiple rows.
    Assumes that the rows that need to be merged have the same value in a unique ID column.
    """
    print(f'\tRemoving duplicate rows from {file}...')
    print('\t\tOriginal df shape: ', df.shape)
    print(f'\t\tOriginal columns:', df.columns)
    df[id_column] = df[id_column].fillna(method=fill_method)
    
    # Group the rows by the unique ID column and aggregate the values in each group

    columns_to_fill = df.drop(columns=groupby_column).columns
    grouped_df = df.groupby(id_column+groupby_column, as_index=False).agg(
        lambda x: ''.join(str(v) for v in x if (type(v)==str) & (x.name in columns_to_fill)))
    grouped_df = grouped_df[grouped_df[groupby_column[0]] != groupby_column[0]]
    # Return the merged dataframe
    print('\t\tTransformed df shape:', grouped_df.shape)
    return grouped_df.replace('', None)



def parse_pdf_tolist(pdf_filenames, filepath=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\BC-Covid-workplace-closures\data\raw', 
    pages='all', last_id_column='HA', value_to_ignore='s.22(1)',
    blank_columns=['workplace_name', 'workplace_address'],
    filename=None,
    path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\BC-Covid-workplace-closures\data\interim'):
    """
    Create dataframe from a PDF file using 
    """
    print(f'Extracting the following files:')
    if type(pdf_filenames) != list:
        pdf_filenames = [pdf_filenames]
    dfs_list = []
    reference_dict = dict()
    corrected_pages = dict()
    for index, file in enumerate(pdf_filenames):
        print(file)
        filepath = f'{filepath}/'.replace('\\','/') if filepath else ''
        path_and_file = filepath+file
        try:
            df_list = read_pdf(path_and_file, pages=pages)
        except:
            print(f'\nError parsing {file}')
        file_dfs = []
        page1_columns = df_list[0].columns.str.replace('\r', '').str.replace(r'(workplace)([a-zA-Z]+)', r'\1_\2', regex=True).to_list()
        print('number of columns:', len(page1_columns))
        if last_id_column in page1_columns:
            page1_columns_sorted = sorted(page1_columns)
        else:
            previous_columns = dfs_list[index-1].columns
            print(f'\t\tNo headers found; using headers from previous document:', previous_columns)
            if len(df_list[page_number].columns) > len(previous_columns):
                n_cols_to_pad = len(df_list[page_number].columns) - len(previous_columns)
                new_columns_padding = [f'blank{index}' for index, col in enumerate(range(n_cols_to_pad))]
                page1_columns_sorted = sorted(page1_columns) + new_columns_padding
                page1_columns = page1_columns + new_columns_padding

            elif len(df_list[page_number].columns) < len(previous_columns):
                page1_columns = (previous_columns[:len(df_list[page_number].columns)].to_list())
                page1_columns_sorted = sorted(page1_columns)
            else:
                page1_columns_sorted = (dfs_list[index-1].columns.to_list())
            page1_columns = ['Unnamed: 0' if column=='index' else column for column in page1_columns]
            print(f'\t\t**Current df columns:',page1_columns)
        
        for page_number in range(len(df_list)):
            print(f'\tpage {page_number+1}...')
            df_list[page_number].columns = df_list[page_number].columns.str.replace('\r', '').str.replace(r'(workplace)([a-zA-Z]+)', r'\1_\2', regex=True)
            print('\t\tCurrent number of columns:', len(df_list[page_number].columns), len(page1_columns_sorted))
            print(f'\t\t{len(page1_columns_sorted)}, {len(page1_columns)}')
            if len(df_list[page_number].columns.to_list()) == len(page1_columns_sorted):
                if sorted(df_list[page_number].columns.to_list()) == page1_columns_sorted:
                    pass
                else:
                    df_list[page_number].columns = page1_columns 
                    print('\t\tKeeping page 1 columns')
            else:
                if len(df_list[page_number].columns) > len(page1_columns_sorted):
                    n_cols_to_pad = len(df_list[page_number].columns) - len(page1_columns_sorted)
                    new_columns_padding = [f'blank{index}' for index, col in enumerate(range(n_cols_to_pad))]
                    new_columns = set(df_list[page_number].columns) - set(page1_columns_sorted)
                    print(f'\t\tFound new columns: {new_columns}')
                    if len(new_columns) < len(page1_columns_sorted):
                        df_list[page_number].columns = df_list[page_number].columns.tolist() + new_columns_padding
                    else:
                        df_list[page_number].columns = page1_columns + new_columns_padding
                    print(f'\t\tPadding {n_cols_to_pad} columns')
                elif len(df_list[page_number].columns) < len(page1_columns_sorted):
                    df_list[page_number].columns = page1_columns[:len(df_list[page_number].columns)]
                    print('\t\tDropping columns')
                else:
                    df_list[page_number].columns = page1_columns
                corrected_pages[file] = page_number+1
                print(f'\n\t**Fixing column parsing for page {page_number+1} of {file}**')
                print(f'\t\t{df_list[page_number].columns}')
            # Consolidate rows that were incorrectly split
            if (df_list[page_number]['Unnamed: 0'].isna().sum() > 0):
                # print('\tAttempting to consolidate rows that were incorrectly split: ', df_list[page_number].columns)
                print('\tAttempting to consolidate rows that were incorrectly split: ')
                try:
                    df_list[page_number] = merge_rows(df_list[page_number], file)
                except:
                    print(f'\t\tUnable to consolidate rows on page {page_number+1} of {file}')
                    reference_dict['unparsed documents'] = file
                    continue

            # Align the columns
            if value_to_ignore in df_list[page_number].iloc[0].values:
                value_to_ignore_column = df_list[page_number].iloc[0].values.tolist().index(value_to_ignore)
                print(f'\tRemoving {value_to_ignore} from column {value_to_ignore_column}')
                df_list[page_number].iloc[0, value_to_ignore_column:-1] = df_list[page_number].iloc[0, value_to_ignore_column+1:].values

            for column in blank_columns:
                try:
                    if df_list[page_number][column].isna().sum() != len(df_list[page_number]): # 2023-03-30 22:45:
                        blank_column_index = df_list[page_number].columns.tolist().index(column)
                        print(f'\tShifting data from column {blank_column_index}({column})')
                        df_list[page_number].iloc[:,blank_column_index+1:] = df_list[page_number].iloc[:,blank_column_index:-1].values 
                        df_list[page_number].iloc[:,blank_column_index] = None
                except:
                    pass
            column_after_blank_columns_index = blank_column_index + 1
            column_after_blank_columns = df_list[page_number].columns[column_after_blank_columns_index]
            try:
                while df_list[page_number][column_after_blank_columns].isna().sum() == len(df_list[page_number]):
                    print(f'\tShifting data to the right until `{column_after_blank_columns}` column')
                    df_list[page_number].iloc[:,column_after_blank_columns_index:-1] = df_list[
                        page_number].iloc[:,column_after_blank_columns_index+1:].values 
                    df_list[page_number].drop(columns=df_list[page_number].columns[-1], inplace=True)
            except:
                pass

            file_dfs.append(df_list[page_number]) # 2023-03-30 23:04 Decrease indent
        dfs_list.append(file_dfs) # SH 2023-04-01 20:58
        

    return dfs_list, reference_dict




def parse_pdf(pdf_filenames,pages='all', last_id_column='HA', value_to_ignore='s.22(1)', 
    value_to_ignore_start_index=1, value_to_ignore_end_index=None,
    blank_columns=['workplace_name', 'workplace_address'],
    filename=None,filepath=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\BC-Covid-workplace-closures\data\raw', 
    path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\BC-Covid-workplace-closures\data\interim'):
    """
    Create dataframe from a PDF file using 
    """
    print(f'Extracting the following files:')
    if type(pdf_filenames) != list:
        pdf_filenames = [pdf_filenames]
    dfs_list = []
    reference_dict = dict()
    corrected_pages = dict()
    for index, file in enumerate(pdf_filenames):
        print(file)
        filepath = f'{filepath}/'.replace('\\','/') if filepath else ''
        path_and_file = filepath+file
        try:
            df_list = read_pdf(path_and_file, pages=pages)
        except:
            print(f'\nError parsing {file}')
        file_dfs = []
        page1_columns = df_list[0].columns.str.replace('\r', '').str.replace(r'(workplace)([a-zA-Z]+)', r'\1_\2', regex=True).to_list()
        print('number of columns:', len(page1_columns))
        if last_id_column in page1_columns:
            page1_columns_sorted = sorted(page1_columns)
        else:
            previous_columns = dfs_list[index-1].columns
            print(f'\t\tNo headers found; using headers from previous document:', previous_columns)
            if len(df_list[page_number].columns) > len(previous_columns):
                n_cols_to_pad = len(df_list[page_number].columns) - len(previous_columns)
                new_columns_padding = [f'blank{index}' for index, col in enumerate(range(n_cols_to_pad))]
                page1_columns_sorted = sorted(page1_columns) + new_columns_padding
                page1_columns = page1_columns + new_columns_padding

            elif len(df_list[page_number].columns) < len(previous_columns):
                page1_columns = (previous_columns[:len(df_list[page_number].columns)].to_list())
                page1_columns_sorted = sorted(page1_columns)
            else:
                page1_columns_sorted = (dfs_list[index-1].columns.to_list())
            page1_columns = ['Unnamed: 0' if column=='index' else column for column in page1_columns]
            print(f'\t\t**Current df columns:',page1_columns)
        
        for page_number in range(len(df_list)):
            print(f'\tpage {page_number+1}...')
            df_list[page_number].columns = df_list[page_number].columns.str.replace('\r', '').str.replace(r'(workplace)([a-zA-Z]+)', r'\1_\2', regex=True)
            if len(df_list[page_number].columns.to_list()) == len(page1_columns_sorted):
                if sorted(df_list[page_number].columns.to_list()) == page1_columns_sorted:
                    pass
                else:
                    df_list[page_number].columns = page1_columns 
                    print('\t\tKeeping page 1 columns')
            else:
                if len(df_list[page_number].columns) > len(page1_columns_sorted):
                    new_columns = set(df_list[page_number].columns) - set(page1_columns_sorted)
                    print(f'\t\tFound new columns: {new_columns}')
                    if len(new_columns) < len(page1_columns_sorted):
                        df_list[page_number].columns = df_list[page_number].columns.tolist() 
                    else:
                        n_cols_to_pad = len(df_list[page_number].columns) - len(page1_columns_sorted)
                        new_columns_padding = [f'blank{index}' for index, col in enumerate(range(n_cols_to_pad))]
                        df_list[page_number].columns = page1_columns + new_columns_padding
                        print(f'\t\tPadding {n_cols_to_pad} columns')
                elif len(df_list[page_number].columns) < len(page1_columns_sorted):
                    df_list[page_number].columns = page1_columns[:len(df_list[page_number].columns)]
                    print('\t\tDropping columns')
                else:
                    df_list[page_number].columns = page1_columns
                corrected_pages[file] = page_number+1
                print(f'\n\t**Fixing column parsing for page {page_number+1} of {file}**')
                print(f'\t\t{df_list[page_number].columns}')
            # Consolidate rows that were incorrectly split
            if (df_list[page_number]['Unnamed: 0'].isna().sum() > 0):
                # print('\tAttempting to consolidate rows that were incorrectly split: ', df_list[page_number].columns)
                print('\tAttempting to consolidate rows that were incorrectly split: ')
                try:
                    df_list[page_number] = merge_rows(df_list[page_number], file)
                except:
                    print(f'\t\tUnable to consolidate rows on page {page_number+1} of {file}')
                    reference_dict['unparsed documents'] = file
                    continue

            # Align the columns
            value_to_ignore_regex = rf'.*{re.escape(value_to_ignore[value_to_ignore_start_index:value_to_ignore_end_index])}.*'
            if df_list[page_number].iloc[0].str.contains(value_to_ignore_regex).any():
                row_values = df_list[page_number].fillna(0).iloc[0].values.tolist()
                for index, item in enumerate(row_values):
                    if re.match(value_to_ignore_regex, str(item)):
                        # print('found regex')
                        value_to_ignore_column = index
                        break

                print(f'\tRemoving {value_to_ignore} from column {value_to_ignore_column}')
                df_list[page_number].iloc[0, value_to_ignore_column:-1] = df_list[page_number].iloc[0, value_to_ignore_column+1:].values

            for column in blank_columns:
                try:
                    if df_list[page_number][column].isna().sum() != len(df_list[page_number]): # 2023-03-30 22:45:
                        blank_column_index = df_list[page_number].columns.tolist().index(column)
                        print(f'\tShifting data from column {blank_column_index}({column})')
                        df_list[page_number].iloc[:,blank_column_index+1:] = df_list[page_number].iloc[:,blank_column_index:-1].values 
                        df_list[page_number].iloc[:,blank_column_index] = None
                except:
                    pass
            column_after_blank_columns_index = blank_column_index + 1
            column_after_blank_columns = df_list[page_number].columns[column_after_blank_columns_index]
            try:
                while df_list[page_number][column_after_blank_columns].isna().sum() == len(df_list[page_number]):
                    print(f'\tShifting data to the right until `{column_after_blank_columns}` column')
                    df_list[page_number].iloc[:,column_after_blank_columns_index:-1] = df_list[
                        page_number].iloc[:,column_after_blank_columns_index+1:].values 
                    df_list[page_number].drop(columns=df_list[page_number].columns[-1], inplace=True)
            except:
                pass
            df_list[page_number]['doc_page_number'] = page_number + 1 # 2023-03-30 23:15
            file_dfs.append(df_list[page_number])  # 2023-03-30 23:04 Decrease indent
        
        try:
            final_file_df = pd.concat(file_dfs)
            final_file_df['file'] = file
            final_file_df.columns = final_file_df.columns.str.replace('cosed', 'closed')
            final_file_df.reset_index(inplace=True, drop=True)
            final_file_df.rename(columns={'Unnamed: 0': 'index'}, inplace=True)
            dfs_list.append(final_file_df)

            print('\n\tDone')
        except:
            print(f'Error concatenating dataframes from {file}')
            return file_dfs, {file: 'DataFrame concatenation error'}
    unique_cols_dict = {}
    for i, df in enumerate(dfs_list):
        other_dfs = dfs_list[:i] + dfs_list[i+1:]
        other_cols = set().union(*[set(other_df.columns) for other_df in other_dfs])
        unique_columns = set(df.columns) - other_cols
        unique_cols_dict[file] = unique_columns if len(unique_columns)>0 else "none"
        print(f'Columns unique to part{pdf_filenames[i]}: {unique_cols_dict[file]}')
    reference_dict['corrected pages'] = corrected_pages
    reference_dict['unique columns'] = unique_cols_dict
    if filename:
        try:
            savepickle(dfs_list, filename=filename, path=path)
            with open(f'{filepath}{filename}_reference.json','w') as json_file:
                json.dump(reference_dict, json_file)
            save_csv(pd.concat(dfs_list), filename=filename, path=path)
        except:
            print('Unable to save outputs')
    try:
        return pd.concat(dfs_list), reference_dict
    except:
        print('Unable to concatenate dataframes from different documents')
        return dfs_list, reference_dict

n_docs = 7
start = 1
filenames = [f'F22-1445 Records_Part{i}.pdf' for i in range(start,start+n_docs)]
part = 1
filename = f'F22-1445 Records_Part{part}.pdf'

dfs1_i7, reference1_i7 = parse_pdf(filename, pages='34,118', filename=None)
# dfs, reference = parse_pdf(filenames, pages='all', filename='pdfs_parsed_2023-03-30')


Extracting the following files:
F22-1445 Records_Part1.pdf
number of columns: 23
	page 1...
	Removing s.22(1) from column 2
	Shifting data from column 2(workplace_name)
	Shifting data from column 3(workplace_address)
	page 2...
		Found new columns: {'Unnamed: 6', 'Unnamed: 20', 'Unnamed: 13', 'Unnamed: 1', 'cases_', 'Unnamed: 5', 'Unnamed: 10', 'Unnamed: 14', 'Unnamed: 22', 'Unnamed: 2', 'Unnamed: 7', 'Unnamed: 19', 'Unnamed: 8', 'Unnamed: 4', 'Unnamed: 23', 'Unnamed: 12', 'Unnamed: 3', 'Unnamed: 18', 'Unnamed: 15', 'Unnamed: 9', 'Unnamed: 21', 'Unnamed: 11', 'Unnamed: 17', 'Unnamed: 16'}
		Padding 2 columns

	**Fixing column parsing for page 2 of F22-1445 Records_Part1.pdf**
		Index(['Unnamed: 0', 'HA', 'workplace_name', 'workplace_address', 'city',
       'date_cluster_confirmed_public_health', 'date_cluster_no_longer_active',
       'earliest_reported_cases', 'latest_reported_cases',
       'workplace_classification', 'cases_when_cluster_confirmed',
       'total_cases', 'workplace_

In [82]:
dfs1_i7

,index,HA,workplace_name,workplace_address,city,date_cluster_confirmed_public_health,date_cluster_no_longer_active,earliest_reported_cases,latest_reported_cases,workplace_classification,...,voluntary_closure,closure_date,partially_closed,open,days_since_last_case,active_clusters,cluster_confirmed_epidate,cluster_closed_epidate,doc_page_number,file
0,1.0,FHA,None,None,DELTA,4/1/2021,4/6/2021,NaN,NaN,Transportatio\rn,...,Unknown,NaN,No,No,NaN,No,13-2021,NaN,1,F22-1445 Records_Part1.pdf
1,2.0,FHA,None,None,MAPLE RIDGE,4/1/2021,4/18/2021,NaN,NaN,Restaurant/B\rar/Lounge,...,Unknown,4/6/2021,No,No,NaN,No,13-2021,14-2021,1,F22-1445 Records_Part1.pdf
2,3.0,FHA,None,None,SURREY,4/1/2021,4/22/2021,NaN,NaN,Restaurant/B\rar/Lounge,...,Unknown,4/12/2021,No,No,NaN,No,13-2021,15-2021,1,F22-1445 Records_Part1.pdf
3,4.0,FHA,None,None,SURREY,4/1/2021,4/14/2021,NaN,NaN,Office,...,Unknown,4/3/2021,No,No,NaN,No,13-2021,13-2021,1,F22-1445 Records_Part1.pdf
4,5.0,FHA,None,None,LANGLEY,4/1/2021,NaN,NaN,NaN,Warehouse,...,Unknown,NaN,No,No,NaN,Yes,13-2021,NaN,1,F22-1445 Records_Part1.pdf
5,6.0,FHA,None,None,BURNABY,4/1/2021,4/21/2021,NaN,NaN,Gym/Fitness,...,Unknown,4/7/2021,No,No,NaN,No,13-2021,14-2021,1,F22-1445 Records_Part1.pdf
6,7.0,FHA,None,None,SURREY,4/1/2021,4/23/2021,NaN,NaN,Gym/Fitness,...,Unknown,NaN,No,No,NaN,No,13-2021,NaN,1,F22-1445 Records_Part1.pdf
7,8.0,FHA,None,None,ABBOTSFORD,4/1/2021,4/19/2021,NaN,NaN,Gym/Fitness,...,Unknown,4/3/2021,No,No,NaN,No,13-2021,13-2021,1,F22-1445 Records_Part1.pdf
8,9.0,FHA,None,None,CHILLIWACK,4/1/2021,4/17/2021,NaN,NaN,Maintenance/\rTrades/Constr\ruction,...,Unknown,NaN,No,No,NaN,No,13-2021,NaN,1,F22-1445 Records_Part1.pdf
9,10.0,FHA,None,None,LANGLEY,4/1/2021,4/30/2021,NaN,NaN,Farm/Agricult\rure,...,Unknown,NaN,No,No,NaN,No,13-2021,NaN,1,F22-1445 Records_Part1.pdf


### Parse documents in batches

In [85]:
part = 1
filename = f'F22-1445 Records_Part{part}.pdf'
pages = '1-30'
dfs1_part1, reference1_part1 = parse_pdf(filename, pages=pages, filename=f'pdfs_parsed_04-01_part1_p{pages}')

Extracting the following files:
F22-1445 Records_Part1.pdf
number of columns: 22
	page 1...
	Shifting data from column 2(workplace_name)
	Shifting data from column 3(workplace_address)
	page 2...
	Removing s.22(1) from column 2
	Shifting data from column 2(workplace_name)
	Shifting data from column 3(workplace_address)
	page 3...
	Shifting data from column 2(workplace_name)
	Shifting data from column 3(workplace_address)
	page 4...
	Removing s.22(1) from column 2
	Shifting data from column 2(workplace_name)
	Shifting data from column 3(workplace_address)
	page 5...
	Shifting data from column 2(workplace_name)
	Shifting data from column 3(workplace_address)
	page 6...
	Shifting data from column 2(workplace_name)
	Shifting data from column 3(workplace_address)
	page 7...
	Removing s.22(1) from column 2
	Shifting data from column 2(workplace_name)
	Shifting data from column 3(workplace_address)
	page 8...
	Removing s.22(1) from column 2
	Shifting data from column 2(workplace_name)
	Shifti

In [86]:
dfs1_part1

,index,HA,workplace_name,workplace_address,city,date_cluster_confirmed_public_health,date_cluster_no_longer_active,earliest_reported_cases,latest_reported_cases,workplace_classification,...,voluntary_closure,closure_date,partially_closed,open,days_since_last_case,cluster_confirmed_epidate,cluster_closed_epidate,doc_page_number,Unnamed: 1,file
0,1.0,FHA,None,None,SURREY,4/1/2021,4/17/2021,NaN,NaN,Restaurant/Bar/L\rounge,...,Unknown,NaN,No,No,NaN,13-2021,NaN,1,NaN,F22-1445 Records_Part1.pdf
1,2.0,FHA,None,None,LANGLEY,4/1/2021,4/13/2021,NaN,NaN,Other,...,Unknown,NaN,No,No,NaN,13-2021,NaN,1,NaN,F22-1445 Records_Part1.pdf
2,3.0,FHA,None,None,SURREY,4/1/2021,4/13/2021,NaN,NaN,Restaurant/Bar/L\rounge,...,Unknown,4/4/2021,No,No,NaN,13-2021,14-2021,1,NaN,F22-1445 Records_Part1.pdf
3,4.0,FHA,None,None,BURNABY,4/1/2021,4/21/2021,NaN,NaN,Gym/Fitness,...,Unknown,4/7/2021,No,No,NaN,13-2021,14-2021,1,NaN,F22-1445 Records_Part1.pdf
4,5.0,FHA,None,None,BURNABY,4/1/2021,4/14/2021,NaN,NaN,Restaurant/Bar/L\rounge,...,Unknown,4/5/2021,No,No,NaN,13-2021,14-2021,1,NaN,F22-1445 Records_Part1.pdf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
459,460.0,IHA,None,None,5/27/2021,NaN,5/27/2021,6/5/2021,Restaurant/Bar/L\rounge,4,...,5/26/2021,Unknown,No,NaN,21-2021,21-2021,NaN,30,Kelowna,F22-1445 Records_Part1.pdf
460,461.0,IHA,None,None,5/28/2021,NaN,5/28/2021,5/3/2021,Personal Care,3,...,NaN,Unknown,No,NaN,21-2021,NaN,NaN,30,Kelowna,F22-1445 Records_Part1.pdf
461,462.0,IHA,None,None,5/31/2021,NaN,5/31/2021,5/31/2021,Maintenance/Tr\rades/Constructio\rn,4,...,NaN,Unknown,No,NaN,22-2021,NaN,NaN,30,Kelowna,F22-1445 Records_Part1.pdf
462,463.0,IHA,None,None,5/31/2021,NaN,5/31/2021,6/3/2021,Retail,3,...,NaN,Unknown,No,NaN,22-2021,NaN,NaN,30,Vernon,F22-1445 Records_Part1.pdf


In [ ]:
n_docs = 6
start = 1
filenames = [f'F22-1445 Records_Part{i}.pdf' for i in range(start,start+n_docs)]
part = 1
filename = f'F22-1445 Records_Part{part}.pdf'

## Iteration 8

In [87]:
from tabula import read_pdf
import json
import re
import warnings
warnings.simplefilter("ignore", category=FutureWarning)

def merge_rows(df, file, id_column=['Unnamed: 0'], fill_method='bfill', groupby_column=['HA']):
    """
    Merges rows in a pandas dataframe that were parsed incorrectly into multiple rows.
    Assumes that the rows that need to be merged have the same value in a unique ID column.
    """
    print(f'\tRemoving duplicate rows from {file}...')
    print('\t\tOriginal df shape: ', df.shape)
    print(f'\t\tOriginal columns:', df.columns)
    df[id_column] = df[id_column].fillna(method=fill_method)
    
    # Group the rows by the unique ID column and aggregate the values in each group

    columns_to_fill = df.drop(columns=groupby_column).columns
    grouped_df = df.groupby(id_column+groupby_column, as_index=False).agg(
        lambda x: ''.join(str(v) for v in x if (type(v)==str) & (x.name in columns_to_fill)))
    grouped_df = grouped_df[grouped_df[groupby_column[0]] != groupby_column[0]]
    # Return the merged dataframe
    print('\t\tTransformed df shape:', grouped_df.shape)
    return grouped_df.replace('', None)



def parse_pdf_tolist(pdf_filenames, filepath=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\BC-Covid-workplace-closures\data\raw', 
    pages='all', last_id_column='HA', value_to_ignore='s.22(1)',
    blank_columns=['workplace_name', 'workplace_address'],
    filename=None,
    path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\BC-Covid-workplace-closures\data\interim'):
    """
    Create dataframe from a PDF file using 
    """
    print(f'Extracting the following files:')
    if type(pdf_filenames) != list:
        pdf_filenames = [pdf_filenames]
    dfs_list = []
    reference_dict = dict()
    corrected_pages = dict()
    for index, file in enumerate(pdf_filenames):
        print(file)
        filepath = f'{filepath}/'.replace('\\','/') if filepath else ''
        path_and_file = filepath+file
        try:
            df_list = read_pdf(path_and_file, pages=pages)
        except:
            print(f'\nError parsing {file}')
        file_dfs = []
        page1_columns = df_list[0].columns.str.replace('\r', '').str.replace(r'(workplace)([a-zA-Z]+)', r'\1_\2', regex=True).to_list()
        print('number of columns:', len(page1_columns))
        if last_id_column in page1_columns:
            page1_columns_sorted = sorted(page1_columns)
        else:
            previous_columns = dfs_list[index-1].columns
            print(f'\t\tNo headers found; using headers from previous document:', previous_columns)
            if len(df_list[page_number].columns) > len(previous_columns):
                n_cols_to_pad = len(df_list[page_number].columns) - len(previous_columns)
                new_columns_padding = [f'blank{index}' for index, col in enumerate(range(n_cols_to_pad))]
                page1_columns_sorted = sorted(page1_columns) + new_columns_padding
                page1_columns = page1_columns + new_columns_padding

            elif len(df_list[page_number].columns) < len(previous_columns):
                page1_columns = (previous_columns[:len(df_list[page_number].columns)].to_list())
                page1_columns_sorted = sorted(page1_columns)
            else:
                page1_columns_sorted = (dfs_list[index-1].columns.to_list())
            page1_columns = ['Unnamed: 0' if column=='index' else column for column in page1_columns]
            print(f'\t\t**Current df columns:',page1_columns)
        
        for page_number in range(len(df_list)):
            print(f'\tpage {page_number+1}...')
            df_list[page_number].columns = df_list[page_number].columns.str.replace('\r', '').str.replace(r'(workplace)([a-zA-Z]+)', r'\1_\2', regex=True)
            print('\t\tCurrent number of columns:', len(df_list[page_number].columns), len(page1_columns_sorted))
            print(f'\t\t{len(page1_columns_sorted)}, {len(page1_columns)}')
            if len(df_list[page_number].columns.to_list()) == len(page1_columns_sorted):
                if sorted(df_list[page_number].columns.to_list()) == page1_columns_sorted:
                    pass
                else:
                    df_list[page_number].columns = page1_columns 
                    print('\t\tKeeping page 1 columns')
            else:
                if len(df_list[page_number].columns) > len(page1_columns_sorted):
                    n_cols_to_pad = len(df_list[page_number].columns) - len(page1_columns_sorted)
                    new_columns_padding = [f'blank{index}' for index, col in enumerate(range(n_cols_to_pad))]
                    df_list[page_number].columns = page1_columns + new_columns_padding
                    print('\t\tPadding columns')
                elif len(df_list[page_number].columns) < len(page1_columns_sorted):
                    df_list[page_number].columns = page1_columns[:len(df_list[page_number].columns)]
                    print('\t\tDropping columns')
                else:
                    df_list[page_number].columns = page1_columns
                corrected_pages[file] = page_number+1
                print(f'\n\t**Fixing column parsing for page {page_number+1} of {file}**')
                print(f'\t\t{df_list[page_number].columns}')
            # Consolidate rows that were incorrectly split
            if (df_list[page_number]['Unnamed: 0'].isna().sum() > 0):
                # print('\tAttempting to consolidate rows that were incorrectly split: ', df_list[page_number].columns)
                print('\tAttempting to consolidate rows that were incorrectly split: ')
                try:
                    df_list[page_number] = merge_rows(df_list[page_number], file)
                except:
                    print(f'\t\tUnable to consolidate rows on page {page_number+1} of {file}')
                    reference_dict['unparsed documents'] = file
                    continue

            # Align the columns
            if value_to_ignore in df_list[page_number].iloc[0].values:
                value_to_ignore_column = df_list[page_number].iloc[0].values.tolist().index(value_to_ignore)
                print(f'\tRemoving {value_to_ignore} from column {value_to_ignore_column}')
                df_list[page_number].iloc[0, value_to_ignore_column:-1] = df_list[page_number].iloc[0, value_to_ignore_column+1:].values

            for column in blank_columns:
                try:
                    if df_list[page_number][column].isna().sum() != len(df_list[page_number]): # 2023-03-30 22:45:
                        blank_column_index = df_list[page_number].columns.tolist().index(column)
                        print(f'\tShifting data from column {blank_column_index}({column})')
                        df_list[page_number].iloc[:,blank_column_index+1:] = df_list[page_number].iloc[:,blank_column_index:-1].values 
                        df_list[page_number].iloc[:,blank_column_index] = None
                except:
                    pass
            column_after_blank_columns_index = blank_column_index + 1
            column_after_blank_columns = df_list[page_number].columns[column_after_blank_columns_index]
            try:
                while df_list[page_number][column_after_blank_columns].isna().sum() == len(df_list[page_number]):
                    print(f'\tShifting data to the right until `{column_after_blank_columns}` column')
                    df_list[page_number].iloc[:,column_after_blank_columns_index:-1] = df_list[
                        page_number].iloc[:,column_after_blank_columns_index+1:].values 
                    df_list[page_number].drop(columns=df_list[page_number].columns[-1], inplace=True)
            except:
                pass

            file_dfs.append(df_list[page_number]) # 2023-03-30 23:04 Decrease indent
        dfs_list.append(file_dfs) # SH 2023-04-01 20:58
        

    return dfs_list, reference_dict




def parse_pdf(pdf_filenames,pages='all', last_id_column='HA', value_to_ignore='s.22(1)', 
    value_to_ignore_start_index=1, value_to_ignore_end_index=None,
    blank_columns=['workplace_name', 'workplace_address'],
    filename=None,filepath=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\BC-Covid-workplace-closures\data\raw', 
    path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\BC-Covid-workplace-closures\data\interim'):
    """
    Create dataframe from a PDF file using 
    """
    print(f'Extracting the following files:')
    if type(pdf_filenames) != list:
        pdf_filenames = [pdf_filenames]
    dfs_list = []
    reference_dict = dict()
    corrected_pages = dict()
    for index, file in enumerate(pdf_filenames):
        print(file)
        filepath = f'{filepath}/'.replace('\\','/') if filepath else ''
        path_and_file = filepath+file
        try:
            df_list = read_pdf(path_and_file, pages=pages)
        except:
            print(f'\nError parsing {file}')
        file_dfs = []
        page1_columns = df_list[0].columns.str.replace('\r', '').str.replace(r'(workplace)([a-zA-Z]+)', r'\1_\2', regex=True).to_list()
        print('number of columns:', len(page1_columns))
        if last_id_column in page1_columns:
            page1_columns_sorted = sorted(page1_columns)
        else:
            previous_columns = dfs_list[index-1].columns
            print(f'\t\tNo headers found; using headers from previous document:', previous_columns)
            if len(df_list[page_number].columns) > len(previous_columns):
                n_cols_to_pad = len(df_list[page_number].columns) - len(previous_columns)
                new_columns_padding = [f'blank{index}' for index, col in enumerate(range(n_cols_to_pad))]
                page1_columns_sorted = sorted(page1_columns) + new_columns_padding
                page1_columns = page1_columns + new_columns_padding

            elif len(df_list[page_number].columns) < len(previous_columns):
                page1_columns = (previous_columns[:len(df_list[page_number].columns)].to_list())
                page1_columns_sorted = sorted(page1_columns)
            else:
                page1_columns_sorted = (dfs_list[index-1].columns.to_list())
            page1_columns = ['Unnamed: 0' if column=='index' else column for column in page1_columns]
            print(f'\t\t**Current df columns:',page1_columns)
        
        for page_number in range(len(df_list)):
            print(f'\tpage {page_number+1}...')
            df_list[page_number].columns = df_list[page_number].columns.str.replace('\r', '').str.replace(r'(workplace)([a-zA-Z]+)', r'\1_\2', regex=True)
            if len(df_list[page_number].columns.to_list()) == len(page1_columns_sorted):
                if sorted(df_list[page_number].columns.to_list()) == page1_columns_sorted:
                    pass
                else:
                    df_list[page_number].columns = page1_columns 
                    print('\t\tKeeping page 1 columns')
            else:
                if len(df_list[page_number].columns) > len(page1_columns_sorted):
                    n_cols_to_pad = len(df_list[page_number].columns) - len(page1_columns_sorted)
                    new_columns_padding = [f'blank{index}' for index, col in enumerate(range(n_cols_to_pad))]
                    df_list[page_number].columns = page1_columns + new_columns_padding
                    print('\t\tPadding columns')
                elif len(df_list[page_number].columns) < len(page1_columns_sorted):
                    df_list[page_number].columns = page1_columns[:len(df_list[page_number].columns)]
                    print('\t\tDropping columns')
                else:
                    df_list[page_number].columns = page1_columns
                corrected_pages[file] = page_number+1
                print(f'\n\t**Fixing column parsing for page {page_number+1} of {file}**')
                print(f'\t\t{df_list[page_number].columns}')
            # Consolidate rows that were incorrectly split
            if (df_list[page_number]['Unnamed: 0'].isna().sum() > 0):
                # print('\tAttempting to consolidate rows that were incorrectly split: ', df_list[page_number].columns)
                print('\tAttempting to consolidate rows that were incorrectly split: ')
                try:
                    df_list[page_number] = merge_rows(df_list[page_number], file)
                except:
                    print(f'\t\tUnable to consolidate rows on page {page_number+1} of {file}')
                    reference_dict['unparsed documents'] = file
                    continue

            # Align the columns
            value_to_ignore_regex = rf'.*{re.escape(value_to_ignore[value_to_ignore_start_index:value_to_ignore_end_index])}.*'
            if df_list[page_number].iloc[0].str.contains(value_to_ignore_regex).any():
                row_values = df_list[page_number].fillna(0).iloc[0].values.tolist()
                for index, item in enumerate(row_values):
                    if re.match(value_to_ignore_regex, str(item)):
                        # print('found regex')
                        value_to_ignore_column = index
                        break

                print(f'\tRemoving {value_to_ignore} from column {value_to_ignore_column}')
                df_list[page_number].iloc[0, value_to_ignore_column:-1] = df_list[page_number].iloc[0, value_to_ignore_column+1:].values

            for column in blank_columns:
                try:
                    if df_list[page_number][column].isna().sum() != len(df_list[page_number]): # 2023-03-30 22:45:
                        blank_column_index = df_list[page_number].columns.tolist().index(column)
                        print(f'\tShifting data from column {blank_column_index}({column})')
                        df_list[page_number].iloc[:,blank_column_index+1:] = df_list[page_number].iloc[:,blank_column_index:-1].values 
                        df_list[page_number].iloc[:,blank_column_index] = None
                except:
                    pass
            column_after_blank_columns_index = blank_column_index + 1
            column_after_blank_columns = df_list[page_number].columns[column_after_blank_columns_index]
            try:
                while df_list[page_number][column_after_blank_columns].isna().sum() == len(df_list[page_number]):
                    print(f'\tShifting data to the right until `{column_after_blank_columns}` column')
                    df_list[page_number].iloc[:,column_after_blank_columns_index:-1] = df_list[
                        page_number].iloc[:,column_after_blank_columns_index+1:].values 
                    df_list[page_number].drop(columns=df_list[page_number].columns[-1], inplace=True)
            except:
                pass
            df_list[page_number]['doc_page_number'] = page_number + 1 # 2023-03-30 23:15
            file_dfs.append(df_list[page_number])  # 2023-03-30 23:04 Decrease indent
        
        try:
            final_file_df = pd.concat(file_dfs)
            final_file_df['file'] = file
            final_file_df.columns = final_file_df.columns.str.replace('cosed', 'closed')
            final_file_df.reset_index(inplace=True, drop=True)
            final_file_df.rename(columns={'Unnamed: 0': 'index'}, inplace=True)
            dfs_list.append(final_file_df)

            print('\n\tDone')
        except:
            print(f'Error concatenating dataframes from {file}')
            return file_dfs, {file: 'DataFrame concatenation error'}
    unique_cols_dict = {}
    for i, df in enumerate(dfs_list):
        other_dfs = dfs_list[:i] + dfs_list[i+1:]
        other_cols = set().union(*[set(other_df.columns) for other_df in other_dfs])
        unique_columns = set(df.columns) - other_cols
        unique_cols_dict[file] = unique_columns if len(unique_columns)>0 else "none"
        print(f'Columns unique to part{pdf_filenames[i]}: {unique_cols_dict[file]}')
    reference_dict['corrected pages'] = corrected_pages
    reference_dict['unique columns'] = unique_cols_dict
    if filename:
        try:
            savepickle(dfs_list, filename=filename, path=path)
            with open(f'{filepath}{filename}_reference.json','w') as json_file:
                json.dump(reference_dict, json_file)
            save_csv(pd.concat(dfs_list), filename=filename, path=path)
        except:
            print('Unable to save outputs')
    try:
        return pd.concat(dfs_list), reference_dict
    except:
        print('Unable to concatenate dataframes from different documents')
        return dfs_list, reference_dict

n_docs = 7
start = 1
filenames = [f'F22-1445 Records_Part{i}.pdf' for i in range(start,start+n_docs)]
part = 1
filename = f'F22-1445 Records_Part{part}.pdf'

dfs1_i8, reference1_i8 = parse_pdf(filename, pages='34,118', filename=None)
# dfs, reference = parse_pdf(filenames, pages='all', filename='pdfs_parsed_2023-03-30')


Extracting the following files:
F22-1445 Records_Part1.pdf
number of columns: 23
	page 1...
	Removing s.22(1) from column 2
	Shifting data from column 2(workplace_name)
	Shifting data from column 3(workplace_address)
	page 2...
		Padding columns

	**Fixing column parsing for page 2 of F22-1445 Records_Part1.pdf**
		Index(['Unnamed: 0', 'HA', 'workplace_name', 'workplace_address', 'city',
       'date_cluster_confirmed_public_health', 'date_cluster_no_longer_active',
       'earliest_reported_cases', 'latest_reported_cases',
       'workplace_classification', 'cases_when_cluster_confirmed',
       'total_cases', 'workplace_status', 'closure_y_n', 'closed_by',
       'voluntary_closure', 'closure_date', 'partially_closed', 'open',
       'days_since_last_case', 'active_clusters', 'cluster_confirmed_epidate',
       'cluster_closed_epidate', 'blank0', 'blank1'],
      dtype='object')
	Attempting to consolidate rows that were incorrectly split: 
	Removing duplicate rows from F22-1445 Recor

In [88]:
dfs1_i8

,index,HA,workplace_name,workplace_address,city,date_cluster_confirmed_public_health,date_cluster_no_longer_active,earliest_reported_cases,latest_reported_cases,workplace_classification,...,voluntary_closure,closure_date,partially_closed,open,days_since_last_case,active_clusters,cluster_confirmed_epidate,cluster_closed_epidate,doc_page_number,file
0,1.0,FHA,None,None,DELTA,4/1/2021,4/6/2021,NaN,NaN,Transportatio\rn,...,Unknown,NaN,No,No,NaN,No,13-2021,NaN,1,F22-1445 Records_Part1.pdf
1,2.0,FHA,None,None,MAPLE RIDGE,4/1/2021,4/18/2021,NaN,NaN,Restaurant/B\rar/Lounge,...,Unknown,4/6/2021,No,No,NaN,No,13-2021,14-2021,1,F22-1445 Records_Part1.pdf
2,3.0,FHA,None,None,SURREY,4/1/2021,4/22/2021,NaN,NaN,Restaurant/B\rar/Lounge,...,Unknown,4/12/2021,No,No,NaN,No,13-2021,15-2021,1,F22-1445 Records_Part1.pdf
3,4.0,FHA,None,None,SURREY,4/1/2021,4/14/2021,NaN,NaN,Office,...,Unknown,4/3/2021,No,No,NaN,No,13-2021,13-2021,1,F22-1445 Records_Part1.pdf
4,5.0,FHA,None,None,LANGLEY,4/1/2021,NaN,NaN,NaN,Warehouse,...,Unknown,NaN,No,No,NaN,Yes,13-2021,NaN,1,F22-1445 Records_Part1.pdf
5,6.0,FHA,None,None,BURNABY,4/1/2021,4/21/2021,NaN,NaN,Gym/Fitness,...,Unknown,4/7/2021,No,No,NaN,No,13-2021,14-2021,1,F22-1445 Records_Part1.pdf
6,7.0,FHA,None,None,SURREY,4/1/2021,4/23/2021,NaN,NaN,Gym/Fitness,...,Unknown,NaN,No,No,NaN,No,13-2021,NaN,1,F22-1445 Records_Part1.pdf
7,8.0,FHA,None,None,ABBOTSFORD,4/1/2021,4/19/2021,NaN,NaN,Gym/Fitness,...,Unknown,4/3/2021,No,No,NaN,No,13-2021,13-2021,1,F22-1445 Records_Part1.pdf
8,9.0,FHA,None,None,CHILLIWACK,4/1/2021,4/17/2021,NaN,NaN,Maintenance/\rTrades/Constr\ruction,...,Unknown,NaN,No,No,NaN,No,13-2021,NaN,1,F22-1445 Records_Part1.pdf
9,10.0,FHA,None,None,LANGLEY,4/1/2021,4/30/2021,NaN,NaN,Farm/Agricult\rure,...,Unknown,NaN,No,No,NaN,No,13-2021,NaN,1,F22-1445 Records_Part1.pdf


In [89]:
part = 1
filename = f'F22-1445 Records_Part{part}.pdf'
pages = '1-30'
dfs1_part1, reference1_part1 = parse_pdf(filename, pages=pages, filename=f'pdfs_parsed_04-01_part1_p{pages}')

Extracting the following files:
F22-1445 Records_Part1.pdf
number of columns: 22
	page 1...
	Shifting data from column 2(workplace_name)
	Shifting data from column 3(workplace_address)
	page 2...
	Removing s.22(1) from column 2
	Shifting data from column 2(workplace_name)
	Shifting data from column 3(workplace_address)
	page 3...
	Shifting data from column 2(workplace_name)
	Shifting data from column 3(workplace_address)
	page 4...
	Removing s.22(1) from column 2
	Shifting data from column 2(workplace_name)
	Shifting data from column 3(workplace_address)
	page 5...
	Shifting data from column 2(workplace_name)
	Shifting data from column 3(workplace_address)
	page 6...
	Shifting data from column 2(workplace_name)
	Shifting data from column 3(workplace_address)
	page 7...
	Removing s.22(1) from column 2
	Shifting data from column 2(workplace_name)
	Shifting data from column 3(workplace_address)
	page 8...
	Removing s.22(1) from column 2
	Shifting data from column 2(workplace_name)
	Shifti

In [90]:
dfs1_part1

,index,HA,workplace_name,workplace_address,city,date_cluster_confirmed_public_health,date_cluster_no_longer_active,earliest_reported_cases,latest_reported_cases,workplace_classification,...,voluntary_closure,closure_date,partially_closed,open,days_since_last_case,cluster_confirmed_epidate,cluster_closed_epidate,doc_page_number,blank0,file
0,1.0,FHA,None,None,SURREY,4/1/2021,4/17/2021,NaN,NaN,Restaurant/Bar/L\rounge,...,Unknown,NaN,No,No,NaN,13-2021,NaN,1,NaN,F22-1445 Records_Part1.pdf
1,2.0,FHA,None,None,LANGLEY,4/1/2021,4/13/2021,NaN,NaN,Other,...,Unknown,NaN,No,No,NaN,13-2021,NaN,1,NaN,F22-1445 Records_Part1.pdf
2,3.0,FHA,None,None,SURREY,4/1/2021,4/13/2021,NaN,NaN,Restaurant/Bar/L\rounge,...,Unknown,4/4/2021,No,No,NaN,13-2021,14-2021,1,NaN,F22-1445 Records_Part1.pdf
3,4.0,FHA,None,None,BURNABY,4/1/2021,4/21/2021,NaN,NaN,Gym/Fitness,...,Unknown,4/7/2021,No,No,NaN,13-2021,14-2021,1,NaN,F22-1445 Records_Part1.pdf
4,5.0,FHA,None,None,BURNABY,4/1/2021,4/14/2021,NaN,NaN,Restaurant/Bar/L\rounge,...,Unknown,4/5/2021,No,No,NaN,13-2021,14-2021,1,NaN,F22-1445 Records_Part1.pdf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
459,460.0,IHA,None,None,Kelowna,5/27/2021,NaN,5/27/2021,6/5/2021,Restaurant/Bar/L\rounge,...,Yes,5/26/2021,Unknown,No,NaN,21-2021,21-2021,30,NaN,F22-1445 Records_Part1.pdf
460,461.0,IHA,None,None,Kelowna,5/28/2021,NaN,5/28/2021,5/3/2021,Personal Care,...,No,NaN,Unknown,No,NaN,21-2021,NaN,30,NaN,F22-1445 Records_Part1.pdf
461,462.0,IHA,None,None,Kelowna,5/31/2021,NaN,5/31/2021,5/31/2021,Maintenance/Tr\rades/Constructio\rn,...,No,NaN,Unknown,No,NaN,22-2021,NaN,30,NaN,F22-1445 Records_Part1.pdf
462,463.0,IHA,None,None,Vernon,5/31/2021,NaN,5/31/2021,6/3/2021,Retail,...,No,NaN,Unknown,No,NaN,22-2021,NaN,30,NaN,F22-1445 Records_Part1.pdf


In [91]:
part = 1
filename = f'F22-1445 Records_Part{part}.pdf'
pages = '1,11'
dfs_list1_part1, reference1_list_part1 = parse_pdf_tolist(filename, pages=pages)

Extracting the following files:
F22-1445 Records_Part1.pdf
number of columns: 22
	page 1...
		Current number of columns: 22 22
		22, 22
	Shifting data from column 2(workplace_name)
	Shifting data from column 3(workplace_address)
	page 2...
		Current number of columns: 23 22
		22, 22
		Padding columns

	**Fixing column parsing for page 2 of F22-1445 Records_Part1.pdf**
		Index(['Unnamed: 0', 'HA', 'workplace_name', 'workplace_address', 'city',
       'date_cluster_confirmed_public_health', 'date_cluster_no_longer_active',
       'earliest_reported_cases', 'latest_reported_cases',
       'workplace_classification', 'cases_when_cluster_confirmed',
       'total_cases', 'workplace_status', 'closure_y_n', 'closed_by_order',
       'voluntary_closure', 'closure_date', 'partially_cosed', 'open',
       'days_since_last_case', 'cluster_confirmed_epidate',
       'cluster_closed_epidate', 'blank0'],
      dtype='object')
	Removing s.22(1) from column 2
	Shifting data from column 2(workplace_nam

In [97]:
print(dfs_list1_part1[0][1].columns)
dfs_list1_part1[0][1]

Index(['Unnamed: 0', 'HA', 'workplace_name', 'workplace_address', 'city',
       'date_cluster_confirmed_public_health', 'date_cluster_no_longer_active',
       'earliest_reported_cases', 'latest_reported_cases',
       'workplace_classification', 'cases_when_cluster_confirmed',
       'total_cases', 'workplace_status', 'closure_y_n', 'closed_by_order',
       'voluntary_closure', 'closure_date', 'partially_cosed', 'open',
       'days_since_last_case', 'cluster_confirmed_epidate',
       'cluster_closed_epidate', 'blank0'],
      dtype='object')


,Unnamed: 0,HA,workplace_name,workplace_address,city,date_cluster_confirmed_public_health,date_cluster_no_longer_active,earliest_reported_cases,latest_reported_cases,workplace_classification,...,closure_y_n,closed_by_order,voluntary_closure,closure_date,partially_cosed,open,days_since_last_case,cluster_confirmed_epidate,cluster_closed_epidate,blank0
0,167,FHA,None,None,HILLIWACK,4/16/2021,4/21/2021,NaN,NaN,Gym/Fitness,...,No,No,Unknown,NaN,No,No,NaN,15-2021,NaN,NaN
1,168,FHA,None,None,ELTA,4/16/2021,4/26/2021,NaN,NaN,Retail,...,No,No,Unknown,NaN,No,No,NaN,15-2021,NaN,NaN
2,169,FHA,None,None,ANGLEY,4/16/2021,5/8/2021,NaN,NaN,Retail,...,No,No,Unknown,NaN,No,No,NaN,15-2021,NaN,NaN
3,170,FHA,None,None,URREY,4/16/2021,4/30/2021,NaN,NaN,Retail,...,Yes,Yes,Unknown,4/18/2021,No,No,NaN,15-2021,16-2021,NaN
4,171,FHA,None,None,URNABY,4/17/2021,4/29/2021,NaN,NaN,Manufacturing/I\rndustry,...,Yes,Yes,Unknown,4/18/2021,No,No,NaN,15-2021,16-2021,NaN
5,172,FHA,None,None,ORT\rOQUITLAM,4/17/2021,4/26/2021,NaN,NaN,Transportation,...,No,No,Unknown,NaN,No,No,NaN,15-2021,NaN,NaN
6,173,NHA,None,None,nknown,4/17/2021,5/20/2021,NaN,NaN,Health Care\rFacility\r(Acute/LTC),...,No,No,No,NaN,Unknown,No,NaN,15-2021,NaN,NaN
7,174,FHA,None,None,ORT\rOQUITLAM,4/18/2021,5/6/2021,NaN,NaN,Gym/Fitness,...,Yes,Yes,Unknown,4/19/2021,No,No,NaN,16-2021,16-2021,NaN
8,175,FHA,None,None,URREY,4/18/2021,5/5/2021,NaN,NaN,Gym/Fitness,...,No,No,Unknown,NaN,No,No,NaN,16-2021,NaN,NaN
9,176,FHA,None,None,URREY,4/18/2021,5/5/2021,NaN,NaN,Retail,...,Yes,Yes,Unknown,4/20/2021,No,No,NaN,16-2021,16-2021,NaN


In [109]:
dfs_list1_part1[0][1].filter(regex='blank').notna().sum()

blank0    0
dtype: int64

In [99]:
print(dfs_list1_part1[0][0].columns)
dfs_list1_part1[0][0]

Index(['Unnamed: 0', 'HA', 'workplace_name', 'workplace_address', 'city',
       'date_cluster_confirmed_public_health', 'date_cluster_no_longer_active',
       'earliest_reported_cases', 'latest_reported_cases',
       'workplace_classification', 'cases_when_cluster_confirmed',
       'total_cases', 'workplace_status', 'closure_y_n', 'closed_by_order',
       'voluntary_closure', 'closure_date', 'partially_cosed', 'open',
       'days_since_last_case', 'cluster_confirmed_epidate',
       'cluster_closed_epidate'],
      dtype='object')


,Unnamed: 0,HA,workplace_name,workplace_address,city,date_cluster_confirmed_public_health,date_cluster_no_longer_active,earliest_reported_cases,latest_reported_cases,workplace_classification,...,workplace_status,closure_y_n,closed_by_order,voluntary_closure,closure_date,partially_cosed,open,days_since_last_case,cluster_confirmed_epidate,cluster_closed_epidate
0,1,FHA,None,None,SURREY,4/1/2021,4/17/2021,NaN,NaN,Restaurant/Bar/L\rounge,...,NaN,No,No,Unknown,NaN,No,No,NaN,13-2021,NaN
1,2,FHA,None,None,LANGLEY,4/1/2021,4/13/2021,NaN,NaN,Other,...,NaN,No,No,Unknown,NaN,No,No,NaN,13-2021,NaN
2,3,FHA,None,None,SURREY,4/1/2021,4/13/2021,NaN,NaN,Restaurant/Bar/L\rounge,...,Closed,Yes,Yes,Unknown,4/4/2021,No,No,NaN,13-2021,14-2021
3,4,FHA,None,None,BURNABY,4/1/2021,4/21/2021,NaN,NaN,Gym/Fitness,...,Closed,Yes,Yes,Unknown,4/7/2021,No,No,NaN,13-2021,14-2021
4,5,FHA,None,None,BURNABY,4/1/2021,4/14/2021,NaN,NaN,Restaurant/Bar/L\rounge,...,Closed,Yes,Yes,Unknown,4/5/2021,No,No,NaN,13-2021,14-2021
5,6,FHA,None,None,CHILLIWACK,4/1/2021,4/17/2021,NaN,NaN,Maintenance/Tr\rades/Constructio\rn,...,NaN,No,No,Unknown,NaN,No,No,NaN,13-2021,NaN
6,7,FHA,None,None,COQUITLAM,4/1/2021,4/14/2021,NaN,NaN,Office,...,NaN,No,No,Unknown,NaN,No,No,NaN,13-2021,NaN
7,8,FHA,None,None,DELTA,4/1/2021,4/6/2021,NaN,NaN,Transportation,...,NaN,No,No,Unknown,NaN,No,No,NaN,13-2021,NaN
8,9,FHA,None,None,LANGLEY,4/1/2021,4/30/2021,NaN,NaN,Farm/Agriculture,...,NaN,No,No,Unknown,NaN,No,No,NaN,13-2021,NaN
9,10,FHA,None,None,LANGLEY,4/1/2021,NaN,NaN,NaN,Warehouse,...,NaN,No,No,Unknown,NaN,No,No,NaN,13-2021,NaN


## Iteration 9

In [126]:
from tabula import read_pdf
import json
import re
import warnings
warnings.simplefilter("ignore", category=FutureWarning)

def merge_rows(df, file, id_column=['Unnamed: 0'], fill_method='bfill', groupby_column=['HA']):
    """
    Merges rows in a pandas dataframe that were parsed incorrectly into multiple rows.
    Assumes that the rows that need to be merged have the same value in a unique ID column.
    """
    print(f'\tRemoving duplicate rows from {file}...')
    print('\t\tOriginal df shape: ', df.shape)
    print(f'\t\tOriginal columns:', df.columns)
    df[id_column] = df[id_column].fillna(method=fill_method)
    
    # Group the rows by the unique ID column and aggregate the values in each group

    columns_to_fill = df.drop(columns=groupby_column).columns
    grouped_df = df.groupby(id_column+groupby_column, as_index=False).agg(
        lambda x: ''.join(str(v) for v in x if (type(v)==str) & (x.name in columns_to_fill)))
    grouped_df = grouped_df[grouped_df[groupby_column[0]] != groupby_column[0]]
    # Return the merged dataframe
    print('\t\tTransformed df shape:', grouped_df.shape)
    return grouped_df.replace('', None)



def parse_pdf_tolist(pdf_filenames, filepath=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\BC-Covid-workplace-closures\data\raw', 
    pages='all', last_id_column='HA', value_to_ignore='s.22(1)',
    blank_columns=['workplace_name', 'workplace_address'],
    filename=None,
    path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\BC-Covid-workplace-closures\data\interim'):
    """
    Create dataframe from a PDF file using 
    """
    print(f'Extracting the following files:')
    if type(pdf_filenames) != list:
        pdf_filenames = [pdf_filenames]
    dfs_list = []
    reference_dict = dict()
    corrected_pages = dict()
    for index, file in enumerate(pdf_filenames):
        print(file)
        filepath = f'{filepath}/'.replace('\\','/') if filepath else ''
        path_and_file = filepath+file
        try:
            df_list = read_pdf(path_and_file, pages=pages)
        except:
            print(f'\nError parsing {file}')
        file_dfs = []
        page1_columns = df_list[0].columns.str.replace('\r', '').str.replace(r'(workplace)([a-zA-Z]+)', r'\1_\2', regex=True).to_list()
        print('number of columns:', len(page1_columns))
        if last_id_column in page1_columns:
            page1_columns_sorted = sorted(page1_columns)
        else:
            previous_columns = dfs_list[index-1].columns
            print(f'\t\tNo headers found; using headers from previous document:', previous_columns)
            if len(df_list[page_number].columns) > len(previous_columns):
                n_cols_to_pad = len(df_list[page_number].columns) - len(previous_columns)
                new_columns_padding = [f'blank{index}' for index, col in enumerate(range(n_cols_to_pad))]
                page1_columns_sorted = sorted(page1_columns) + new_columns_padding
                page1_columns = page1_columns + new_columns_padding

            elif len(df_list[page_number].columns) < len(previous_columns):
                page1_columns = (previous_columns[:len(df_list[page_number].columns)].to_list())
                page1_columns_sorted = sorted(page1_columns)
            else:
                page1_columns_sorted = (dfs_list[index-1].columns.to_list())
            page1_columns = ['Unnamed: 0' if column=='index' else column for column in page1_columns]
            print(f'\t\t**Current df columns:',page1_columns)
        
        for page_number in range(len(df_list)):
            print(f'\tpage {page_number+1}...')
            df_list[page_number].columns = df_list[page_number].columns.str.replace('\r', '').str.replace(r'(workplace)([a-zA-Z]+)', r'\1_\2', regex=True)
            print('\t\tCurrent number of columns:', len(df_list[page_number].columns), len(page1_columns_sorted))
            print(f'\t\t{len(page1_columns_sorted)}, {len(page1_columns)}')
            if len(df_list[page_number].columns.to_list()) == len(page1_columns_sorted):
                if sorted(df_list[page_number].columns.to_list()) == page1_columns_sorted:
                    pass
                else:
                    df_list[page_number].columns = page1_columns 
                    print('\t\tKeeping page 1 columns')
            else:
                if len(df_list[page_number].columns) > len(page1_columns_sorted):
                    n_cols_to_pad = len(df_list[page_number].columns) - len(page1_columns_sorted)
                    new_columns_padding = [f'blank{index}' for index, col in enumerate(range(n_cols_to_pad))]
                    df_list[page_number].columns = page1_columns + new_columns_padding
                    print('\t\tPadding columns')
                elif len(df_list[page_number].columns) < len(page1_columns_sorted):
                    df_list[page_number].columns = page1_columns[:len(df_list[page_number].columns)]
                    print('\t\tDropping columns')
                else:
                    df_list[page_number].columns = page1_columns
                corrected_pages[file] = page_number+1
                print(f'\n\t**Fixing column parsing for page {page_number+1} of {file}**')
                print(f'\t\t{df_list[page_number].columns}')
            # Consolidate rows that were incorrectly split
            if (df_list[page_number]['Unnamed: 0'].isna().sum() > 0):
                # print('\tAttempting to consolidate rows that were incorrectly split: ', df_list[page_number].columns)
                print('\tAttempting to consolidate rows that were incorrectly split: ')
                try:
                    df_list[page_number] = merge_rows(df_list[page_number], file)
                except:
                    print(f'\t\tUnable to consolidate rows on page {page_number+1} of {file}')
                    reference_dict['unparsed documents'] = file
                    continue

            # Align the columns
            if value_to_ignore in df_list[page_number].iloc[0].values:
                value_to_ignore_column = df_list[page_number].iloc[0].values.tolist().index(value_to_ignore)
                print(f'\tRemoving {value_to_ignore} from column {value_to_ignore_column}')
                df_list[page_number].iloc[0, value_to_ignore_column:-1] = df_list[page_number].iloc[0, value_to_ignore_column+1:].values

            for column in blank_columns:
                try:
                    if df_list[page_number][column].isna().sum() != len(df_list[page_number]): # 2023-03-30 22:45:
                        blank_column_index = df_list[page_number].columns.tolist().index(column)
                        print(f'\tShifting data from column {blank_column_index}({column})')
                        df_list[page_number].iloc[:,blank_column_index+1:] = df_list[page_number].iloc[:,blank_column_index:-1].values 
                        df_list[page_number].iloc[:,blank_column_index] = None
                except:
                    pass
            column_after_blank_columns_index = blank_column_index + 1
            column_after_blank_columns = df_list[page_number].columns[column_after_blank_columns_index]
            try:
                while df_list[page_number][column_after_blank_columns].isna().sum() == len(df_list[page_number]):
                    print(f'\tShifting data to the right until `{column_after_blank_columns}` column')
                    df_list[page_number].iloc[:,column_after_blank_columns_index:-1] = df_list[
                        page_number].iloc[:,column_after_blank_columns_index+1:].values 
                    df_list[page_number].drop(columns=df_list[page_number].columns[-1], inplace=True)
            except:
                pass

            file_dfs.append(df_list[page_number]) # 2023-03-30 23:04 Decrease indent
        dfs_list.append(file_dfs) # SH 2023-04-01 20:58
        

    return dfs_list, reference_dict




def parse_pdf(pdf_filenames,pages='all', last_id_column='HA', value_to_ignore='s.22(1)', 
    value_to_ignore_start_index=1, value_to_ignore_end_index=None,
    blank_columns=['workplace_name', 'workplace_address'],
    filename=None,filepath=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\BC-Covid-workplace-closures\data\raw', 
    path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\BC-Covid-workplace-closures\data\interim'):
    """
    Create dataframe from a PDF file using 
    """
    print(f'Extracting the following files:')
    if type(pdf_filenames) != list:
        pdf_filenames = [pdf_filenames]
    dfs_list = []
    reference_dict = dict()
    corrected_pages = dict()
    for index, file in enumerate(pdf_filenames):
        print(file)
        filepath = f'{filepath}/'.replace('\\','/') if filepath else ''
        path_and_file = filepath+file
        try:
            df_list = read_pdf(path_and_file, pages=pages)
        except:
            print(f'\nError parsing {file}')
        file_dfs = []
        page1_columns = df_list[0].columns.str.replace('\r', '').str.replace(r'(workplace)([a-zA-Z]+)', r'\1_\2', regex=True).to_list()
        print('number of columns:', len(page1_columns))
        if last_id_column in page1_columns:
            page1_columns_sorted = sorted(page1_columns)
        else:
            previous_columns = dfs_list[index-1].columns
            print(f'\t\tNo headers found; using headers from previous document:', previous_columns)
            if len(df_list[page_number].columns) > len(previous_columns):
                n_cols_to_pad = len(df_list[page_number].columns) - len(previous_columns)
                new_columns_padding = [f'blank{index}' for index, col in enumerate(range(n_cols_to_pad))]
                page1_columns_sorted = sorted(page1_columns) + new_columns_padding
                page1_columns = page1_columns + new_columns_padding

            elif len(df_list[page_number].columns) < len(previous_columns):
                page1_columns = (previous_columns[:len(df_list[page_number].columns)].to_list())
                page1_columns_sorted = sorted(page1_columns)
            else:
                page1_columns_sorted = (dfs_list[index-1].columns.to_list())
            page1_columns = ['Unnamed: 0' if column=='index' else column for column in page1_columns]
            print(f'\t\t**Current df columns:',page1_columns)
        
        for page_number in range(len(df_list)):
            print(f'\tpage {page_number+1}...')
            df_list[page_number].columns = df_list[page_number].columns.str.replace('\r', '').str.replace(r'(workplace)([a-zA-Z]+)', r'\1_\2', regex=True)
            if len(df_list[page_number].columns.to_list()) == len(page1_columns_sorted):
                if sorted(df_list[page_number].columns.to_list()) == page1_columns_sorted:
                    pass
                else:
                    df_list[page_number].columns = page1_columns 
                    print('\t\tKeeping page 1 columns')
            else:
                if len(df_list[page_number].columns) > len(page1_columns_sorted):
                    n_cols_to_pad = len(df_list[page_number].columns) - len(page1_columns_sorted)
                    new_columns_padding = [f'blank{index}' for index, col in enumerate(range(n_cols_to_pad))]
                    df_list[page_number].columns = page1_columns + new_columns_padding
                    print('\t\tPadding columns')
                elif len(df_list[page_number].columns) < len(page1_columns_sorted):
                    df_list[page_number].columns = page1_columns[:len(df_list[page_number].columns)]
                    print('\t\tDropping columns')
                else:
                    df_list[page_number].columns = page1_columns
                corrected_pages[file] = page_number+1
                print(f'\n\t**Fixing column parsing for page {page_number+1} of {file}**')
                print(f'\t\t{df_list[page_number].columns}')
            # Consolidate rows that were incorrectly split
            if (df_list[page_number]['Unnamed: 0'].isna().sum() > 0):
                # print('\tAttempting to consolidate rows that were incorrectly split: ', df_list[page_number].columns)
                print('\tAttempting to consolidate rows that were incorrectly split: ')
                try:
                    df_list[page_number] = merge_rows(df_list[page_number], file)
                except:
                    print(f'\t\tUnable to consolidate rows on page {page_number+1} of {file}')
                    reference_dict['unparsed documents'] = file
                    continue

            # Align the columns
            value_to_ignore_regex = rf'.*{re.escape(value_to_ignore[value_to_ignore_start_index:value_to_ignore_end_index])}.*'
            if df_list[page_number].iloc[0].str.contains(value_to_ignore_regex).any():
                row_values = df_list[page_number].fillna(0).iloc[0].values.tolist()
                for index, item in enumerate(row_values):
                    if re.match(value_to_ignore_regex, str(item)):
                        # print('found regex')
                        value_to_ignore_column = index
                        break

                print(f'\tRemoving {value_to_ignore} from column {value_to_ignore_column}')
                df_list[page_number].iloc[0, value_to_ignore_column:-1] = df_list[page_number].iloc[0, value_to_ignore_column+1:].values

            for column in blank_columns:
                try:
                    if df_list[page_number][column].isna().sum() != len(df_list[page_number]): # 2023-03-30 22:45:
                        blank_column_index = df_list[page_number].columns.tolist().index(column)
                        print(f'\tShifting data from column {blank_column_index}({column})')
                        df_list[page_number].iloc[:,blank_column_index+1:] = df_list[page_number].iloc[:,blank_column_index:-1].values 
                        df_list[page_number].iloc[:,blank_column_index] = None
                except:
                    pass
            column_after_blank_columns_index = blank_column_index + 1
            column_after_blank_columns = df_list[page_number].columns[column_after_blank_columns_index]
            try:
                while df_list[page_number][column_after_blank_columns].isna().sum() == len(df_list[page_number]):
                    print(f'\tShifting data to the right until `{column_after_blank_columns}` column')
                    df_list[page_number].iloc[:,column_after_blank_columns_index:-1] = df_list[
                        page_number].iloc[:,column_after_blank_columns_index+1:].values 
                    df_list[page_number].drop(columns=df_list[page_number].columns[-1], inplace=True)
            except:
                pass
            try:
                for column in df_list[page_number].filter(regex='blank').columns:
                    if df_list[page_number][column].notna().sum() == 0:
                        df_list[page_number].drop(columns=column, inplace=True)
                        print(f'\tDropping blank column `{column}`')
            except:
                pass
            df_list[page_number]['doc_page_number'] = page_number + 1 # 2023-03-30 23:15
            file_dfs.append(df_list[page_number])  # 2023-03-30 23:04 Decrease indent
        
        try:
            final_file_df = pd.concat(file_dfs)
            final_file_df['file'] = file
            final_file_df.columns = final_file_df.columns.str.replace('cosed', 'closed')
            final_file_df.reset_index(inplace=True, drop=True)
            final_file_df.rename(columns={'Unnamed: 0': 'index'}, inplace=True)
            dfs_list.append(final_file_df)

            print('\n\tDone')
        except:
            print(f'Error concatenating dataframes from {file}')
            return file_dfs, {file: 'DataFrame concatenation error'}
    unique_cols_dict = {}
    for i, df in enumerate(dfs_list):
        other_dfs = dfs_list[:i] + dfs_list[i+1:]
        other_cols = set().union(*[set(other_df.columns) for other_df in other_dfs])
        unique_columns = set(df.columns) - other_cols
        unique_cols_dict[file] = unique_columns if len(unique_columns)>0 else "none"
        print(f'Columns unique to part{pdf_filenames[i]}: {unique_cols_dict[file]}')
    reference_dict['corrected pages'] = corrected_pages
    reference_dict['unique columns'] = unique_cols_dict
    if filename:
        try:
            savepickle(dfs_list, filename=filename, path=path)
            with open(f'{filepath}{filename}_reference.json','w') as json_file:
                json.dump(reference_dict, json_file)
            save_csv(pd.concat(dfs_list), filename=filename, path=path)
        except:
            print('Unable to save outputs')
    try:
        return pd.concat(dfs_list), reference_dict
    except:
        print('Unable to concatenate dataframes from different documents')
        return dfs_list, reference_dict

n_docs = 7
start = 1
filenames = [f'F22-1445 Records_Part{i}.pdf' for i in range(start,start+n_docs)]
part = 1
filename = f'F22-1445 Records_Part{part}.pdf'

dfs1_i9, reference1_i9 = parse_pdf(filename, pages='1,11', filename=None)
# dfs, reference = parse_pdf(filenames, pages='all', filename='pdfs_parsed_2023-03-30')


Extracting the following files:
F22-1445 Records_Part1.pdf
number of columns: 22
	page 1...
	Shifting data from column 2(workplace_name)
	Shifting data from column 3(workplace_address)
	page 2...
		Padding columns

	**Fixing column parsing for page 2 of F22-1445 Records_Part1.pdf**
		Index(['Unnamed: 0', 'HA', 'workplace_name', 'workplace_address', 'city',
       'date_cluster_confirmed_public_health', 'date_cluster_no_longer_active',
       'earliest_reported_cases', 'latest_reported_cases',
       'workplace_classification', 'cases_when_cluster_confirmed',
       'total_cases', 'workplace_status', 'closure_y_n', 'closed_by_order',
       'voluntary_closure', 'closure_date', 'partially_cosed', 'open',
       'days_since_last_case', 'cluster_confirmed_epidate',
       'cluster_closed_epidate', 'blank0'],
      dtype='object')
	Removing s.22(1) from column 2
	Shifting data from column 2(workplace_name)
	Shifting data from column 3(workplace_address)
	Dropping blank column `blank0`

	Don

In [125]:
print(dfs1_i9.columns)
dfs1_i9

Index(['index', 'HA', 'workplace_name', 'workplace_address', 'city',
       'date_cluster_confirmed_public_health', 'date_cluster_no_longer_active',
       'earliest_reported_cases', 'latest_reported_cases',
       'workplace_classification', 'cases_when_cluster_confirmed',
       'total_cases', 'workplace_status', 'closure_y_n', 'closed_by_order',
       'voluntary_closure', 'closure_date', 'partially_closed', 'open',
       'days_since_last_case', 'cluster_confirmed_epidate',
       'cluster_closed_epidate', 'doc_page_number', 'file'],
      dtype='object')


,index,HA,workplace_name,workplace_address,city,date_cluster_confirmed_public_health,date_cluster_no_longer_active,earliest_reported_cases,latest_reported_cases,workplace_classification,...,closed_by_order,voluntary_closure,closure_date,partially_closed,open,days_since_last_case,cluster_confirmed_epidate,cluster_closed_epidate,doc_page_number,file
0,1,FHA,None,None,SURREY,4/1/2021,4/17/2021,NaN,NaN,Restaurant/Bar/L\rounge,...,No,Unknown,NaN,No,No,NaN,13-2021,NaN,1,F22-1445 Records_Part1.pdf
1,2,FHA,None,None,LANGLEY,4/1/2021,4/13/2021,NaN,NaN,Other,...,No,Unknown,NaN,No,No,NaN,13-2021,NaN,1,F22-1445 Records_Part1.pdf
2,3,FHA,None,None,SURREY,4/1/2021,4/13/2021,NaN,NaN,Restaurant/Bar/L\rounge,...,Yes,Unknown,4/4/2021,No,No,NaN,13-2021,14-2021,1,F22-1445 Records_Part1.pdf
3,4,FHA,None,None,BURNABY,4/1/2021,4/21/2021,NaN,NaN,Gym/Fitness,...,Yes,Unknown,4/7/2021,No,No,NaN,13-2021,14-2021,1,F22-1445 Records_Part1.pdf
4,5,FHA,None,None,BURNABY,4/1/2021,4/14/2021,NaN,NaN,Restaurant/Bar/L\rounge,...,Yes,Unknown,4/5/2021,No,No,NaN,13-2021,14-2021,1,F22-1445 Records_Part1.pdf
5,6,FHA,None,None,CHILLIWACK,4/1/2021,4/17/2021,NaN,NaN,Maintenance/Tr\rades/Constructio\rn,...,No,Unknown,NaN,No,No,NaN,13-2021,NaN,1,F22-1445 Records_Part1.pdf
6,7,FHA,None,None,COQUITLAM,4/1/2021,4/14/2021,NaN,NaN,Office,...,No,Unknown,NaN,No,No,NaN,13-2021,NaN,1,F22-1445 Records_Part1.pdf
7,8,FHA,None,None,DELTA,4/1/2021,4/6/2021,NaN,NaN,Transportation,...,No,Unknown,NaN,No,No,NaN,13-2021,NaN,1,F22-1445 Records_Part1.pdf
8,9,FHA,None,None,LANGLEY,4/1/2021,4/30/2021,NaN,NaN,Farm/Agriculture,...,No,Unknown,NaN,No,No,NaN,13-2021,NaN,1,F22-1445 Records_Part1.pdf
9,10,FHA,None,None,LANGLEY,4/1/2021,NaN,NaN,NaN,Warehouse,...,No,Unknown,NaN,No,No,NaN,13-2021,NaN,1,F22-1445 Records_Part1.pdf


In [127]:
part = 1
filename = f'F22-1445 Records_Part{part}.pdf'
pages = '1-30'
dfs1_part1_i9, reference1_part1_i9 = parse_pdf(filename, pages=pages, filename=f'pdfs_parsed_04-01_part1_p{pages}')

Extracting the following files:
F22-1445 Records_Part1.pdf
number of columns: 22
	page 1...
	Shifting data from column 2(workplace_name)
	Shifting data from column 3(workplace_address)
	page 2...
	Removing s.22(1) from column 2
	Shifting data from column 2(workplace_name)
	Shifting data from column 3(workplace_address)
	page 3...
	Shifting data from column 2(workplace_name)
	Shifting data from column 3(workplace_address)
	page 4...
	Removing s.22(1) from column 2
	Shifting data from column 2(workplace_name)
	Shifting data from column 3(workplace_address)
	page 5...
	Shifting data from column 2(workplace_name)
	Shifting data from column 3(workplace_address)
	page 6...
	Shifting data from column 2(workplace_name)
	Shifting data from column 3(workplace_address)
	page 7...
	Removing s.22(1) from column 2
	Shifting data from column 2(workplace_name)
	Shifting data from column 3(workplace_address)
	page 8...
	Removing s.22(1) from column 2
	Shifting data from column 2(workplace_name)
	Shifti

In [128]:
dfs1_part1_i9

,index,HA,workplace_name,workplace_address,city,date_cluster_confirmed_public_health,date_cluster_no_longer_active,earliest_reported_cases,latest_reported_cases,workplace_classification,...,closed_by_order,voluntary_closure,closure_date,partially_closed,open,days_since_last_case,cluster_confirmed_epidate,cluster_closed_epidate,doc_page_number,file
0,1.0,FHA,None,None,SURREY,4/1/2021,4/17/2021,NaN,NaN,Restaurant/Bar/L\rounge,...,No,Unknown,NaN,No,No,NaN,13-2021,NaN,1,F22-1445 Records_Part1.pdf
1,2.0,FHA,None,None,LANGLEY,4/1/2021,4/13/2021,NaN,NaN,Other,...,No,Unknown,NaN,No,No,NaN,13-2021,NaN,1,F22-1445 Records_Part1.pdf
2,3.0,FHA,None,None,SURREY,4/1/2021,4/13/2021,NaN,NaN,Restaurant/Bar/L\rounge,...,Yes,Unknown,4/4/2021,No,No,NaN,13-2021,14-2021,1,F22-1445 Records_Part1.pdf
3,4.0,FHA,None,None,BURNABY,4/1/2021,4/21/2021,NaN,NaN,Gym/Fitness,...,Yes,Unknown,4/7/2021,No,No,NaN,13-2021,14-2021,1,F22-1445 Records_Part1.pdf
4,5.0,FHA,None,None,BURNABY,4/1/2021,4/14/2021,NaN,NaN,Restaurant/Bar/L\rounge,...,Yes,Unknown,4/5/2021,No,No,NaN,13-2021,14-2021,1,F22-1445 Records_Part1.pdf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
459,460.0,IHA,None,None,Kelowna,5/27/2021,NaN,5/27/2021,6/5/2021,Restaurant/Bar/L\rounge,...,No,Yes,5/26/2021,Unknown,No,NaN,21-2021,21-2021,30,F22-1445 Records_Part1.pdf
460,461.0,IHA,None,None,Kelowna,5/28/2021,NaN,5/28/2021,5/3/2021,Personal Care,...,No,No,NaN,Unknown,No,NaN,21-2021,NaN,30,F22-1445 Records_Part1.pdf
461,462.0,IHA,None,None,Kelowna,5/31/2021,NaN,5/31/2021,5/31/2021,Maintenance/Tr\rades/Constructio\rn,...,No,No,NaN,Unknown,No,NaN,22-2021,NaN,30,F22-1445 Records_Part1.pdf
462,463.0,IHA,None,None,Vernon,5/31/2021,NaN,5/31/2021,6/3/2021,Retail,...,No,No,NaN,Unknown,No,NaN,22-2021,NaN,30,F22-1445 Records_Part1.pdf


In [129]:
part = 1
filename = f'F22-1445 Records_Part{part}.pdf'
pages = '34-137'
dfs1_part2, reference1_part2 = parse_pdf(filename, pages=pages, filename=None)

Extracting the following files:
F22-1445 Records_Part1.pdf
number of columns: 23
	page 1...
	Removing s.22(1) from column 2
	Shifting data from column 2(workplace_name)
	Shifting data from column 3(workplace_address)
	page 2...
		Keeping page 1 columns
	Removing s.22(1) from column 2
	Shifting data from column 2(workplace_name)
	Shifting data from column 3(workplace_address)
	page 3...
	Removing s.22(1) from column 3
	Shifting data from column 3(workplace_address)
	page 4...
	Removing s.22(1) from column 2
	Shifting data from column 2(workplace_name)
	Shifting data from column 3(workplace_address)
	page 5...
	Removing s.22(1) from column 2
	Shifting data from column 2(workplace_name)
	Shifting data from column 3(workplace_address)
	page 6...
	Removing s.22(1) from column 2
	Shifting data from column 2(workplace_name)
	Shifting data from column 3(workplace_address)
	page 7...
	Removing s.22(1) from column 2
	Shifting data from column 2(workplace_name)
	Shifting data from column 3(workpl

In [133]:
dfs1_part2['blank0'][dfs1_part2['blank0'].notna()]

730    17-2021
Name: blank0, dtype: object

In [135]:
dfs1_part2.loc[730:]

,index,HA,workplace_name,workplace_address,city,date_cluster_confirmed_public_health,date_cluster_no_longer_active,earliest_reported_cases,latest_reported_cases,workplace_classification,...,closure_date,partially_closed,open,days_since_last_case,active_clusters,cluster_confirmed_epidate,cluster_closed_epidate,doc_page_number,blank0,file
730,241,FHA,NaN,NaN,LANGLEY,4/24/2021,5/4/2021,NaN,NaN,Retail,...,4/25/2021,No,No,NaN,No,16-2021,17-2021,40,17-2021,F22-1445 Records_Part1.pdf
731,242,FHA,NaN,NaN,SURREY,4/24/2021,5/4/2021,NaN,NaN,Retail,...,4/25/2021,No,No,NaN,No,16-2021,17-2021,40,NaN,F22-1445 Records_Part1.pdf
732,243,FHA,NaN,NaN,MAPLE RIDGE,4/24/2021,5/6/2021,NaN,NaN,Restaurant/Bar/L\rounge,...,4/24/2021,Yes,No,NaN,No,16-2021,16-2021,40,NaN,F22-1445 Records_Part1.pdf
733,244,FHA,NaN,NaN,CHILLIWACK,4/24/2021,5/14/2021,NaN,NaN,Other,...,4/24/2021,No,No,NaN,No,16-2021,16-2021,40,NaN,F22-1445 Records_Part1.pdf
734,245,FHA,NaN,NaN,ABBOTSFORD,4/25/2021,5/12/2021,NaN,NaN,Gym/Fitness,...,NaN,No,No,NaN,No,17-2021,NaN,40,NaN,F22-1445 Records_Part1.pdf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1703,174,FHA,None,None,SURREY,4/14/2021,4/27/2021,NaN,NaN,Retail,...,4/16/2021,No,No,NaN,No,15-2021,15-2021,94,NaN,F22-1445 Records_Part1.pdf
1704,175,FHA,None,None,SURREY,4/14/2021,4/24/2021,NaN,NaN,Retail,...,4/14/2021,No,No,NaN,No,15-2021,15-2021,94,NaN,F22-1445 Records_Part1.pdf
1705,176,FHA,None,None,SURREY,4/14/2021,4/24/2021,NaN,NaN,Restaurant/Bar\r/Lounge,...,4/15/2021,No,No,NaN,No,15-2021,15-2021,94,NaN,F22-1445 Records_Part1.pdf
1706,177,FHA,None,None,BURNABY,4/14/2021,4/27/2021,NaN,NaN,Transportation,...,NaN,No,No,NaN,No,15-2021,NaN,94,NaN,F22-1445 Records_Part1.pdf


## Iteration 10

In [150]:
from tabula import read_pdf
import json
import re
import warnings
warnings.simplefilter("ignore", category=FutureWarning)

def merge_rows(df, file, id_column=['Unnamed: 0'], fill_method='bfill', groupby_column=['HA']):
    """
    Merges rows in a pandas dataframe that were parsed incorrectly into multiple rows.
    Assumes that the rows that need to be merged have the same value in a unique ID column.
    """
    print(f'\tRemoving duplicate rows from {file}...')
    print('\t\tOriginal df shape: ', df.shape)
    print(f'\t\tOriginal columns:', df.columns)
    df[id_column] = df[id_column].fillna(method=fill_method)
    
    # Group the rows by the unique ID column and aggregate the values in each group

    columns_to_fill = df.drop(columns=groupby_column).columns
    grouped_df = df.groupby(id_column+groupby_column, as_index=False).agg(
        lambda x: ''.join(str(v) for v in x if (type(v)==str) & (x.name in columns_to_fill)))
    grouped_df = grouped_df[grouped_df[groupby_column[0]] != groupby_column[0]]
    # Return the merged dataframe
    print('\t\tTransformed df shape:', grouped_df.shape)
    return grouped_df.replace('', None)



def parse_pdf_tolist(pdf_filenames, filepath=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\BC-Covid-workplace-closures\data\raw', 
    pages='all', last_id_column='HA', value_to_ignore='s.22(1)',
    blank_columns=['workplace_name', 'workplace_address'],
    filename=None,
    path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\BC-Covid-workplace-closures\data\interim'):
    """
    Create dataframe from a PDF file using 
    """
    print(f'Extracting the following files:')
    if type(pdf_filenames) != list:
        pdf_filenames = [pdf_filenames]
    dfs_list = []
    reference_dict = dict()
    corrected_pages = dict()
    for index, file in enumerate(pdf_filenames):
        print(file)
        filepath = f'{filepath}/'.replace('\\','/') if filepath else ''
        path_and_file = filepath+file
        try:
            df_list = read_pdf(path_and_file, pages=pages)
        except:
            print(f'\nError parsing {file}')
        file_dfs = []
        page1_columns = df_list[0].columns.str.replace('\r', '').str.replace(r'(workplace)([a-zA-Z]+)', r'\1_\2', regex=True).to_list()
        print('number of columns:', len(page1_columns))
        if last_id_column in page1_columns:
            page1_columns_sorted = sorted(page1_columns)
        else:
            previous_columns = dfs_list[index-1].columns
            print(f'\t\tNo headers found; using headers from previous document:', previous_columns)
            if len(df_list[page_number].columns) > len(previous_columns):
                n_cols_to_pad = len(df_list[page_number].columns) - len(previous_columns)
                new_columns_padding = [f'blank{index}' for index, col in enumerate(range(n_cols_to_pad))]
                page1_columns_sorted = sorted(page1_columns) + new_columns_padding
                page1_columns = page1_columns + new_columns_padding

            elif len(df_list[page_number].columns) < len(previous_columns):
                page1_columns = (previous_columns[:len(df_list[page_number].columns)].to_list())
                page1_columns_sorted = sorted(page1_columns)
            else:
                page1_columns_sorted = (dfs_list[index-1].columns.to_list())
            page1_columns = ['Unnamed: 0' if column=='index' else column for column in page1_columns]
            print(f'\t\t**Current df columns:',page1_columns)
        
        for page_number in range(len(df_list)):
            print(f'\tpage {page_number+1}...')
            df_list[page_number].columns = df_list[page_number].columns.str.replace('\r', '').str.replace(r'(workplace)([a-zA-Z]+)', r'\1_\2', regex=True)
            print('\t\tCurrent number of columns:', len(df_list[page_number].columns), len(page1_columns_sorted))
            print(f'\t\t{len(page1_columns_sorted)}, {len(page1_columns)}')
            if len(df_list[page_number].columns.to_list()) == len(page1_columns_sorted):
                if sorted(df_list[page_number].columns.to_list()) == page1_columns_sorted:
                    pass
                else:
                    df_list[page_number].columns = page1_columns 
                    print('\t\tKeeping page 1 columns')
            else:
                if len(df_list[page_number].columns) > len(page1_columns_sorted):
                    n_cols_to_pad = len(df_list[page_number].columns) - len(page1_columns_sorted)
                    new_columns_padding = [f'blank{index}' for index, col in enumerate(range(n_cols_to_pad))]
                    df_list[page_number].columns = page1_columns + new_columns_padding
                    print('\t\tPadding columns')
                elif len(df_list[page_number].columns) < len(page1_columns_sorted):
                    df_list[page_number].columns = page1_columns[:len(df_list[page_number].columns)]
                    print('\t\tDropping columns')
                else:
                    df_list[page_number].columns = page1_columns
                corrected_pages[file] = page_number+1
                print(f'\n\t**Fixing column parsing for page {page_number+1} of {file}**')
                print(f'\t\t{df_list[page_number].columns}')
            # Consolidate rows that were incorrectly split
            if (df_list[page_number]['Unnamed: 0'].isna().sum() > 0):
                # print('\tAttempting to consolidate rows that were incorrectly split: ', df_list[page_number].columns)
                print('\tAttempting to consolidate rows that were incorrectly split: ')
                try:
                    df_list[page_number] = merge_rows(df_list[page_number], file)
                except:
                    print(f'\t\tUnable to consolidate rows on page {page_number+1} of {file}')
                    reference_dict['unparsed documents'] = file
                    continue

            # Align the columns
            if value_to_ignore in df_list[page_number].iloc[0].values:
                value_to_ignore_column = df_list[page_number].iloc[0].values.tolist().index(value_to_ignore)
                print(f'\tRemoving {value_to_ignore} from column {value_to_ignore_column}')
                df_list[page_number].iloc[0, value_to_ignore_column:-1] = df_list[page_number].iloc[0, value_to_ignore_column+1:].values
                df_list[page_number].drop(columns=df_list[page_number].columns[-1], inplace=True)

            for column in blank_columns:
                try:
                    if df_list[page_number][column].isna().sum() != len(df_list[page_number]): # 2023-03-30 22:45:
                        blank_column_index = df_list[page_number].columns.tolist().index(column)
                        print(f'\tShifting data from column {blank_column_index}({column})')
                        df_list[page_number].iloc[:,blank_column_index+1:] = df_list[page_number].iloc[:,blank_column_index:-1].values 
                        df_list[page_number].iloc[:,blank_column_index] = None
                except:
                    pass
            column_after_blank_columns_index = blank_column_index + 1
            column_after_blank_columns = df_list[page_number].columns[column_after_blank_columns_index]
            try:
                while df_list[page_number][column_after_blank_columns].isna().sum() == len(df_list[page_number]):
                    print(f'\tShifting data to the right until `{column_after_blank_columns}` column')
                    df_list[page_number].iloc[:,column_after_blank_columns_index:-1] = df_list[
                        page_number].iloc[:,column_after_blank_columns_index+1:].values 
                    df_list[page_number].drop(columns=df_list[page_number].columns[-1], inplace=True)
            except:
                pass
            try:
                for column in df_list[page_number].filter(regex='blank').columns:
                    if df_list[page_number][column].notna().sum() == 0:
                        df_list[page_number].drop(columns=column, inplace=True)
                        print(f'\tDropping blank column `{column}`')
            except:
                pass

            file_dfs.append(df_list[page_number]) # 2023-03-30 23:04 Decrease indent
        dfs_list.append(file_dfs) # SH 2023-04-01 20:58
        

    return dfs_list, reference_dict




def parse_pdf(pdf_filenames,pages='all', last_id_column='HA', value_to_ignore='s.22(1)', 
    value_to_ignore_start_index=1, value_to_ignore_end_index=None,
    blank_columns=['workplace_name', 'workplace_address'],
    filename=None,filepath=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\BC-Covid-workplace-closures\data\raw', 
    path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\BC-Covid-workplace-closures\data\interim'):
    """
    Create dataframe from a PDF file using 
    """
    print(f'Extracting the following files:')
    if type(pdf_filenames) != list:
        pdf_filenames = [pdf_filenames]
    dfs_list = []
    reference_dict = dict()
    corrected_pages = dict()
    for index, file in enumerate(pdf_filenames):
        print(file)
        filepath = f'{filepath}/'.replace('\\','/') if filepath else ''
        path_and_file = filepath+file
        try:
            df_list = read_pdf(path_and_file, pages=pages)
        except:
            print(f'\nError parsing {file}')
        file_dfs = []
        page1_columns = df_list[0].columns.str.replace('\r', '').str.replace(r'(workplace)([a-zA-Z]+)', r'\1_\2', regex=True).to_list()
        print('number of columns:', len(page1_columns))
        if last_id_column in page1_columns:
            page1_columns_sorted = sorted(page1_columns)
        else:
            previous_columns = dfs_list[index-1].columns
            print(f'\t\tNo headers found; using headers from previous document:', previous_columns)
            if len(df_list[page_number].columns) > len(previous_columns):
                n_cols_to_pad = len(df_list[page_number].columns) - len(previous_columns)
                new_columns_padding = [f'blank{index}' for index, col in enumerate(range(n_cols_to_pad))]
                page1_columns_sorted = sorted(page1_columns) + new_columns_padding
                page1_columns = page1_columns + new_columns_padding

            elif len(df_list[page_number].columns) < len(previous_columns):
                page1_columns = (previous_columns[:len(df_list[page_number].columns)].to_list())
                page1_columns_sorted = sorted(page1_columns)
            else:
                page1_columns_sorted = (dfs_list[index-1].columns.to_list())
            page1_columns = ['Unnamed: 0' if column=='index' else column for column in page1_columns]
            print(f'\t\t**Current df columns:',page1_columns)
        
        for page_number in range(len(df_list)):
            print(f'\tpage {page_number+1}...')
            df_list[page_number].columns = df_list[page_number].columns.str.replace('\r', '').str.replace(r'(workplace)([a-zA-Z]+)', r'\1_\2', regex=True)
            if len(df_list[page_number].columns.to_list()) == len(page1_columns_sorted):
                if sorted(df_list[page_number].columns.to_list()) == page1_columns_sorted:
                    pass
                else:
                    df_list[page_number].columns = page1_columns 
                    print('\t\tKeeping page 1 columns')
            else:
                if len(df_list[page_number].columns) > len(page1_columns_sorted):
                    n_cols_to_pad = len(df_list[page_number].columns) - len(page1_columns_sorted)
                    new_columns_padding = [f'blank{index}' for index, col in enumerate(range(n_cols_to_pad))]
                    df_list[page_number].columns = page1_columns + new_columns_padding
                    print('\t\tPadding columns')
                elif len(df_list[page_number].columns) < len(page1_columns_sorted):
                    df_list[page_number].columns = page1_columns[:len(df_list[page_number].columns)]
                    print('\t\tDropping columns')
                else:
                    df_list[page_number].columns = page1_columns
                corrected_pages[file] = page_number+1
                print(f'\n\t**Fixing column parsing for page {page_number+1} of {file}**')
                print(f'\t\t{df_list[page_number].columns}')
            # Consolidate rows that were incorrectly split
            if (df_list[page_number]['Unnamed: 0'].isna().sum() > 0):
                # print('\tAttempting to consolidate rows that were incorrectly split: ', df_list[page_number].columns)
                print('\tAttempting to consolidate rows that were incorrectly split: ')
                try:
                    df_list[page_number] = merge_rows(df_list[page_number], file)
                except:
                    print(f'\t\tUnable to consolidate rows on page {page_number+1} of {file}')
                    reference_dict['unparsed documents'] = file
                    continue

            # Align the columns
            value_to_ignore_regex = rf'.*{re.escape(value_to_ignore[value_to_ignore_start_index:value_to_ignore_end_index])}.*'
            if df_list[page_number].iloc[0].str.contains(value_to_ignore_regex).any():
                row_values = df_list[page_number].fillna(0).iloc[0].values.tolist()
                for index, item in enumerate(row_values):
                    if re.match(value_to_ignore_regex, str(item)):
                        value_to_ignore_column = index
                        break

                print(f'\tRemoving {value_to_ignore} from column {value_to_ignore_column}')
                df_list[page_number].iloc[0, value_to_ignore_column:-1] = df_list[page_number].iloc[0, value_to_ignore_column+1:].values
                df_list[page_number].drop(columns=df_list[page_number].columns[-1], inplace=True)

            for column in blank_columns:
                try:
                    if df_list[page_number][column].isna().sum() != len(df_list[page_number]): # 2023-03-30 22:45:
                        blank_column_index = df_list[page_number].columns.tolist().index(column)
                        print(f'\tShifting data from column {blank_column_index}({column})')
                        df_list[page_number].iloc[:,blank_column_index+1:] = df_list[page_number].iloc[:,blank_column_index:-1].values 
                        df_list[page_number].iloc[:,blank_column_index] = None
                except:
                    pass
            column_after_blank_columns_index = blank_column_index + 1
            column_after_blank_columns = df_list[page_number].columns[column_after_blank_columns_index]
            try:
                while df_list[page_number][column_after_blank_columns].isna().sum() == len(df_list[page_number]):
                    print(f'\tShifting data to the right until `{column_after_blank_columns}` column')
                    df_list[page_number].iloc[:,column_after_blank_columns_index:-1] = df_list[
                        page_number].iloc[:,column_after_blank_columns_index+1:].values 
                    df_list[page_number].drop(columns=df_list[page_number].columns[-1], inplace=True)
            except:
                pass
            try:
                for column in df_list[page_number].filter(regex='blank').columns:
                    if df_list[page_number][column].notna().sum() == 0:
                        df_list[page_number].drop(columns=column, inplace=True)
                        print(f'\tDropping blank column `{column}`')
            except:
                pass
            df_list[page_number]['doc_page_number'] = page_number + 1 # 2023-03-30 23:15
            file_dfs.append(df_list[page_number])  # 2023-03-30 23:04 Decrease indent
        
        try:
            final_file_df = pd.concat(file_dfs)
            final_file_df['file'] = file
            final_file_df.columns = final_file_df.columns.str.replace('cosed', 'closed')
            final_file_df.reset_index(inplace=True, drop=True)
            final_file_df.rename(columns={'Unnamed: 0': 'index'}, inplace=True)
            dfs_list.append(final_file_df)

            print('\n\tDone')
        except:
            print(f'Error concatenating dataframes from {file}')
            return file_dfs, {file: 'DataFrame concatenation error'}
    unique_cols_dict = {}
    for i, df in enumerate(dfs_list):
        other_dfs = dfs_list[:i] + dfs_list[i+1:]
        other_cols = set().union(*[set(other_df.columns) for other_df in other_dfs])
        unique_columns = set(df.columns) - other_cols
        unique_cols_dict[file] = unique_columns if len(unique_columns)>0 else "none"
        print(f'Columns unique to part{pdf_filenames[i]}: {unique_cols_dict[file]}')
    reference_dict['corrected pages'] = corrected_pages
    reference_dict['unique columns'] = unique_cols_dict
    if filename:
        try:
            savepickle(dfs_list, filename=filename, path=path)
            with open(f'{filepath}{filename}_reference.json','w') as json_file:
                json.dump(reference_dict, json_file)
            save_csv(pd.concat(dfs_list), filename=filename, path=path)
        except:
            print('Unable to save outputs')
    try:
        return pd.concat(dfs_list), reference_dict
    except:
        print('Unable to concatenate dataframes from different documents')
        return dfs_list, reference_dict

n_docs = 7
start = 1
filenames = [f'F22-1445 Records_Part{i}.pdf' for i in range(start,start+n_docs)]
part = 1
filename = f'F22-1445 Records_Part{part}.pdf'

dfs_list1_i10, reference_list1_i10 = parse_pdf_tolist(filename, pages='34,76', filename=None)
# dfs, reference = parse_pdf(filenames, pages='all', filename='pdfs_parsed_2023-03-30')


Extracting the following files:
F22-1445 Records_Part1.pdf
number of columns: 23
	page 1...
		Current number of columns: 23 23
		23, 23
	Removing s.22(1) from column 2
	Shifting data from column 2(workplace_name)
	Shifting data from column 3(workplace_address)
	page 2...
		Current number of columns: 24 23
		23, 23
		Padding columns

	**Fixing column parsing for page 2 of F22-1445 Records_Part1.pdf**
		Index(['Unnamed: 0', 'HA', 'workplace_name', 'workplace_address', 'city',
       'date_cluster_confirmed_public_health', 'date_cluster_no_longer_active',
       'earliest_reported_cases', 'latest_reported_cases',
       'workplace_classification', 'cases_when_cluster_confirmed',
       'total_cases', 'workplace_status', 'closure_y_n', 'closed_by',
       'voluntary_closure', 'closure_date', 'partially_closed', 'open',
       'days_since_last_case', 'active_clusters', 'cluster_confirmed_epidate',
       'cluster_closed_epidate', 'blank0'],
      dtype='object')
	Removing s.22(1) from colum

In [151]:
dfs_list1_i10[0][1]

,Unnamed: 0,HA,workplace_name,workplace_address,city,date_cluster_confirmed_public_health,date_cluster_no_longer_active,earliest_reported_cases,latest_reported_cases,workplace_classification,...,closure_y_n,closed_by,voluntary_closure,closure_date,partially_closed,open,days_since_last_case,active_clusters,cluster_confirmed_epidate,cluster_closed_epidate
0,241,FHA,NaN,NaN,LANGLEY,4/24/2021,5/4/2021,NaN,NaN,Retail,...,Yes,Yes,Unknown,4/25/2021,No,No,NaN,No,16-2021,17-2021
1,242,FHA,NaN,NaN,SURREY,4/24/2021,5/4/2021,NaN,NaN,Retail,...,Yes,Yes,Unknown,4/25/2021,No,No,NaN,No,16-2021,17-2021
2,243,FHA,NaN,NaN,MAPLE RIDGE,4/24/2021,5/6/2021,NaN,NaN,Restaurant/Bar/L\rounge,...,Yes,No,Unknown,4/24/2021,Yes,No,NaN,No,16-2021,16-2021
3,244,FHA,NaN,NaN,CHILLIWACK,4/24/2021,5/14/2021,NaN,NaN,Other,...,Yes,Yes,Unknown,4/24/2021,No,No,NaN,No,16-2021,16-2021
4,245,FHA,NaN,NaN,ABBOTSFORD,4/25/2021,5/12/2021,NaN,NaN,Gym/Fitness,...,No,No,Unknown,NaN,No,No,NaN,No,17-2021,NaN
5,246,FHA,NaN,NaN,NEW\rWESTMINSTER,4/25/2021,5/8/2021,NaN,NaN,Warehouse,...,Yes,Yes,Unknown,4/26/2021,No,No,NaN,No,17-2021,17-2021
6,247,VIHA,NaN,NaN,Victoria,4/25/2021,NaN,4/21/2021,4/28/2021,Restaurant/Bar/L\rounge,...,Yes,No,Yes,NaN,Unknown,No,48,No,17-2021,NaN
7,248,VIHA,NaN,NaN,Langford,4/25/2021,NaN,4/18/2021,4/25/2021,Office,...,Yes,No,Yes,NaN,Unknown,No,51,No,17-2021,NaN
8,249,NHA,NaN,NaN,Unknown,4/25/2021,5/3/2021,NaN,NaN,Restaurant/Bar/L\rounge,...,Yes,No,Yes,NaN,Unknown,No,NaN,No,17-2021,NaN
9,250,FHA,NaN,NaN,LANGLEY,4/26/2021,5/9/2021,NaN,NaN,Farm/Agriculture,...,No,No,Unknown,NaN,No,Yes,NaN,No,17-2021,NaN


In [152]:
part = 1
filename = f'F22-1445 Records_Part{part}.pdf'
pages = '34-137'
dfs1_part2_i10, reference1_part2_i10 = parse_pdf(filename, pages=pages, filename=None)

Extracting the following files:
F22-1445 Records_Part1.pdf
number of columns: 23
	page 1...
	Removing s.22(1) from column 2
	Shifting data from column 2(workplace_name)
	Shifting data from column 3(workplace_address)
	page 2...
		Keeping page 1 columns
	Removing s.22(1) from column 2
	Shifting data from column 2(workplace_name)
	Shifting data from column 3(workplace_address)
	page 3...
	Removing s.22(1) from column 3
	Shifting data from column 3(workplace_address)
	page 4...
	Removing s.22(1) from column 2
	Shifting data from column 2(workplace_name)
	Shifting data from column 3(workplace_address)
	page 5...
	Removing s.22(1) from column 2
	Shifting data from column 2(workplace_name)
	Shifting data from column 3(workplace_address)
	page 6...
	Removing s.22(1) from column 2
	Shifting data from column 2(workplace_name)
	Shifting data from column 3(workplace_address)
	page 7...
	Removing s.22(1) from column 2
	Shifting data from column 2(workplace_name)
	Shifting data from column 3(workpl

In [157]:
print(dfs1_part2_i10.columns)
dfs1_part2_i10

Index(['index', 'HA', 'workplace_name', 'workplace_address', 'city',
       'date_cluster_confirmed_public_health', 'date_cluster_no_longer_active',
       'earliest_reported_cases', 'latest_reported_cases',
       'workplace_classification', 'cases_when_cluster_confirmed',
       'total_cases', 'workplace_status', 'closure_y_n', 'closed_by',
       'voluntary_closure', 'closure_date', 'partially_closed', 'open',
       'days_since_last_case', 'active_clusters', 'cluster_confirmed_epidate',
       'doc_page_number', 'cluster_closed_epidate', 'file'],
      dtype='object')


,index,HA,workplace_name,workplace_address,city,date_cluster_confirmed_public_health,date_cluster_no_longer_active,earliest_reported_cases,latest_reported_cases,workplace_classification,...,voluntary_closure,closure_date,partially_closed,open,days_since_last_case,active_clusters,cluster_confirmed_epidate,doc_page_number,cluster_closed_epidate,file
0,1,FHA,None,None,DELTA,4/1/2021,4/6/2021,NaN,NaN,Transportatio\rn,...,Unknown,NaN,No,No,NaN,No,13-2021,1,NaN,F22-1445 Records_Part1.pdf
1,2,FHA,None,None,MAPLE RIDGE,4/1/2021,4/18/2021,NaN,NaN,Restaurant/B\rar/Lounge,...,Unknown,4/6/2021,No,No,NaN,No,13-2021,1,NaN,F22-1445 Records_Part1.pdf
2,3,FHA,None,None,SURREY,4/1/2021,4/22/2021,NaN,NaN,Restaurant/B\rar/Lounge,...,Unknown,4/12/2021,No,No,NaN,No,13-2021,1,NaN,F22-1445 Records_Part1.pdf
3,4,FHA,None,None,SURREY,4/1/2021,4/14/2021,NaN,NaN,Office,...,Unknown,4/3/2021,No,No,NaN,No,13-2021,1,NaN,F22-1445 Records_Part1.pdf
4,5,FHA,None,None,LANGLEY,4/1/2021,NaN,NaN,NaN,Warehouse,...,Unknown,NaN,No,No,NaN,Yes,13-2021,1,NaN,F22-1445 Records_Part1.pdf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1703,174,FHA,None,None,SURREY,4/14/2021,4/27/2021,NaN,NaN,Retail,...,Unknown,4/16/2021,No,No,NaN,No,15-2021,94,15-2021,F22-1445 Records_Part1.pdf
1704,175,FHA,None,None,SURREY,4/14/2021,4/24/2021,NaN,NaN,Retail,...,Unknown,4/14/2021,No,No,NaN,No,15-2021,94,15-2021,F22-1445 Records_Part1.pdf
1705,176,FHA,None,None,SURREY,4/14/2021,4/24/2021,NaN,NaN,Restaurant/Bar\r/Lounge,...,Unknown,4/15/2021,No,No,NaN,No,15-2021,94,15-2021,F22-1445 Records_Part1.pdf
1706,177,FHA,None,None,BURNABY,4/14/2021,4/27/2021,NaN,NaN,Transportation,...,Unknown,NaN,No,No,NaN,No,15-2021,94,NaN,F22-1445 Records_Part1.pdf


In [155]:
set(dfs1_part1_i9.columns) - set(dfs1_part2_i10)

{'closed_by_order'}

In [156]:
 set(dfs1_part2_i10) - set(dfs1_part1_i9.columns)

{'active_clusters', 'closed_by'}

In [154]:
pd.concat([dfs1_part1_i9, dfs1_part2_i10])

,index,HA,workplace_name,workplace_address,city,date_cluster_confirmed_public_health,date_cluster_no_longer_active,earliest_reported_cases,latest_reported_cases,workplace_classification,...,closure_date,partially_closed,open,days_since_last_case,cluster_confirmed_epidate,cluster_closed_epidate,doc_page_number,file,closed_by,active_clusters
0,1.0,FHA,None,None,SURREY,4/1/2021,4/17/2021,NaN,NaN,Restaurant/Bar/L\rounge,...,NaN,No,No,NaN,13-2021,NaN,1,F22-1445 Records_Part1.pdf,NaN,NaN
1,2.0,FHA,None,None,LANGLEY,4/1/2021,4/13/2021,NaN,NaN,Other,...,NaN,No,No,NaN,13-2021,NaN,1,F22-1445 Records_Part1.pdf,NaN,NaN
2,3.0,FHA,None,None,SURREY,4/1/2021,4/13/2021,NaN,NaN,Restaurant/Bar/L\rounge,...,4/4/2021,No,No,NaN,13-2021,14-2021,1,F22-1445 Records_Part1.pdf,NaN,NaN
3,4.0,FHA,None,None,BURNABY,4/1/2021,4/21/2021,NaN,NaN,Gym/Fitness,...,4/7/2021,No,No,NaN,13-2021,14-2021,1,F22-1445 Records_Part1.pdf,NaN,NaN
4,5.0,FHA,None,None,BURNABY,4/1/2021,4/14/2021,NaN,NaN,Restaurant/Bar/L\rounge,...,4/5/2021,No,No,NaN,13-2021,14-2021,1,F22-1445 Records_Part1.pdf,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1703,174,FHA,None,None,SURREY,4/14/2021,4/27/2021,NaN,NaN,Retail,...,4/16/2021,No,No,NaN,15-2021,15-2021,94,F22-1445 Records_Part1.pdf,Yes,No
1704,175,FHA,None,None,SURREY,4/14/2021,4/24/2021,NaN,NaN,Retail,...,4/14/2021,No,No,NaN,15-2021,15-2021,94,F22-1445 Records_Part1.pdf,Yes,No
1705,176,FHA,None,None,SURREY,4/14/2021,4/24/2021,NaN,NaN,Restaurant/Bar\r/Lounge,...,4/15/2021,No,No,NaN,15-2021,15-2021,94,F22-1445 Records_Part1.pdf,Yes,No
1706,177,FHA,None,None,BURNABY,4/14/2021,4/27/2021,NaN,NaN,Transportation,...,NaN,No,No,NaN,15-2021,NaN,94,F22-1445 Records_Part1.pdf,No,No


## Iteration 11
Fix column parsing with regex

In [168]:
from tabula import read_pdf
import json
import re
import warnings
warnings.simplefilter("ignore", category=FutureWarning)

def merge_rows(df, file, id_column=['Unnamed: 0'], fill_method='bfill', groupby_column=['HA']):
    """
    Merges rows in a pandas dataframe that were parsed incorrectly into multiple rows.
    Assumes that the rows that need to be merged have the same value in a unique ID column.
    """
    print(f'\tRemoving duplicate rows from {file}...')
    print('\t\tOriginal df shape: ', df.shape)
    print(f'\t\tOriginal columns:', df.columns)
    df[id_column] = df[id_column].fillna(method=fill_method)
    
    # Group the rows by the unique ID column and aggregate the values in each group

    columns_to_fill = df.drop(columns=groupby_column).columns
    grouped_df = df.groupby(id_column+groupby_column, as_index=False).agg(
        lambda x: ''.join(str(v) for v in x if (type(v)==str) & (x.name in columns_to_fill)))
    grouped_df = grouped_df[grouped_df[groupby_column[0]] != groupby_column[0]]
    # Return the merged dataframe
    print('\t\tTransformed df shape:', grouped_df.shape)
    return grouped_df.replace('', None)



def parse_pdf_tolist(pdf_filenames, filepath=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\BC-Covid-workplace-closures\data\raw', 
    pages='all', last_id_column='HA', value_to_ignore='s.22(1)',
    blank_columns=['workplace_name', 'workplace_address'],
    filename=None,
    path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\BC-Covid-workplace-closures\data\interim'):
    """
    Create dataframe from a PDF file using 
    """
    print(f'Extracting the following files:')
    if type(pdf_filenames) != list:
        pdf_filenames = [pdf_filenames]
    dfs_list = []
    reference_dict = dict()
    corrected_pages = dict()
    for index, file in enumerate(pdf_filenames):
        print(file)
        filepath = f'{filepath}/'.replace('\\','/') if filepath else ''
        path_and_file = filepath+file
        try:
            df_list = read_pdf(path_and_file, pages=pages)
        except:
            print(f'\nError parsing {file}')
        file_dfs = []
        page1_columns = df_list[0].columns.str.replace('\r', '').str.replace(r'(workplace)([a-zA-Z]+)', r'\1_\2', regex=True).to_list()
        print('number of columns:', len(page1_columns))
        if last_id_column in page1_columns:
            page1_columns_sorted = sorted(page1_columns)
        else:
            previous_columns = dfs_list[index-1].columns
            print(f'\t\tNo headers found; using headers from previous document:', previous_columns)
            if len(df_list[page_number].columns) > len(previous_columns):
                n_cols_to_pad = len(df_list[page_number].columns) - len(previous_columns)
                new_columns_padding = [f'blank{index}' for index, col in enumerate(range(n_cols_to_pad))]
                page1_columns_sorted = sorted(page1_columns) + new_columns_padding
                page1_columns = page1_columns + new_columns_padding

            elif len(df_list[page_number].columns) < len(previous_columns):
                page1_columns = (previous_columns[:len(df_list[page_number].columns)].to_list())
                page1_columns_sorted = sorted(page1_columns)
            else:
                page1_columns_sorted = (dfs_list[index-1].columns.to_list())
            page1_columns = ['Unnamed: 0' if column=='index' else column for column in page1_columns]
            print(f'\t\t**Current df columns:',page1_columns)
        
        for page_number in range(len(df_list)):
            print(f'\tpage {page_number+1}...')
            df_list[page_number].columns = df_list[page_number].columns.str.replace('\r', '').str.replace(r'(workplace)([a-zA-Z]+)', r'\1_\2', regex=True)
            print('\t\tCurrent number of columns:', len(df_list[page_number].columns), len(page1_columns_sorted))
            print(f'\t\t{len(page1_columns_sorted)}, {len(page1_columns)}')
            if len(df_list[page_number].columns.to_list()) == len(page1_columns_sorted):
                if sorted(df_list[page_number].columns.to_list()) == page1_columns_sorted:
                    pass
                else:
                    df_list[page_number].columns = page1_columns 
                    print('\t\tKeeping page 1 columns')
            else:
                if len(df_list[page_number].columns) > len(page1_columns_sorted):
                    n_cols_to_pad = len(df_list[page_number].columns) - len(page1_columns_sorted)
                    new_columns_padding = [f'blank{index}' for index, col in enumerate(range(n_cols_to_pad))]
                    df_list[page_number].columns = page1_columns + new_columns_padding
                    print('\t\tPadding columns')
                elif len(df_list[page_number].columns) < len(page1_columns_sorted):
                    df_list[page_number].columns = page1_columns[:len(df_list[page_number].columns)]
                    print('\t\tDropping columns')
                else:
                    df_list[page_number].columns = page1_columns
                corrected_pages[file] = page_number+1
                print(f'\n\t**Fixing column parsing for page {page_number+1} of {file}**')
                print(f'\t\t{df_list[page_number].columns}')
            # Consolidate rows that were incorrectly split
            if (df_list[page_number]['Unnamed: 0'].isna().sum() > 0):
                # print('\tAttempting to consolidate rows that were incorrectly split: ', df_list[page_number].columns)
                print('\tAttempting to consolidate rows that were incorrectly split: ')
                try:
                    df_list[page_number] = merge_rows(df_list[page_number], file)
                except:
                    print(f'\t\tUnable to consolidate rows on page {page_number+1} of {file}')
                    reference_dict['unparsed documents'] = file
                    continue

            # Align the columns
            if value_to_ignore in df_list[page_number].iloc[0].values:
                value_to_ignore_column = df_list[page_number].iloc[0].values.tolist().index(value_to_ignore)
                print(f'\tRemoving {value_to_ignore} from column {value_to_ignore_column}')
                df_list[page_number].iloc[0, value_to_ignore_column:-1] = df_list[page_number].iloc[0, value_to_ignore_column+1:].values
                df_list[page_number].drop(columns=df_list[page_number].columns[-1], inplace=True)

            for column in blank_columns:
                try:
                    if df_list[page_number][column].isna().sum() != len(df_list[page_number]): # 2023-03-30 22:45:
                        blank_column_index = df_list[page_number].columns.tolist().index(column)
                        print(f'\tShifting data from column {blank_column_index}({column})')
                        df_list[page_number].iloc[:,blank_column_index+1:] = df_list[page_number].iloc[:,blank_column_index:-1].values 
                        df_list[page_number].iloc[:,blank_column_index] = None
                except:
                    pass
            column_after_blank_columns_index = blank_column_index + 1
            column_after_blank_columns = df_list[page_number].columns[column_after_blank_columns_index]
            try:
                while df_list[page_number][column_after_blank_columns].isna().sum() == len(df_list[page_number]):
                    print(f'\tShifting data to the right until `{column_after_blank_columns}` column')
                    df_list[page_number].iloc[:,column_after_blank_columns_index:-1] = df_list[
                        page_number].iloc[:,column_after_blank_columns_index+1:].values 
                    df_list[page_number].drop(columns=df_list[page_number].columns[-1], inplace=True)
            except:
                pass
            try:
                for column in df_list[page_number].filter(regex='blank').columns:
                    if df_list[page_number][column].notna().sum() == 0:
                        df_list[page_number].drop(columns=column, inplace=True)
                        print(f'\tDropping blank column `{column}`')
            except:
                pass

            file_dfs.append(df_list[page_number]) # 2023-03-30 23:04 Decrease indent
        dfs_list.append(file_dfs) # SH 2023-04-01 20:58
        

    return dfs_list, reference_dict




def parse_pdf(pdf_filenames,pages='all', last_id_column='HA', value_to_ignore='s.22(1)', 
    value_to_ignore_start_index=1, value_to_ignore_end_index=None,
    blank_columns=['workplace_name', 'workplace_address'],
    filename=None,filepath=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\BC-Covid-workplace-closures\data\raw', 
    path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\BC-Covid-workplace-closures\data\interim'):
    """
    Create dataframe from a PDF file using 
    """
    print(f'Extracting the following files:')
    if type(pdf_filenames) != list:
        pdf_filenames = [pdf_filenames]
    dfs_list = []
    reference_dict = dict()
    corrected_pages = dict()
    try:
        first_doc_page = int(pages.split("-")[0])
    except:
        first_doc_page = 1
    print(f'Starting from page {first_doc_page} of PDF')
    for index, file in enumerate(pdf_filenames):
        print(file)
        filepath = f'{filepath}/'.replace('\\','/') if filepath else ''
        path_and_file = filepath+file
        try:
            df_list = read_pdf(path_and_file, pages=pages)
        except:
            print(f'\nError parsing {file}')
        file_dfs = []
        page1_columns = df_list[0].columns.str.replace('\r', '').str.replace(r'(workplace)([a-zA-Z]+)', r'\1_\2', regex=True).to_list()
        print('number of columns:', len(page1_columns))
        if last_id_column in page1_columns:
            page1_columns_sorted = sorted(page1_columns)
        else:
            previous_columns = dfs_list[index-1].columns
            print(f'\t\tNo headers found; using headers from previous document:', previous_columns)
            if len(df_list[page_number].columns) > len(previous_columns):
                n_cols_to_pad = len(df_list[page_number].columns) - len(previous_columns)
                new_columns_padding = [f'blank{index}' for index, col in enumerate(range(n_cols_to_pad))]
                page1_columns_sorted = sorted(page1_columns) + new_columns_padding
                page1_columns = page1_columns + new_columns_padding

            elif len(df_list[page_number].columns) < len(previous_columns):
                page1_columns = (previous_columns[:len(df_list[page_number].columns)].to_list())
                page1_columns_sorted = sorted(page1_columns)
            else:
                page1_columns_sorted = (dfs_list[index-1].columns.to_list())
            page1_columns = ['Unnamed: 0' if column=='index' else column for column in page1_columns]
            print(f'\t\t**Current df columns:',page1_columns)
        
        for page_number in range(len(df_list)):
            print(f'\tpage {page_number+1}...')
            df_list[page_number].columns = df_list[page_number].columns.str.replace('\r', '').str.replace(r'(workplace)([a-zA-Z]+)', r'\1_\2', regex=True)
            if len(df_list[page_number].columns.to_list()) == len(page1_columns_sorted):
                if sorted(df_list[page_number].columns.to_list()) == page1_columns_sorted:
                    pass
                else:
                    df_list[page_number].columns = page1_columns 
                    print('\t\tKeeping page 1 columns')
            else:
                if len(df_list[page_number].columns) > len(page1_columns_sorted):
                    n_cols_to_pad = len(df_list[page_number].columns) - len(page1_columns_sorted)
                    new_columns_padding = [f'blank{index}' for index, col in enumerate(range(n_cols_to_pad))]
                    df_list[page_number].columns = page1_columns + new_columns_padding
                    print('\t\tPadding columns')
                elif len(df_list[page_number].columns) < len(page1_columns_sorted):
                    df_list[page_number].columns = page1_columns[:len(df_list[page_number].columns)]
                    print('\t\tDropping columns')
                else:
                    df_list[page_number].columns = page1_columns
                corrected_pages[file] = page_number+1
                print(f'\n\t**Fixing column parsing for page {page_number+1} of {file}**')
                print(f'\t\t{df_list[page_number].columns}')
            # Consolidate rows that were incorrectly split
            if (df_list[page_number]['Unnamed: 0'].isna().sum() > 0):
                # print('\tAttempting to consolidate rows that were incorrectly split: ', df_list[page_number].columns)
                print('\tAttempting to consolidate rows that were incorrectly split: ')
                try:
                    df_list[page_number] = merge_rows(df_list[page_number], file)
                except:
                    print(f'\t\tUnable to consolidate rows on page {page_number+1} of {file}')
                    reference_dict['unparsed documents'] = file
                    continue

            # Align the columns
            value_to_ignore_regex = rf'.*{re.escape(value_to_ignore[value_to_ignore_start_index:value_to_ignore_end_index])}.*'
            if df_list[page_number].iloc[0].str.contains(value_to_ignore_regex).any():
                row_values = df_list[page_number].fillna(0).iloc[0].values.tolist()
                for index, item in enumerate(row_values):
                    if re.match(value_to_ignore_regex, str(item)):
                        value_to_ignore_column = index
                        break

                print(f'\tRemoving {value_to_ignore} from column {value_to_ignore_column}')
                df_list[page_number].iloc[0, value_to_ignore_column:-1] = df_list[page_number].iloc[0, value_to_ignore_column+1:].values
                df_list[page_number].drop(columns=df_list[page_number].columns[-1], inplace=True)

            for column in blank_columns:
                try:
                    if df_list[page_number][column].isna().sum() != len(df_list[page_number]): # 2023-03-30 22:45:
                        blank_column_index = df_list[page_number].columns.tolist().index(column)
                        print(f'\tShifting data from column {blank_column_index}({column})')
                        df_list[page_number].iloc[:,blank_column_index+1:] = df_list[page_number].iloc[:,blank_column_index:-1].values 
                        df_list[page_number].iloc[:,blank_column_index] = None
                except:
                    pass
            column_after_blank_columns_index = blank_column_index + 1
            column_after_blank_columns = df_list[page_number].columns[column_after_blank_columns_index]
            try:
                while df_list[page_number][column_after_blank_columns].isna().sum() == len(df_list[page_number]):
                    print(f'\tShifting data to the right until `{column_after_blank_columns}` column')
                    df_list[page_number].iloc[:,column_after_blank_columns_index:-1] = df_list[
                        page_number].iloc[:,column_after_blank_columns_index+1:].values 
                    df_list[page_number].drop(columns=df_list[page_number].columns[-1], inplace=True)
            except:
                pass
            try:
                for column in df_list[page_number].filter(regex='blank').columns:
                    if df_list[page_number][column].notna().sum() == 0:
                        df_list[page_number].drop(columns=column, inplace=True)
                        print(f'\tDropping blank column `{column}`')
            except:
                pass
            df_list[page_number]['doc_page_number'] = page_number + first_doc_page  
            file_dfs.append(df_list[page_number])  # 2023-03-30 23:04 Decrease indent
        
        try:
            final_file_df = pd.concat(file_dfs)
            final_file_df['file'] = file
            final_file_df.columns = final_file_df.columns.str.replace('cosed', 'closed')
            final_file_df.columns = final_file_df.columns.str.replace('closed_by', 'closed_by_order')
            final_file_df.reset_index(inplace=True, drop=True)
            final_file_df.rename(columns={'Unnamed: 0': 'index'}, inplace=True)
            dfs_list.append(final_file_df)

            print('\n\tDone')
        except:
            print(f'Error concatenating dataframes from {file}')
            return file_dfs, {file: 'DataFrame concatenation error'}
    unique_cols_dict = {}
    for i, df in enumerate(dfs_list):
        other_dfs = dfs_list[:i] + dfs_list[i+1:]
        other_cols = set().union(*[set(other_df.columns) for other_df in other_dfs])
        unique_columns = set(df.columns) - other_cols
        unique_cols_dict[file] = unique_columns if len(unique_columns)>0 else "none"
        print(f'Columns unique to part{pdf_filenames[i]}: {unique_cols_dict[file]}')
    reference_dict['corrected pages'] = corrected_pages
    reference_dict['unique columns'] = unique_cols_dict
    if filename:
        try:
            savepickle(dfs_list, filename=filename, path=path)
            with open(f'{filepath}{filename}_reference.json','w') as json_file:
                json.dump(reference_dict, json_file)
            save_csv(pd.concat(dfs_list), filename=filename, path=path)
        except:
            print('Unable to save outputs')
    try:
        return pd.concat(dfs_list), reference_dict
    except:
        print('Unable to concatenate dataframes from different documents')
        return dfs_list, reference_dict

# n_docs = 7
# start = 1
# filenames = [f'F22-1445 Records_Part{i}.pdf' for i in range(start,start+n_docs)]
# part = 1
# filename = f'F22-1445 Records_Part{part}.pdf'

# dfs_list1_i10, reference_list1_i10 = parse_pdf_tolist(filename, pages='34,76', filename=None)
# # dfs, reference = parse_pdf(filenames, pages='all', filename='pdfs_parsed_2023-03-30')


# Parse part 1

In [169]:
part = 1
filename = f'F22-1445 Records_Part{part}.pdf'
pages = '1-30'
dfs1_part1_i11, reference1_part1_i11 = parse_pdf(filename, pages=pages, filename=None)

Extracting the following files:
Starting from page 1 of PDF
F22-1445 Records_Part1.pdf
number of columns: 22
	page 1...
	Shifting data from column 2(workplace_name)
	Shifting data from column 3(workplace_address)
	page 2...
	Removing s.22(1) from column 2
	Shifting data from column 2(workplace_name)
	Shifting data from column 3(workplace_address)
	page 3...
	Shifting data from column 2(workplace_name)
	Shifting data from column 3(workplace_address)
	page 4...
	Removing s.22(1) from column 2
	Shifting data from column 2(workplace_name)
	Shifting data from column 3(workplace_address)
	page 5...
	Shifting data from column 2(workplace_name)
	Shifting data from column 3(workplace_address)
	page 6...
	Shifting data from column 2(workplace_name)
	Shifting data from column 3(workplace_address)
	page 7...
	Removing s.22(1) from column 2
	Shifting data from column 2(workplace_name)
	Shifting data from column 3(workplace_address)
	page 8...
	Removing s.22(1) from column 2
	Shifting data from colu

In [165]:
dfs1_part1_i11

,index,HA,workplace_name,workplace_address,city,date_cluster_confirmed_public_health,date_cluster_no_longer_active,earliest_reported_cases,latest_reported_cases,workplace_classification,...,closed_by_order_order,voluntary_closure,closure_date,partially_closed,open,days_since_last_case,cluster_confirmed_epidate,cluster_closed_epidate,doc_page_number,file
0,1.0,FHA,None,None,SURREY,4/1/2021,4/17/2021,NaN,NaN,Restaurant/Bar/L\rounge,...,No,Unknown,NaN,No,No,NaN,13-2021,NaN,1,F22-1445 Records_Part1.pdf
1,2.0,FHA,None,None,LANGLEY,4/1/2021,4/13/2021,NaN,NaN,Other,...,No,Unknown,NaN,No,No,NaN,13-2021,NaN,1,F22-1445 Records_Part1.pdf
2,3.0,FHA,None,None,SURREY,4/1/2021,4/13/2021,NaN,NaN,Restaurant/Bar/L\rounge,...,Yes,Unknown,4/4/2021,No,No,NaN,13-2021,14-2021,1,F22-1445 Records_Part1.pdf
3,4.0,FHA,None,None,BURNABY,4/1/2021,4/21/2021,NaN,NaN,Gym/Fitness,...,Yes,Unknown,4/7/2021,No,No,NaN,13-2021,14-2021,1,F22-1445 Records_Part1.pdf
4,5.0,FHA,None,None,BURNABY,4/1/2021,4/14/2021,NaN,NaN,Restaurant/Bar/L\rounge,...,Yes,Unknown,4/5/2021,No,No,NaN,13-2021,14-2021,1,F22-1445 Records_Part1.pdf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
459,460.0,IHA,None,None,Kelowna,5/27/2021,NaN,5/27/2021,6/5/2021,Restaurant/Bar/L\rounge,...,No,Yes,5/26/2021,Unknown,No,NaN,21-2021,21-2021,30,F22-1445 Records_Part1.pdf
460,461.0,IHA,None,None,Kelowna,5/28/2021,NaN,5/28/2021,5/3/2021,Personal Care,...,No,No,NaN,Unknown,No,NaN,21-2021,NaN,30,F22-1445 Records_Part1.pdf
461,462.0,IHA,None,None,Kelowna,5/31/2021,NaN,5/31/2021,5/31/2021,Maintenance/Tr\rades/Constructio\rn,...,No,No,NaN,Unknown,No,NaN,22-2021,NaN,30,F22-1445 Records_Part1.pdf
462,463.0,IHA,None,None,Vernon,5/31/2021,NaN,5/31/2021,6/3/2021,Retail,...,No,No,NaN,Unknown,No,NaN,22-2021,NaN,30,F22-1445 Records_Part1.pdf


In [172]:
part = 1
filename = f'F22-1445 Records_Part{part}.pdf'
pages = '34-137'
dfs1_part2_i11, reference1_part2_i11 = parse_pdf(filename, pages=pages, filename=None)

Extracting the following files:
Starting from page 34 of PDF
F22-1445 Records_Part1.pdf
number of columns: 23
	page 1...
	Removing s.22(1) from column 2
	Shifting data from column 2(workplace_name)
	Shifting data from column 3(workplace_address)
	page 2...
		Keeping page 1 columns
	Removing s.22(1) from column 2
	Shifting data from column 2(workplace_name)
	Shifting data from column 3(workplace_address)
	page 3...
	Removing s.22(1) from column 3
	Shifting data from column 3(workplace_address)
	page 4...
	Removing s.22(1) from column 2
	Shifting data from column 2(workplace_name)
	Shifting data from column 3(workplace_address)
	page 5...
	Removing s.22(1) from column 2
	Shifting data from column 2(workplace_name)
	Shifting data from column 3(workplace_address)
	page 6...
	Removing s.22(1) from column 2
	Shifting data from column 2(workplace_name)
	Shifting data from column 3(workplace_address)
	page 7...
	Removing s.22(1) from column 2
	Shifting data from column 2(workplace_name)
	Shift

In [301]:
dfs1_part2_i11[dfs1_part2_i11['index'] == 197]

,index,HA,workplace_name,workplace_address,city,date_cluster_confirmed_public_health,date_cluster_no_longer_active,earliest_reported_cases,latest_reported_cases,workplace_classification,...,voluntary_closure,closure_date,partially_closed,open,days_since_last_case,active_clusters,cluster_confirmed_epidate,doc_page_number,cluster_closed_epidate,file
196,197,FHA,None,None,DELTA,4/20/2021,5/4/2021,NaN,NaN,Transportatio\rn,...,Unknown,4/21/2021,No,No,NaN,No,16-2021,44,NaN,F22-1445 Records_Part1.pdf
1179,197,VCH,None,None,Vancouver,4/16/2021,NaN,4/16/2021,5/9/2021,Retail,...,No,4/21/2021,No,No,NaN,NaN,15-2021,95,NaN,F22-1445 Records_Part1.pdf


In [176]:
n_docs = 6
start = 2
filenames = [f'F22-1445 Records_Part{i}.pdf' for i in range(start,start+n_docs)]

dfs_remainder_i11, reference_remainder_i11 = parse_pdf(filenames, pages='all', filename=None)

Extracting the following files:
Starting from page 1 of PDF
F22-1445 Records_Part2.pdf
number of columns: 23
	page 1...
	Removing s.22(1) from column 3
	Shifting data from column 3(workplace_address)
	page 2...
	Removing s.22(1) from column 3
	Shifting data from column 3(workplace_address)
	page 3...
		Padding columns

	**Fixing column parsing for page 3 of F22-1445 Records_Part2.pdf**
		Index(['Unnamed: 0', 'HA', 'workplace_name', 'workplace_address', 'city',
       'date_cluster_confirmed_public_health', 'date_cluster_no_longer_active',
       'earliest_reported_cases', 'latest_reported_cases',
       'workplace_classification', 'cases_when_cluster_confirmed',
       'total_cases', 'workplace_status', 'closure_y_n', 'closed_by_order',
       'voluntary_closure', 'closure_date', 'partially_closed', 'open',
       'days_since_last_case', 'active_clusters', 'cluster_confirmed_epidate',
       'cluster_closed_epidate', 'blank0', 'blank1'],
      dtype='object')
	Attempting to consolidate

In [180]:
print(dfs_remainder_i11.columns)
dfs_remainder_i11

Index(['index', 'HA', 'workplace_name', 'workplace_address', 'city',
       'date_cluster_confirmed_public_health', 'date_cluster_no_longer_active',
       'earliest_reported_cases', 'latest_reported_cases',
       'workplace_classification', 'cases_when_cluster_confirmed',
       'total_cases', 'workplace_status', 'closure_y_n',
       'closed_by_order_order', 'voluntary_closure', 'closure_date',
       'partially_closed', 'open', 'days_since_last_case', 'active_clusters',
       'cluster_confirmed_epidate', 'doc_page_number',
       'cluster_closed_epidate', 'file', 'blank0', 'unknown_closure_type',
       'closed_by_order_order_order'],
      dtype='object')


,index,HA,workplace_name,workplace_address,city,date_cluster_confirmed_public_health,date_cluster_no_longer_active,earliest_reported_cases,latest_reported_cases,workplace_classification,...,open,days_since_last_case,active_clusters,cluster_confirmed_epidate,doc_page_number,cluster_closed_epidate,file,blank0,unknown_closure_type,closed_by_order_order_order
0,179.0,FHA,NaN,None,ABBOTSFORD,4/14/2021,5/3/2021,NaN,NaN,Farm/Agricultu\rre,...,No,NaN,No,15-2021,1,NaN,F22-1445 Records_Part2.pdf,NaN,NaN,NaN
1,180.0,FHA,NaN,None,DELTA,4/14/2021,5/9/2021,NaN,NaN,Retail,...,No,NaN,No,15-2021,1,NaN,F22-1445 Records_Part2.pdf,NaN,NaN,NaN
2,181.0,FHA,NaN,None,PORT\rCOQUITLAM,4/14/2021,5/6/2021,NaN,NaN,Maintenance/T\rrades/Construc\rtion,...,No,NaN,No,15-2021,1,NaN,F22-1445 Records_Part2.pdf,NaN,NaN,NaN
3,182.0,VCH,NaN,None,Vancouver,4/14/2021,NaN,4/14/2021,5/5/2021,Manufacturing\r/Industry,...,No,NaN,No,15-2021,1,NaN,F22-1445 Records_Part2.pdf,NaN,NaN,NaN
4,183.0,NHA,NaN,None,Unknown,4/14/2021,4/29/2021,NaN,NaN,Maintenance/T\rrades/Construc\rtion,...,No,NaN,No,15-2021,1,NaN,F22-1445 Records_Part2.pdf,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1397,1090.0,IHA,None,None,Kamloops,1/26/2022,NaN,1/26/2022,NaN,School/Daycare,...,No,NaN,No,4-2022,81,4-2022,F22-1445 Records_Part7.pdf,NaN,No,NaN
1398,1091.0,IHA,None,None,Kamloops,1/27/2022,NaN,1/27/2022,NaN,School/Daycare,...,No,NaN,No,4-2022,81,4-2022,F22-1445 Records_Part7.pdf,NaN,No,NaN
1399,1092.0,IHA,None,None,Revelstoke,1/28/2022,NaN,1/28/2022,NaN,School/Daycare,...,No,NaN,No,4-2022,81,2-2022,F22-1445 Records_Part7.pdf,NaN,No,NaN
1400,1093.0,IHA,None,None,260 Cedar Ave,None,1/31/2022,None,1/31/2022,None,...,No,No,None,No,82,5-2022,F22-1445 Records_Part7.pdf,NaN,No,NaN


In [194]:
dfs_remainder_i11.rename(columns={'city': 'city2'}).columns

Index(['index', 'HA', 'workplace_name', 'workplace_address', 'city2',
       'date_cluster_confirmed_public_health', 'date_cluster_no_longer_active',
       'earliest_reported_cases', 'latest_reported_cases',
       'workplace_classification', 'cases_when_cluster_confirmed',
       'total_cases', 'workplace_status', 'closure_y_n',
       'closed_by_order_order', 'voluntary_closure', 'closure_date',
       'partially_closed', 'open', 'days_since_last_case', 'active_clusters',
       'cluster_confirmed_epidate', 'doc_page_number',
       'cluster_closed_epidate', 'file', 'blank0', 'unknown_closure_type',
       'closed_by_order_order_order'],
      dtype='object')

In [211]:
pd.concat([
    dfs1_part1_i11,
    dfs1_part2_i11,
    dfs_remainder_i11
]).columns

Index(['index', 'HA', 'workplace_name', 'workplace_address', 'city',
       'date_cluster_confirmed_public_health', 'date_cluster_no_longer_active',
       'earliest_reported_cases', 'latest_reported_cases',
       'workplace_classification', 'cases_when_cluster_confirmed',
       'total_cases', 'workplace_status', 'closure_y_n',
       'closed_by_order_order', 'voluntary_closure', 'closure_date',
       'partially_closed', 'open', 'days_since_last_case',
       'cluster_confirmed_epidate', 'cluster_closed_epidate',
       'doc_page_number', 'file', 'closed_by_order', 'active_clusters',
       'blank0', 'unknown_closure_type', 'closed_by_order_order_order'],
      dtype='object')

## Iteration 12

In [212]:
from tabula import read_pdf
import json
import re
import warnings
warnings.simplefilter("ignore", category=FutureWarning)

def merge_rows(df, file, id_column=['Unnamed: 0'], fill_method='bfill', groupby_column=['HA']):
    """
    Merges rows in a pandas dataframe that were parsed incorrectly into multiple rows.
    Assumes that the rows that need to be merged have the same value in a unique ID column.
    """
    print(f'\tRemoving duplicate rows from {file}...')
    print('\t\tOriginal df shape: ', df.shape)
    print(f'\t\tOriginal columns:', df.columns)
    df[id_column] = df[id_column].fillna(method=fill_method)
    
    # Group the rows by the unique ID column and aggregate the values in each group

    columns_to_fill = df.drop(columns=groupby_column).columns
    grouped_df = df.groupby(id_column+groupby_column, as_index=False).agg(
        lambda x: ''.join(str(v) for v in x if (type(v)==str) & (x.name in columns_to_fill)))
    grouped_df = grouped_df[grouped_df[groupby_column[0]] != groupby_column[0]]
    # Return the merged dataframe
    print('\t\tTransformed df shape:', grouped_df.shape)
    return grouped_df.replace('', None)



def parse_pdf_tolist(pdf_filenames, filepath=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\BC-Covid-workplace-closures\data\raw', 
    pages='all', last_id_column='HA', value_to_ignore='s.22(1)',
    blank_columns=['workplace_name', 'workplace_address'],
    filename=None,
    path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\BC-Covid-workplace-closures\data\interim'):
    """
    Create dataframe from a PDF file using 
    """
    print(f'Extracting the following files:')
    if type(pdf_filenames) != list:
        pdf_filenames = [pdf_filenames]
    dfs_list = []
    reference_dict = dict()
    corrected_pages = dict()
    for index, file in enumerate(pdf_filenames):
        print(file)
        filepath = f'{filepath}/'.replace('\\','/') if filepath else ''
        path_and_file = filepath+file
        try:
            df_list = read_pdf(path_and_file, pages=pages)
        except:
            print(f'\nError parsing {file}')
        file_dfs = []
        page1_columns = df_list[0].columns.str.replace('\r', '').str.replace(r'(workplace)([a-zA-Z]+)', r'\1_\2', regex=True).to_list()
        print('number of columns:', len(page1_columns))
        if last_id_column in page1_columns:
            page1_columns_sorted = sorted(page1_columns)
        else:
            previous_columns = dfs_list[index-1].columns
            print(f'\t\tNo headers found; using headers from previous document:', previous_columns)
            if len(df_list[page_number].columns) > len(previous_columns):
                n_cols_to_pad = len(df_list[page_number].columns) - len(previous_columns)
                new_columns_padding = [f'blank{index}' for index, col in enumerate(range(n_cols_to_pad))]
                page1_columns_sorted = sorted(page1_columns) + new_columns_padding
                page1_columns = page1_columns + new_columns_padding

            elif len(df_list[page_number].columns) < len(previous_columns):
                page1_columns = (previous_columns[:len(df_list[page_number].columns)].to_list())
                page1_columns_sorted = sorted(page1_columns)
            else:
                page1_columns_sorted = (dfs_list[index-1].columns.to_list())
            page1_columns = ['Unnamed: 0' if column=='index' else column for column in page1_columns]
            print(f'\t\t**Current df columns:',page1_columns)
        
        for page_number in range(len(df_list)):
            print(f'\tpage {page_number+1}...')
            df_list[page_number].columns = df_list[page_number].columns.str.replace('\r', '').str.replace(r'(workplace)([a-zA-Z]+)', r'\1_\2', regex=True)
            print('\t\tCurrent number of columns:', len(df_list[page_number].columns), len(page1_columns_sorted))
            print(f'\t\t{len(page1_columns_sorted)}, {len(page1_columns)}')
            if len(df_list[page_number].columns.to_list()) == len(page1_columns_sorted):
                if sorted(df_list[page_number].columns.to_list()) == page1_columns_sorted:
                    pass
                else:
                    df_list[page_number].columns = page1_columns 
                    print('\t\tKeeping page 1 columns')
            else:
                if len(df_list[page_number].columns) > len(page1_columns_sorted):
                    n_cols_to_pad = len(df_list[page_number].columns) - len(page1_columns_sorted)
                    new_columns_padding = [f'blank{index}' for index, col in enumerate(range(n_cols_to_pad))]
                    df_list[page_number].columns = page1_columns + new_columns_padding
                    print('\t\tPadding columns')
                elif len(df_list[page_number].columns) < len(page1_columns_sorted):
                    df_list[page_number].columns = page1_columns[:len(df_list[page_number].columns)]
                    print('\t\tDropping columns')
                else:
                    df_list[page_number].columns = page1_columns
                corrected_pages[file] = page_number+1
                print(f'\n\t**Fixing column parsing for page {page_number+1} of {file}**')
                print(f'\t\t{df_list[page_number].columns}')
            # Consolidate rows that were incorrectly split
            if (df_list[page_number]['Unnamed: 0'].isna().sum() > 0):
                # print('\tAttempting to consolidate rows that were incorrectly split: ', df_list[page_number].columns)
                print('\tAttempting to consolidate rows that were incorrectly split: ')
                try:
                    df_list[page_number] = merge_rows(df_list[page_number], file)
                except:
                    print(f'\t\tUnable to consolidate rows on page {page_number+1} of {file}')
                    reference_dict['unparsed documents'] = file
                    continue

            # Align the columns
            if value_to_ignore in df_list[page_number].iloc[0].values:
                value_to_ignore_column = df_list[page_number].iloc[0].values.tolist().index(value_to_ignore)
                print(f'\tRemoving {value_to_ignore} from column {value_to_ignore_column}')
                df_list[page_number].iloc[0, value_to_ignore_column:-1] = df_list[page_number].iloc[0, value_to_ignore_column+1:].values
                df_list[page_number].drop(columns=df_list[page_number].columns[-1], inplace=True)

            for column in blank_columns:
                try:
                    if df_list[page_number][column].isna().sum() != len(df_list[page_number]): # 2023-03-30 22:45:
                        blank_column_index = df_list[page_number].columns.tolist().index(column)
                        print(f'\tShifting data from column {blank_column_index}({column})')
                        df_list[page_number].iloc[:,blank_column_index+1:] = df_list[page_number].iloc[:,blank_column_index:-1].values 
                        df_list[page_number].iloc[:,blank_column_index] = None
                except:
                    pass
            column_after_blank_columns_index = blank_column_index + 1
            column_after_blank_columns = df_list[page_number].columns[column_after_blank_columns_index]
            try:
                while df_list[page_number][column_after_blank_columns].isna().sum() == len(df_list[page_number]):
                    print(f'\tShifting data to the right until `{column_after_blank_columns}` column')
                    df_list[page_number].iloc[:,column_after_blank_columns_index:-1] = df_list[
                        page_number].iloc[:,column_after_blank_columns_index+1:].values 
                    df_list[page_number].drop(columns=df_list[page_number].columns[-1], inplace=True)
            except:
                pass
            try:
                for column in df_list[page_number].filter(regex='blank').columns:
                    if df_list[page_number][column].notna().sum() == 0:
                        df_list[page_number].drop(columns=column, inplace=True)
                        print(f'\tDropping blank column `{column}`')
            except:
                pass

            file_dfs.append(df_list[page_number]) # 2023-03-30 23:04 Decrease indent
        dfs_list.append(file_dfs) # SH 2023-04-01 20:58
        

    return dfs_list, reference_dict




def parse_pdf(pdf_filenames,pages='all', last_id_column='HA', value_to_ignore='s.22(1)', 
    value_to_ignore_start_index=1, value_to_ignore_end_index=None,
    blank_columns=['workplace_name', 'workplace_address'],
    filename=None,filepath=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\BC-Covid-workplace-closures\data\raw', 
    path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\BC-Covid-workplace-closures\data\interim'):
    """
    Create dataframe from a PDF file using 
    """
    print(f'Extracting the following files:')
    if type(pdf_filenames) != list:
        pdf_filenames = [pdf_filenames]
    dfs_list = []
    reference_dict = dict()
    corrected_pages = dict()
    try:
        first_doc_page = int(pages.split("-")[0])
    except:
        first_doc_page = 1
    print(f'Starting from page {first_doc_page} of PDF')
    for index, file in enumerate(pdf_filenames):
        print(file)
        filepath = f'{filepath}/'.replace('\\','/') if filepath else ''
        path_and_file = filepath+file
        try:
            df_list = read_pdf(path_and_file, pages=pages)
        except:
            print(f'\nError parsing {file}')
        file_dfs = []
        page1_columns = df_list[0].columns.str.replace('\r', '').str.replace(r'(workplace)([a-zA-Z]+)', r'\1_\2', regex=True).to_list()
        print('number of columns:', len(page1_columns))
        if last_id_column in page1_columns:
            page1_columns_sorted = sorted(page1_columns)
        else:
            previous_columns = dfs_list[index-1].columns
            print(f'\t\tNo headers found; using headers from previous document:', previous_columns)
            if len(df_list[page_number].columns) > len(previous_columns):
                n_cols_to_pad = len(df_list[page_number].columns) - len(previous_columns)
                new_columns_padding = [f'blank{index}' for index, col in enumerate(range(n_cols_to_pad))]
                page1_columns_sorted = sorted(page1_columns) + new_columns_padding
                page1_columns = page1_columns + new_columns_padding

            elif len(df_list[page_number].columns) < len(previous_columns):
                page1_columns = (previous_columns[:len(df_list[page_number].columns)].to_list())
                page1_columns_sorted = sorted(page1_columns)
            else:
                page1_columns_sorted = (dfs_list[index-1].columns.to_list())
            page1_columns = ['Unnamed: 0' if column=='index' else column for column in page1_columns]
            print(f'\t\t**Current df columns:',page1_columns)
        
        for page_number in range(len(df_list)):
            print(f'\tpage {page_number+1}...')
            df_list[page_number].columns = df_list[page_number].columns.str.replace('\r', '').str.replace(r'(workplace)([a-zA-Z]+)', r'\1_\2', regex=True)
            if len(df_list[page_number].columns.to_list()) == len(page1_columns_sorted):
                if sorted(df_list[page_number].columns.to_list()) == page1_columns_sorted:
                    pass
                else:
                    df_list[page_number].columns = page1_columns 
                    print('\t\tKeeping page 1 columns')
            else:
                if len(df_list[page_number].columns) > len(page1_columns_sorted):
                    n_cols_to_pad = len(df_list[page_number].columns) - len(page1_columns_sorted)
                    new_columns_padding = [f'blank{index}' for index, col in enumerate(range(n_cols_to_pad))]
                    df_list[page_number].columns = page1_columns + new_columns_padding
                    print('\t\tPadding columns')
                elif len(df_list[page_number].columns) < len(page1_columns_sorted):
                    df_list[page_number].columns = page1_columns[:len(df_list[page_number].columns)]
                    print('\t\tDropping columns')
                else:
                    df_list[page_number].columns = page1_columns
                corrected_pages[file] = page_number+1
                print(f'\n\t**Fixing column parsing for page {page_number+1} of {file}**')
                print(f'\t\t{df_list[page_number].columns}')
            # Consolidate rows that were incorrectly split
            if (df_list[page_number]['Unnamed: 0'].isna().sum() > 0):
                # print('\tAttempting to consolidate rows that were incorrectly split: ', df_list[page_number].columns)
                print('\tAttempting to consolidate rows that were incorrectly split: ')
                try:
                    df_list[page_number] = merge_rows(df_list[page_number], file)
                except:
                    print(f'\t\tUnable to consolidate rows on page {page_number+1} of {file}')
                    reference_dict['unparsed documents'] = file
                    continue

            # Align the columns
            value_to_ignore_regex = rf'.*{re.escape(value_to_ignore[value_to_ignore_start_index:value_to_ignore_end_index])}.*'
            if df_list[page_number].iloc[0].str.contains(value_to_ignore_regex).any():
                row_values = df_list[page_number].fillna(0).iloc[0].values.tolist()
                for index, item in enumerate(row_values):
                    if re.match(value_to_ignore_regex, str(item)):
                        value_to_ignore_column = index
                        break

                print(f'\tRemoving {value_to_ignore} from column {value_to_ignore_column}')
                df_list[page_number].iloc[0, value_to_ignore_column:-1] = df_list[page_number].iloc[0, value_to_ignore_column+1:].values
                df_list[page_number].drop(columns=df_list[page_number].columns[-1], inplace=True)

            for column in blank_columns:
                try:
                    if df_list[page_number][column].isna().sum() != len(df_list[page_number]): # 2023-03-30 22:45:
                        blank_column_index = df_list[page_number].columns.tolist().index(column)
                        print(f'\tShifting data from column {blank_column_index}({column})')
                        df_list[page_number].iloc[:,blank_column_index+1:] = df_list[page_number].iloc[:,blank_column_index:-1].values 
                        df_list[page_number].iloc[:,blank_column_index] = None
                except:
                    pass
            column_after_blank_columns_index = blank_column_index + 1
            column_after_blank_columns = df_list[page_number].columns[column_after_blank_columns_index]
            try:
                while df_list[page_number][column_after_blank_columns].isna().sum() == len(df_list[page_number]):
                    print(f'\tShifting data to the right until `{column_after_blank_columns}` column')
                    df_list[page_number].iloc[:,column_after_blank_columns_index:-1] = df_list[
                        page_number].iloc[:,column_after_blank_columns_index+1:].values 
                    df_list[page_number].drop(columns=df_list[page_number].columns[-1], inplace=True)
            except:
                pass
            try:
                for column in df_list[page_number].filter(regex='blank').columns:
                    if df_list[page_number][column].notna().sum() == 0:
                        df_list[page_number].drop(columns=column, inplace=True)
                        print(f'\tDropping blank column `{column}`')
            except:
                pass
            df_list[page_number]['doc_page_number'] = page_number + first_doc_page  
            file_dfs.append(df_list[page_number])  # 2023-03-30 23:04 Decrease indent
        
        try:
            final_file_df = pd.concat(file_dfs)
            final_file_df['file'] = file
            final_file_df.columns = final_file_df.columns.str.replace('cosed', 'closed')
            final_file_df.reset_index(inplace=True, drop=True)
            final_file_df.rename(columns={'closure_y_': 'closure_y_n'}, inplace=True)
            final_file_df.rename(columns={'closed_by': 'closed_by_order'}, inplace=True)
            final_file_df.rename(columns={'Unnamed: 0': 'index'}, inplace=True)
            dfs_list.append(final_file_df)

            print('\n\tDone')
        except:
            print(f'Error concatenating dataframes from {file}')
            return file_dfs, {file: 'DataFrame concatenation error'}
    unique_cols_dict = {}
    for i, df in enumerate(dfs_list):
        other_dfs = dfs_list[:i] + dfs_list[i+1:]
        other_cols = set().union(*[set(other_df.columns) for other_df in other_dfs])
        unique_columns = set(df.columns) - other_cols
        unique_cols_dict[file] = unique_columns if len(unique_columns)>0 else "none"
        print(f'Columns unique to part{pdf_filenames[i]}: {unique_cols_dict[file]}')
    reference_dict['corrected pages'] = corrected_pages
    reference_dict['unique columns'] = unique_cols_dict
    if filename:
        try:
            savepickle(dfs_list, filename=filename, path=path)
            with open(f'{filepath}{filename}_reference.json','w') as json_file:
                json.dump(reference_dict, json_file)
            save_csv(pd.concat(dfs_list), filename=filename, path=path)
        except:
            print('Unable to save outputs')
    try:
        return pd.concat(dfs_list), reference_dict
    except:
        print('Unable to concatenate dataframes from different documents')
        return dfs_list, reference_dict

# n_docs = 7
# start = 1
# filenames = [f'F22-1445 Records_Part{i}.pdf' for i in range(start,start+n_docs)]
# part = 1
# filename = f'F22-1445 Records_Part{part}.pdf'

# dfs_list1_i10, reference_list1_i10 = parse_pdf_tolist(filename, pages='34,76', filename=None)
# # dfs, reference = parse_pdf(filenames, pages='all', filename='pdfs_parsed_2023-03-30')


In [196]:
n_docs = 6
start = 2
filenames = [f'F22-1445 Records_Part{i}.pdf' for i in range(start,start+n_docs)]

dfs_remainder_i12, reference_remainder_i12 = parse_pdf(filenames, pages='all', filename=None)

Extracting the following files:
Starting from page 1 of PDF
F22-1445 Records_Part2.pdf
number of columns: 23
	page 1...
	Removing s.22(1) from column 3
	Shifting data from column 3(workplace_address)
	page 2...
	Removing s.22(1) from column 3
	Shifting data from column 3(workplace_address)
	page 3...
		Padding columns

	**Fixing column parsing for page 3 of F22-1445 Records_Part2.pdf**
		Index(['Unnamed: 0', 'HA', 'workplace_name', 'workplace_address', 'city',
       'date_cluster_confirmed_public_health', 'date_cluster_no_longer_active',
       'earliest_reported_cases', 'latest_reported_cases',
       'workplace_classification', 'cases_when_cluster_confirmed',
       'total_cases', 'workplace_status', 'closure_y_n', 'closed_by_order',
       'voluntary_closure', 'closure_date', 'partially_closed', 'open',
       'days_since_last_case', 'active_clusters', 'cluster_confirmed_epidate',
       'cluster_closed_epidate', 'blank0', 'blank1'],
      dtype='object')
	Attempting to consolidate

In [218]:
print(dfs_remainder_i12.columns)
dfs_remainder_i12

Index(['index', 'HA', 'workplace_name', 'workplace_address', 'city',
       'date_cluster_confirmed_public_health', 'date_cluster_no_longer_active',
       'earliest_reported_cases', 'latest_reported_cases',
       'workplace_classification', 'cases_when_cluster_confirmed',
       'total_cases', 'workplace_status', 'closure_y_n', 'closed_by_order',
       'voluntary_closure', 'closure_date', 'partially_closed', 'open',
       'days_since_last_case', 'active_clusters', 'cluster_confirmed_epidate',
       'doc_page_number', 'cluster_closed_epidate', 'file', 'blank0',
       'unknown_closure_type'],
      dtype='object')


,index,HA,workplace_name,workplace_address,city,date_cluster_confirmed_public_health,date_cluster_no_longer_active,earliest_reported_cases,latest_reported_cases,workplace_classification,...,partially_closed,open,days_since_last_case,active_clusters,cluster_confirmed_epidate,doc_page_number,cluster_closed_epidate,file,blank0,unknown_closure_type
0,179.0,FHA,NaN,None,ABBOTSFORD,4/14/2021,5/3/2021,NaN,NaN,Farm/Agricultu\rre,...,No,No,NaN,No,15-2021,1,NaN,F22-1445 Records_Part2.pdf,NaN,NaN
1,180.0,FHA,NaN,None,DELTA,4/14/2021,5/9/2021,NaN,NaN,Retail,...,No,No,NaN,No,15-2021,1,NaN,F22-1445 Records_Part2.pdf,NaN,NaN
2,181.0,FHA,NaN,None,PORT\rCOQUITLAM,4/14/2021,5/6/2021,NaN,NaN,Maintenance/T\rrades/Construc\rtion,...,No,No,NaN,No,15-2021,1,NaN,F22-1445 Records_Part2.pdf,NaN,NaN
3,182.0,VCH,NaN,None,Vancouver,4/14/2021,NaN,4/14/2021,5/5/2021,Manufacturing\r/Industry,...,Yes,No,NaN,No,15-2021,1,NaN,F22-1445 Records_Part2.pdf,NaN,NaN
4,183.0,NHA,NaN,None,Unknown,4/14/2021,4/29/2021,NaN,NaN,Maintenance/T\rrades/Construc\rtion,...,Unknown,No,NaN,No,15-2021,1,NaN,F22-1445 Records_Part2.pdf,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1397,1090.0,IHA,None,None,Kamloops,1/26/2022,NaN,1/26/2022,NaN,School/Daycare,...,No,No,NaN,No,4-2022,81,4-2022,F22-1445 Records_Part7.pdf,NaN,No
1398,1091.0,IHA,None,None,Kamloops,1/27/2022,NaN,1/27/2022,NaN,School/Daycare,...,No,No,NaN,No,4-2022,81,4-2022,F22-1445 Records_Part7.pdf,NaN,No
1399,1092.0,IHA,None,None,Revelstoke,1/28/2022,NaN,1/28/2022,NaN,School/Daycare,...,No,No,NaN,No,4-2022,81,2-2022,F22-1445 Records_Part7.pdf,NaN,No
1400,1093.0,IHA,None,None,260 Cedar Ave,None,1/31/2022,None,1/31/2022,None,...,1/17/2022,No,No,None,No,82,5-2022,F22-1445 Records_Part7.pdf,NaN,No


In [202]:
dfs_remainder_i12[dfs_remainder_i12['blank0'].notna()]

,index,HA,workplace_name,workplace_address,city,date_cluster_confirmed_public_health,date_cluster_no_longer_active,earliest_reported_cases,latest_reported_cases,workplace_classification,...,partially_closed,open,days_since_last_case,active_clusters,cluster_confirmed_epidate,doc_page_number,cluster_closed_epidate,file,blank0,unknown_closure_type
1221,2.0,FHA,None,None,APLE\rIDGE,4/1/2021,4/18/2021,NaN,NaN,Restaurant/Bar\r/Lounge,...,No,No,Yes,NaN,No,84,13-2021,F22-1445 Records_Part3.pdf,14-2021,NaN
1222,3.0,FHA,None,None,URREY,4/1/2021,4/22/2021,NaN,NaN,Restaurant/Bar\r/Lounge,...,No,No,Yes,NaN,No,84,13-2021,F22-1445 Records_Part3.pdf,15-2021,NaN
1223,4.0,FHA,None,None,URREY,4/1/2021,4/14/2021,NaN,NaN,Office,...,No,No,Yes,NaN,No,84,13-2021,F22-1445 Records_Part3.pdf,13-2021,NaN
1225,6.0,FHA,None,None,URNABY,4/1/2021,4/21/2021,NaN,NaN,Gym/Fitness,...,No,No,Yes,NaN,No,84,13-2021,F22-1445 Records_Part3.pdf,14-2021,NaN
1227,8.0,FHA,None,None,BBOTSFOR,4/1/2021,4/19/2021,NaN,NaN,Gym/Fitness,...,No,No,Yes,NaN,No,84,13-2021,F22-1445 Records_Part3.pdf,13-2021,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1955,40.0,FHA,None,None,BURNABY,4/5/2021,4/23/2021,NaN,NaN,Office,...,No,No,Yes,NaN,No,131,14-2021,F22-1445 Records_Part3.pdf,14-2021,NaN
1977,62.0,FHA,None,None,ABBOTSFORD,4/6/2021,4/19/2021,NaN,NaN,Restaurant/Bar/\rLounge,...,No,No,Yes,NaN,No,133,14-2021,F22-1445 Records_Part3.pdf,14-2021,NaN
1982,67.0,FHA,None,None,NEW\rWESTMINSTE\rR,4/6/2021,4/14/2021,NaN,NaN,Personal Care,...,No,No,Yes,NaN,No,133,14-2021,F22-1445 Records_Part3.pdf,14-2021,NaN
1985,70.0,FHA,None,None,PORT MOODY,4/6/2021,4/15/2021,NaN,NaN,Office,...,No,No,Yes,NaN,No,133,14-2021,F22-1445 Records_Part3.pdf,14-2021,NaN


# Reparse part 3

In [213]:
part = 3
filename = f'F22-1445 Records_Part{part}.pdf'
pages = '1-87'
dfs3_part1_i12, reference3_part1_i12 = parse_pdf(filename, pages=pages, filename=None)

Extracting the following files:
Starting from page 1 of PDF
F22-1445 Records_Part3.pdf
number of columns: 23
	page 1...
	Removing s.22(1) from column 2
	Shifting data from column 2(workplace_name)
	Shifting data from column 3(workplace_address)
	page 2...
		Padding columns

	**Fixing column parsing for page 2 of F22-1445 Records_Part3.pdf**
		Index(['Unnamed: 0', 'HA', 'workplace_name', 'workplace_address', 'city',
       'date_cluster_confirmed_public_health', 'date_cluster_no_longer_active',
       'earliest_reported_cases', 'latest_reported_cases',
       'workplace_classification', 'cases_when_cluster_confirmed',
       'total_cases', 'workplace_status', 'closure_y_n', 'closed_by_order',
       'voluntary_closure', 'closure_date', 'partially_closed', 'open',
       'days_since_last_case', 'active_clusters', 'cluster_confirmed_epidate',
       'cluster_closed_epidate', 'blank0'],
      dtype='object')
	Attempting to consolidate rows that were incorrectly split: 
	Removing duplicate 

In [214]:
print(dfs3_part1_i12.columns)
dfs3_part1_i12

Index(['index', 'HA', 'workplace_name', 'workplace_address', 'city',
       'date_cluster_confirmed_public_health', 'date_cluster_no_longer_active',
       'earliest_reported_cases', 'latest_reported_cases',
       'workplace_classification', 'cases_when_cluster_confirmed',
       'total_cases', 'workplace_status', 'closure_y_n', 'closed_by_order',
       'voluntary_closure', 'closure_date', 'partially_closed', 'open',
       'days_since_last_case', 'active_clusters', 'cluster_confirmed_epidate',
       'doc_page_number', 'cluster_closed_epidate', 'file'],
      dtype='object')


,index,HA,workplace_name,workplace_address,city,date_cluster_confirmed_public_health,date_cluster_no_longer_active,earliest_reported_cases,latest_reported_cases,workplace_classification,...,voluntary_closure,closure_date,partially_closed,open,days_since_last_case,active_clusters,cluster_confirmed_epidate,doc_page_number,cluster_closed_epidate,file
0,1.0,FHA,None,None,LANGLEY,4/1/2021,4/24/2021,NaN,NaN,Personal\rCare,...,Unknown,NaN,No,No,NaN,No,13-2021,1,NaN,F22-1445 Records_Part3.pdf
1,2.0,FHA,None,None,SURREY,4/1/2021,4/23/2021,NaN,NaN,Gym/Fitness,...,Unknown,NaN,No,No,NaN,No,13-2021,1,NaN,F22-1445 Records_Part3.pdf
2,3.0,FHA,None,None,COQUITLA\rM,4/1/2021,4/14/2021,NaN,NaN,Office,...,Unknown,NaN,No,No,NaN,No,13-2021,1,NaN,F22-1445 Records_Part3.pdf
3,4.0,FHA,None,None,LANGLEY,4/1/2021,4/30/2021,NaN,NaN,Farm/Agricu\rlture,...,Unknown,NaN,No,No,NaN,No,13-2021,1,NaN,F22-1445 Records_Part3.pdf
4,5.0,FHA,None,None,LANGLEY,4/1/2021,6/13/2021,NaN,NaN,Warehouse,...,Unknown,NaN,No,No,NaN,No,13-2021,1,NaN,F22-1445 Records_Part3.pdf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1215,628.0,IHA,None,None,elowna,8/12/2021,NaN,8/12/2021,8/8/2021,Restaurant/Bar\r/Lounge,...,No,NaN,Unknow\rn,No,NaN,No,32-2021,83,NaN,F22-1445 Records_Part3.pdf
1216,629.0,IHA,None,None,elowna,8/12/2021,NaN,8/12/2021,8/17/2021,School/Daycare,...,Yes,8/6/2021,Unknow\rn,No,NaN,No,32-2021,83,31-2021,F22-1445 Records_Part3.pdf
1217,630.0,VIHA,None,None,ictoria,8/12/2021,NaN,8/5/2021,8/12/2021,Restaurant/Bar\r/Lounge,...,No,NaN,Unknow\rn,Yes,5.0,Yes,32-2021,83,NaN,F22-1445 Records_Part3.pdf
1218,631.0,IHA,None,None,est Kelowna,8/16/2021,NaN,8/16/2021,8/13/2021,Manufacturing/\rIndustry,...,No,NaN,Unknow\rn,No,NaN,No,33-2021,83,NaN,F22-1445 Records_Part3.pdf


In [215]:
part = 3
filename = f'F22-1445 Records_Part{part}.pdf'
pages = '91-146'
dfs3_part2_i12, reference3_part2_i12 = parse_pdf(filename, pages=pages, filename=None)

Extracting the following files:
Starting from page 91 of PDF
F22-1445 Records_Part3.pdf
number of columns: 24
	page 1...
	Shifting data from column 2(workplace_name)
	Shifting data from column 3(workplace_address)
	page 2...
	Removing s.22(1) from column 2
	Shifting data from column 2(workplace_name)
	Shifting data from column 3(workplace_address)
	page 3...
	Removing s.22(1) from column 2
	Shifting data from column 2(workplace_name)
	Shifting data from column 3(workplace_address)
	page 4...
	Shifting data from column 2(workplace_name)
	Shifting data from column 3(workplace_address)
	page 5...
		Padding columns

	**Fixing column parsing for page 5 of F22-1445 Records_Part3.pdf**
		Index(['Unnamed: 0', 'HA', 'workplace_name', 'workplace_address', 'city',
       'date_cluster_confirmed_public_health', 'date_cluster_no_longer_active',
       'earliest_reported_cases', 'latest_reported_cases',
       'workplace_classification', 'cases_when_cluster_confirmed',
       'total_cases', 'workpla

In [216]:
print(dfs3_part2_i12.columns)
dfs3_part2_i12

Index(['index', 'HA', 'workplace_name', 'workplace_address', 'city',
       'date_cluster_confirmed_public_health', 'date_cluster_no_longer_active',
       'earliest_reported_cases', 'latest_reported_cases',
       'workplace_classification', 'cases_when_cluster_confirmed',
       'total_cases', 'workplace_status', 'closure_y_n', 'closed_by_order',
       'voluntary_closure', 'closure_date', 'partially_closed',
       'unknown_closure_type', 'open', 'days_since_last_case',
       'active_clusters', 'cluster_confirmed_epidate',
       'cluster_closed_epidate', 'doc_page_number', 'file'],
      dtype='object')


,index,HA,workplace_name,workplace_address,city,date_cluster_confirmed_public_health,date_cluster_no_longer_active,earliest_reported_cases,latest_reported_cases,workplace_classification,...,closure_date,partially_closed,unknown_closure_type,open,days_since_last_case,active_clusters,cluster_confirmed_epidate,cluster_closed_epidate,doc_page_number,file
0,1.0,FHA,None,None,ELTA,4/1/2021,4/6/2021,NaN,NaN,Transportation,...,NaN,No,No,No,NaN,No,13-2021,NaN,91,F22-1445 Records_Part3.pdf
1,2.0,FHA,None,None,APLE\rIDGE,4/1/2021,4/18/2021,NaN,NaN,Restaurant/Bar\r/Lounge,...,4/6/2021,No,No,Yes,NaN,No,13-2021,14-2021,91,F22-1445 Records_Part3.pdf
2,3.0,FHA,None,None,URREY,4/1/2021,4/22/2021,NaN,NaN,Restaurant/Bar\r/Lounge,...,4/12/2021,No,No,Yes,NaN,No,13-2021,15-2021,91,F22-1445 Records_Part3.pdf
3,4.0,FHA,None,None,URREY,4/1/2021,4/14/2021,NaN,NaN,Office,...,4/3/2021,No,No,Yes,NaN,No,13-2021,13-2021,91,F22-1445 Records_Part3.pdf
4,5.0,FHA,None,None,ANGLEY,4/1/2021,6/13/2021,NaN,NaN,Warehouse,...,NaN,No,No,No,NaN,No,13-2021,NaN,91,F22-1445 Records_Part3.pdf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
808,113.0,FHA,None,None,SURREY,4/9/2021,4/26/2021,NaN,NaN,Retail,...,NaN,No,No,No,NaN,No,14-2021,NaN,143,F22-1445 Records_Part3.pdf
809,114.0,FHA,None,None,SURREY,4/9/2021,4/21/2021,NaN,NaN,Office,...,NaN,No,No,No,NaN,No,14-2021,NaN,143,F22-1445 Records_Part3.pdf
810,115.0,FHA,None,None,SURREY,4/9/2021,4/23/2021,NaN,NaN,Office,...,4/12/2021,No,No,Yes,NaN,No,14-2021,NaN,143,F22-1445 Records_Part3.pdf
811,116.0,FHA,None,None,SURREY,4/9/2021,5/29/2021,NaN,NaN,Food Processing,...,NaN,No,No,No,NaN,No,14-2021,NaN,143,F22-1445 Records_Part3.pdf


In [217]:
set(dfs3_part2_i12.columns) - set(dfs3_part1_i12.columns)

{'unknown_closure_type'}

# Reparse parts [2, 4, 5, 6, 7]

In [219]:
parts = [2, 4, 5, 6]
filenames = [f'F22-1445 Records_Part{i}.pdf' for i in parts]

dfs_24567_i12, reference_24567_i12 = parse_pdf(filenames, pages='all', filename=None)

Extracting the following files:
Starting from page 1 of PDF
F22-1445 Records_Part2.pdf
number of columns: 23
	page 1...
	Removing s.22(1) from column 3
	Shifting data from column 3(workplace_address)
	page 2...
	Removing s.22(1) from column 3
	Shifting data from column 3(workplace_address)
	page 3...
		Padding columns

	**Fixing column parsing for page 3 of F22-1445 Records_Part2.pdf**
		Index(['Unnamed: 0', 'HA', 'workplace_name', 'workplace_address', 'city',
       'date_cluster_confirmed_public_health', 'date_cluster_no_longer_active',
       'earliest_reported_cases', 'latest_reported_cases',
       'workplace_classification', 'cases_when_cluster_confirmed',
       'total_cases', 'workplace_status', 'closure_y_n', 'closed_by_order',
       'voluntary_closure', 'closure_date', 'partially_closed', 'open',
       'days_since_last_case', 'active_clusters', 'cluster_confirmed_epidate',
       'cluster_closed_epidate', 'blank0', 'blank1'],
      dtype='object')
	Attempting to consolidate

In [223]:
reference_24567_i12

{'corrected pages': {'F22-1445 Records_Part2.pdf': 139,
  'F22-1445 Records_Part4.pdf': 123,
  'F22-1445 Records_Part5.pdf': 86,
  'F22-1445 Records_Part6.pdf': 109,
  'F22-1445 Records_Part7.pdf': 81},
 'unique columns': {'F22-1445 Records_Part7.pdf': 'none'}}

In [221]:
print(dfs_24567_i12.columns)
dfs_24567_i12

Index(['index', 'HA', 'workplace_name', 'workplace_address', 'city',
       'date_cluster_confirmed_public_health', 'date_cluster_no_longer_active',
       'earliest_reported_cases', 'latest_reported_cases',
       'workplace_classification', 'cases_when_cluster_confirmed',
       'total_cases', 'workplace_status', 'closure_y_n', 'closed_by_order',
       'voluntary_closure', 'closure_date', 'partially_closed', 'open',
       'days_since_last_case', 'active_clusters', 'cluster_confirmed_epidate',
       'doc_page_number', 'cluster_closed_epidate', 'file',
       'unknown_closure_type'],
      dtype='object')


,index,HA,workplace_name,workplace_address,city,date_cluster_confirmed_public_health,date_cluster_no_longer_active,earliest_reported_cases,latest_reported_cases,workplace_classification,...,closure_date,partially_closed,open,days_since_last_case,active_clusters,cluster_confirmed_epidate,doc_page_number,cluster_closed_epidate,file,unknown_closure_type
0,179.0,FHA,NaN,None,ABBOTSFORD,4/14/2021,5/3/2021,NaN,NaN,Farm/Agricultu\rre,...,4/15/2021,No,No,NaN,No,15-2021,1,NaN,F22-1445 Records_Part2.pdf,NaN
1,180.0,FHA,NaN,None,DELTA,4/14/2021,5/9/2021,NaN,NaN,Retail,...,NaN,No,No,NaN,No,15-2021,1,NaN,F22-1445 Records_Part2.pdf,NaN
2,181.0,FHA,NaN,None,PORT\rCOQUITLAM,4/14/2021,5/6/2021,NaN,NaN,Maintenance/T\rrades/Construc\rtion,...,4/14/2021,No,No,NaN,No,15-2021,1,NaN,F22-1445 Records_Part2.pdf,NaN
3,182.0,VCH,NaN,None,Vancouver,4/14/2021,NaN,4/14/2021,5/5/2021,Manufacturing\r/Industry,...,NaN,Yes,No,NaN,No,15-2021,1,NaN,F22-1445 Records_Part2.pdf,NaN
4,183.0,NHA,NaN,None,Unknown,4/14/2021,4/29/2021,NaN,NaN,Maintenance/T\rrades/Construc\rtion,...,NaN,Unknown,No,NaN,No,15-2021,1,NaN,F22-1445 Records_Part2.pdf,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1397,1090.0,IHA,None,None,Kamloops,1/26/2022,NaN,1/26/2022,NaN,School/Daycare,...,1/23/2022,No,No,NaN,No,4-2022,81,4-2022,F22-1445 Records_Part7.pdf,No
1398,1091.0,IHA,None,None,Kamloops,1/27/2022,NaN,1/27/2022,NaN,School/Daycare,...,1/27/2022,No,No,NaN,No,4-2022,81,4-2022,F22-1445 Records_Part7.pdf,No
1399,1092.0,IHA,None,None,Revelstoke,1/28/2022,NaN,1/28/2022,NaN,School/Daycare,...,1/11/2022,No,No,NaN,No,4-2022,81,2-2022,F22-1445 Records_Part7.pdf,No
1400,1093.0,IHA,None,None,260 Cedar Ave,None,1/31/2022,None,1/31/2022,None,...,Yes,1/17/2022,No,No,None,No,82,5-2022,F22-1445 Records_Part7.pdf,No


# Reparse last page of part7

In [231]:
part = 7
filename = f'F22-1445 Records_Part{part}.pdf'
pages = '88'
dfs7_last_page_i12, reference7_last_page_i12 = parse_pdf(filename, pages=pages, filename=None)

Extracting the following files:
Starting from page 88 of PDF
F22-1445 Records_Part7.pdf


IndexError: list index out of range

In [249]:
dfs_24567_i12.reset_index(drop=True)

,index,HA,workplace_name,workplace_address,city,date_cluster_confirmed_public_health,date_cluster_no_longer_active,earliest_reported_cases,latest_reported_cases,workplace_classification,...,closure_date,partially_closed,open,days_since_last_case,active_clusters,cluster_confirmed_epidate,doc_page_number,cluster_closed_epidate,file,unknown_closure_type
0,179.0,FHA,NaN,None,ABBOTSFORD,4/14/2021,5/3/2021,NaN,NaN,Farm/Agricultu\rre,...,4/15/2021,No,No,NaN,No,15-2021,1,NaN,F22-1445 Records_Part2.pdf,NaN
1,180.0,FHA,NaN,None,DELTA,4/14/2021,5/9/2021,NaN,NaN,Retail,...,NaN,No,No,NaN,No,15-2021,1,NaN,F22-1445 Records_Part2.pdf,NaN
2,181.0,FHA,NaN,None,PORT\rCOQUITLAM,4/14/2021,5/6/2021,NaN,NaN,Maintenance/T\rrades/Construc\rtion,...,4/14/2021,No,No,NaN,No,15-2021,1,NaN,F22-1445 Records_Part2.pdf,NaN
3,182.0,VCH,NaN,None,Vancouver,4/14/2021,NaN,4/14/2021,5/5/2021,Manufacturing\r/Industry,...,NaN,Yes,No,NaN,No,15-2021,1,NaN,F22-1445 Records_Part2.pdf,NaN
4,183.0,NHA,NaN,None,Unknown,4/14/2021,4/29/2021,NaN,NaN,Maintenance/T\rrades/Construc\rtion,...,NaN,Unknown,No,NaN,No,15-2021,1,NaN,F22-1445 Records_Part2.pdf,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9089,1090.0,IHA,None,None,Kamloops,1/26/2022,NaN,1/26/2022,NaN,School/Daycare,...,1/23/2022,No,No,NaN,No,4-2022,81,4-2022,F22-1445 Records_Part7.pdf,No
9090,1091.0,IHA,None,None,Kamloops,1/27/2022,NaN,1/27/2022,NaN,School/Daycare,...,1/27/2022,No,No,NaN,No,4-2022,81,4-2022,F22-1445 Records_Part7.pdf,No
9091,1092.0,IHA,None,None,Revelstoke,1/28/2022,NaN,1/28/2022,NaN,School/Daycare,...,1/11/2022,No,No,NaN,No,4-2022,81,2-2022,F22-1445 Records_Part7.pdf,No
9092,1093.0,IHA,None,None,260 Cedar Ave,None,1/31/2022,None,1/31/2022,None,...,Yes,1/17/2022,No,No,None,No,82,5-2022,F22-1445 Records_Part7.pdf,No


In [263]:
dfs_24567_i12_p2 = dfs_24567_i12.reset_index(drop=True)
dfs_24567_i12_p2[9092:]

,index,HA,workplace_name,workplace_address,city,date_cluster_confirmed_public_health,date_cluster_no_longer_active,earliest_reported_cases,latest_reported_cases,workplace_classification,...,closure_date,partially_closed,open,days_since_last_case,active_clusters,cluster_confirmed_epidate,doc_page_number,cluster_closed_epidate,file,unknown_closure_type
9092,1093.0,IHA,None,None,260 Cedar Ave,None,1/31/2022,None,1/31/2022,None,...,Yes,1/17/2022,No,No,None,No,82,5-2022,F22-1445 Records_Part7.pdf,No
9093,1094.0,IHA,None,None,Road,None,1/31/2022,None,1/31/2022,None,...,Yes,1/31/2022,No,No,None,No,82,5-2022,F22-1445 Records_Part7.pdf,No


In [264]:
dfs_24567_i12_p2.loc[9092:, 'city':'closure_date'] = dfs_24567_i12_p2.loc[9092:, 'date_cluster_confirmed_public_health':'partially_closed'].values
dfs_24567_i12_p2[9092:]

,index,HA,workplace_name,workplace_address,city,date_cluster_confirmed_public_health,date_cluster_no_longer_active,earliest_reported_cases,latest_reported_cases,workplace_classification,...,closure_date,partially_closed,open,days_since_last_case,active_clusters,cluster_confirmed_epidate,doc_page_number,cluster_closed_epidate,file,unknown_closure_type
9092,1093.0,IHA,None,None,None,1/31/2022,None,1/31/2022,None,School/Daycare,...,1/17/2022,1/17/2022,No,No,None,No,82,5-2022,F22-1445 Records_Part7.pdf,No
9093,1094.0,IHA,None,None,None,1/31/2022,None,1/31/2022,None,School/Daycare,...,1/31/2022,1/31/2022,No,No,None,No,82,5-2022,F22-1445 Records_Part7.pdf,No


In [266]:
dfs_24567_i12_p2.loc[9092:,'days_since_last_case'] = None
dfs_24567_i12_p2.loc[9092:,'active_clusters'] = None
dfs_24567_i12_p2.loc[9092:,'cluster_confirmed_epidate'] = dfs_24567_i12_p2.loc[9092:,'cluster_closed_epidate'].values
dfs_24567_i12_p2.loc[9092:,'cluster_closed_epidate'] = ['3-2022', '5-2022']
dfs_24567_i12_p2.loc[9092:]

,index,HA,workplace_name,workplace_address,city,date_cluster_confirmed_public_health,date_cluster_no_longer_active,earliest_reported_cases,latest_reported_cases,workplace_classification,...,closure_date,partially_closed,open,days_since_last_case,active_clusters,cluster_confirmed_epidate,doc_page_number,cluster_closed_epidate,file,unknown_closure_type
9092,1093.0,IHA,None,None,None,1/31/2022,None,1/31/2022,None,School/Daycare,...,1/17/2022,1/17/2022,No,None,None,5-2022,82,3-2022,F22-1445 Records_Part7.pdf,No
9093,1094.0,IHA,None,None,None,1/31/2022,None,1/31/2022,None,School/Daycare,...,1/31/2022,1/31/2022,No,None,None,5-2022,82,5-2022,F22-1445 Records_Part7.pdf,No


In [271]:
dfs_24567_i12_p2

,index,HA,workplace_name,workplace_address,city,date_cluster_confirmed_public_health,date_cluster_no_longer_active,earliest_reported_cases,latest_reported_cases,workplace_classification,...,closure_date,partially_closed,open,days_since_last_case,active_clusters,cluster_confirmed_epidate,doc_page_number,cluster_closed_epidate,file,unknown_closure_type
0,179.0,FHA,NaN,None,ABBOTSFORD,4/14/2021,5/3/2021,NaN,NaN,Farm/Agricultu\rre,...,4/15/2021,No,No,NaN,No,15-2021,1,NaN,F22-1445 Records_Part2.pdf,NaN
1,180.0,FHA,NaN,None,DELTA,4/14/2021,5/9/2021,NaN,NaN,Retail,...,NaN,No,No,NaN,No,15-2021,1,NaN,F22-1445 Records_Part2.pdf,NaN
2,181.0,FHA,NaN,None,PORT\rCOQUITLAM,4/14/2021,5/6/2021,NaN,NaN,Maintenance/T\rrades/Construc\rtion,...,4/14/2021,No,No,NaN,No,15-2021,1,NaN,F22-1445 Records_Part2.pdf,NaN
3,182.0,VCH,NaN,None,Vancouver,4/14/2021,NaN,4/14/2021,5/5/2021,Manufacturing\r/Industry,...,NaN,Yes,No,NaN,No,15-2021,1,NaN,F22-1445 Records_Part2.pdf,NaN
4,183.0,NHA,NaN,None,Unknown,4/14/2021,4/29/2021,NaN,NaN,Maintenance/T\rrades/Construc\rtion,...,NaN,Unknown,No,NaN,No,15-2021,1,NaN,F22-1445 Records_Part2.pdf,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9089,1090.0,IHA,None,None,Kamloops,1/26/2022,NaN,1/26/2022,NaN,School/Daycare,...,1/23/2022,No,No,NaN,No,4-2022,81,4-2022,F22-1445 Records_Part7.pdf,No
9090,1091.0,IHA,None,None,Kamloops,1/27/2022,NaN,1/27/2022,NaN,School/Daycare,...,1/27/2022,No,No,NaN,No,4-2022,81,4-2022,F22-1445 Records_Part7.pdf,No
9091,1092.0,IHA,None,None,Revelstoke,1/28/2022,NaN,1/28/2022,NaN,School/Daycare,...,1/11/2022,No,No,NaN,No,4-2022,81,2-2022,F22-1445 Records_Part7.pdf,No
9092,1093.0,IHA,None,None,None,1/31/2022,None,1/31/2022,None,School/Daycare,...,1/17/2022,1/17/2022,No,None,None,5-2022,82,3-2022,F22-1445 Records_Part7.pdf,No


# Concatenate results

In [281]:
dfs_24567_i12_p2.columns

Index(['index', 'HA', 'workplace_name', 'workplace_address', 'city',
       'date_cluster_confirmed_public_health', 'date_cluster_no_longer_active',
       'earliest_reported_cases', 'latest_reported_cases',
       'workplace_classification', 'cases_when_cluster_confirmed',
       'total_cases', 'workplace_status', 'closure_y_n', 'closed_by_order',
       'voluntary_closure', 'closure_date', 'partially_closed', 'open',
       'days_since_last_case', 'active_clusters', 'cluster_confirmed_epidate',
       'doc_page_number', 'cluster_closed_epidate', 'file',
       'unknown_closure_type'],
      dtype='object')

In [284]:
print(len(dfs1_part2_i11.columns))
dfs1_part2_i11.columns

25


Index(['index', 'HA', 'workplace_name', 'workplace_address', 'city',
       'date_cluster_confirmed_public_health', 'date_cluster_no_longer_active',
       'earliest_reported_cases', 'latest_reported_cases',
       'workplace_classification', 'cases_when_cluster_confirmed',
       'total_cases', 'workplace_status', 'closure_y_n', 'closed_by_order',
       'voluntary_closure', 'closure_date', 'partially_closed', 'open',
       'days_since_last_case', 'active_clusters', 'cluster_confirmed_epidate',
       'doc_page_number', 'cluster_closed_epidate', 'file'],
      dtype='object')

In [285]:
print(len(dfs1_part1_i11.columns))
dfs1_part1_i11.columns

24


Index(['index', 'HA', 'workplace_name', 'workplace_address', 'city',
       'date_cluster_confirmed_public_health', 'date_cluster_no_longer_active',
       'earliest_reported_cases', 'latest_reported_cases',
       'workplace_classification', 'cases_when_cluster_confirmed',
       'total_cases', 'workplace_status', 'closure_y_n',
       'closed_by_order_order', 'voluntary_closure', 'closure_date',
       'partially_closed', 'open', 'days_since_last_case',
       'cluster_confirmed_epidate', 'cluster_closed_epidate',
       'doc_page_number', 'file'],
      dtype='object')

In [286]:
pd.concat([
    dfs_24567_i12_p2,
    dfs1_part1_i11.rename(columns={'closed_by_order_order': 'closed_by_order'}, inplace=False),
    dfs1_part2_i11,
    dfs3_part1_i12,
    dfs3_part2_i12,
    # dfs_24567_i12_p2,
])

,index,HA,workplace_name,workplace_address,city,date_cluster_confirmed_public_health,date_cluster_no_longer_active,earliest_reported_cases,latest_reported_cases,workplace_classification,...,closure_date,partially_closed,open,days_since_last_case,active_clusters,cluster_confirmed_epidate,doc_page_number,cluster_closed_epidate,file,unknown_closure_type
0,179.0,FHA,NaN,None,ABBOTSFORD,4/14/2021,5/3/2021,NaN,NaN,Farm/Agricultu\rre,...,4/15/2021,No,No,NaN,No,15-2021,1,NaN,F22-1445 Records_Part2.pdf,NaN
1,180.0,FHA,NaN,None,DELTA,4/14/2021,5/9/2021,NaN,NaN,Retail,...,NaN,No,No,NaN,No,15-2021,1,NaN,F22-1445 Records_Part2.pdf,NaN
2,181.0,FHA,NaN,None,PORT\rCOQUITLAM,4/14/2021,5/6/2021,NaN,NaN,Maintenance/T\rrades/Construc\rtion,...,4/14/2021,No,No,NaN,No,15-2021,1,NaN,F22-1445 Records_Part2.pdf,NaN
3,182.0,VCH,NaN,None,Vancouver,4/14/2021,NaN,4/14/2021,5/5/2021,Manufacturing\r/Industry,...,NaN,Yes,No,NaN,No,15-2021,1,NaN,F22-1445 Records_Part2.pdf,NaN
4,183.0,NHA,NaN,None,Unknown,4/14/2021,4/29/2021,NaN,NaN,Maintenance/T\rrades/Construc\rtion,...,NaN,Unknown,No,NaN,No,15-2021,1,NaN,F22-1445 Records_Part2.pdf,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
808,113.0,FHA,None,None,SURREY,4/9/2021,4/26/2021,NaN,NaN,Retail,...,NaN,No,No,NaN,No,14-2021,143,NaN,F22-1445 Records_Part3.pdf,No
809,114.0,FHA,None,None,SURREY,4/9/2021,4/21/2021,NaN,NaN,Office,...,NaN,No,No,NaN,No,14-2021,143,NaN,F22-1445 Records_Part3.pdf,No
810,115.0,FHA,None,None,SURREY,4/9/2021,4/23/2021,NaN,NaN,Office,...,4/12/2021,No,Yes,NaN,No,14-2021,143,NaN,F22-1445 Records_Part3.pdf,No
811,116.0,FHA,None,None,SURREY,4/9/2021,5/29/2021,NaN,NaN,Food Processing,...,NaN,No,No,NaN,No,14-2021,143,NaN,F22-1445 Records_Part3.pdf,No


In [287]:
pd.concat([
    dfs_24567_i12_p2,
    dfs1_part1_i11.rename(columns={'closed_by_order_order': 'closed_by_order'}, inplace=False),
    dfs1_part2_i11,
    dfs3_part1_i12,
    dfs3_part2_i12,
    # dfs_24567_i12_p2,
]).columns.tolist()

['index',
 'HA',
 'workplace_name',
 'workplace_address',
 'city',
 'date_cluster_confirmed_public_health',
 'date_cluster_no_longer_active',
 'earliest_reported_cases',
 'latest_reported_cases',
 'workplace_classification',
 'cases_when_cluster_confirmed',
 'total_cases',
 'workplace_status',
 'closure_y_n',
 'closed_by_order',
 'voluntary_closure',
 'closure_date',
 'partially_closed',
 'open',
 'days_since_last_case',
 'active_clusters',
 'cluster_confirmed_epidate',
 'doc_page_number',
 'cluster_closed_epidate',
 'file',
 'unknown_closure_type']

In [296]:
column_sequence = [
    'index',
    'HA',
    'workplace_name',
    'workplace_address',
    'city',
    'date_cluster_confirmed_public_health',
    'date_cluster_no_longer_active',
    'earliest_reported_cases',
    'latest_reported_cases',
    'workplace_classification',
    'cases_when_cluster_confirmed',
    'total_cases',
    'workplace_status',
    'closure_y_n',
    'closed_by_order',
    'voluntary_closure',
    'closure_date',
    'partially_closed',
    'unknown_closure_type',
    'open',
    'days_since_last_case',
    'active_clusters',
    'cluster_confirmed_epidate',
    'doc_page_number',
    'cluster_closed_epidate',
    'file'
    ]


In [298]:
 df_v1 = pd.concat([
    dfs_24567_i12_p2,
    dfs1_part1_i11.rename(columns={'closed_by_order_order': 'closed_by_order'}, inplace=False),
    dfs1_part2_i11,
    dfs3_part1_i12,
    dfs3_part2_i12
    ])
df_v2 = df_v1[column_sequence].reset_index(drop=True)
df_v2

,index,HA,workplace_name,workplace_address,city,date_cluster_confirmed_public_health,date_cluster_no_longer_active,earliest_reported_cases,latest_reported_cases,workplace_classification,...,closure_date,partially_closed,unknown_closure_type,open,days_since_last_case,active_clusters,cluster_confirmed_epidate,doc_page_number,cluster_closed_epidate,file
0,179.0,FHA,NaN,None,ABBOTSFORD,4/14/2021,5/3/2021,NaN,NaN,Farm/Agricultu\rre,...,4/15/2021,No,NaN,No,NaN,No,15-2021,1,NaN,F22-1445 Records_Part2.pdf
1,180.0,FHA,NaN,None,DELTA,4/14/2021,5/9/2021,NaN,NaN,Retail,...,NaN,No,NaN,No,NaN,No,15-2021,1,NaN,F22-1445 Records_Part2.pdf
2,181.0,FHA,NaN,None,PORT\rCOQUITLAM,4/14/2021,5/6/2021,NaN,NaN,Maintenance/T\rrades/Construc\rtion,...,4/14/2021,No,NaN,No,NaN,No,15-2021,1,NaN,F22-1445 Records_Part2.pdf
3,182.0,VCH,NaN,None,Vancouver,4/14/2021,NaN,4/14/2021,5/5/2021,Manufacturing\r/Industry,...,NaN,Yes,NaN,No,NaN,No,15-2021,1,NaN,F22-1445 Records_Part2.pdf
4,183.0,NHA,NaN,None,Unknown,4/14/2021,4/29/2021,NaN,NaN,Maintenance/T\rrades/Construc\rtion,...,NaN,Unknown,NaN,No,NaN,No,15-2021,1,NaN,F22-1445 Records_Part2.pdf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13294,113.0,FHA,None,None,SURREY,4/9/2021,4/26/2021,NaN,NaN,Retail,...,NaN,No,No,No,NaN,No,14-2021,143,NaN,F22-1445 Records_Part3.pdf
13295,114.0,FHA,None,None,SURREY,4/9/2021,4/21/2021,NaN,NaN,Office,...,NaN,No,No,No,NaN,No,14-2021,143,NaN,F22-1445 Records_Part3.pdf
13296,115.0,FHA,None,None,SURREY,4/9/2021,4/23/2021,NaN,NaN,Office,...,4/12/2021,No,No,Yes,NaN,No,14-2021,143,NaN,F22-1445 Records_Part3.pdf
13297,116.0,FHA,None,None,SURREY,4/9/2021,5/29/2021,NaN,NaN,Food Processing,...,NaN,No,No,No,NaN,No,14-2021,143,NaN,F22-1445 Records_Part3.pdf


In [306]:
df_v2[(df_v2['index'] == 197) & (df_v2['file'] == 'F22-1445 Records_Part1.pdf') &  (df_v2['HA'] == 'FHA')]

,index,HA,workplace_name,workplace_address,city,date_cluster_confirmed_public_health,date_cluster_no_longer_active,earliest_reported_cases,latest_reported_cases,workplace_classification,...,closure_date,partially_closed,unknown_closure_type,open,days_since_last_case,active_clusters,cluster_confirmed_epidate,doc_page_number,cluster_closed_epidate,file
9290,197.0,FHA,None,None,BBOTSFORD,4/20/2021,5/5/2021,NaN,NaN,Personal Care,...,4/22/2021,No,NaN,No,NaN,NaN,16-2021,12,NaN,F22-1445 Records_Part1.pdf
9754,197,FHA,None,None,DELTA,4/20/2021,5/4/2021,NaN,NaN,Transportatio\rn,...,4/21/2021,No,NaN,No,NaN,No,16-2021,44,NaN,F22-1445 Records_Part1.pdf


In [310]:
df_v3 = df_v2
df_v3.loc[9754, 'cluster_closed_epidate'] = df_v3.loc[9754, 'cluster_confirmed_epidate']
df_v3.loc[9754:]

,index,HA,workplace_name,workplace_address,city,date_cluster_confirmed_public_health,date_cluster_no_longer_active,earliest_reported_cases,latest_reported_cases,workplace_classification,...,closure_date,partially_closed,unknown_closure_type,open,days_since_last_case,active_clusters,cluster_confirmed_epidate,doc_page_number,cluster_closed_epidate,file
9754,197,FHA,None,None,DELTA,4/20/2021,5/4/2021,NaN,NaN,Transportatio\rn,...,4/21/2021,No,NaN,No,NaN,No,16-2021,44,16-2021,F22-1445 Records_Part1.pdf
9755,198,FHA,None,None,SURREY,4/20/2021,5/3/2021,NaN,NaN,Personal Care,...,4/21/2021,No,NaN,No,NaN,No,16-2021,45,NaN,F22-1445 Records_Part1.pdf
9756,199,FHA,None,None,LANGLEY,4/20/2021,5/9/2021,NaN,NaN,Maintenance/\rTrades/Constr\ruction,...,NaN,No,NaN,No,NaN,No,16-2021,45,NaN,F22-1445 Records_Part1.pdf
9757,200,FHA,None,None,SURREY,4/20/2021,4/27/2021,NaN,NaN,Gym/Fitness,...,4/20/2021,No,NaN,No,NaN,No,16-2021,45,NaN,F22-1445 Records_Part1.pdf
9758,201,FHA,None,None,BURNABY,4/20/2021,5/4/2021,NaN,NaN,Manufacturin\rg/Industry,...,NaN,No,NaN,No,NaN,No,16-2021,45,NaN,F22-1445 Records_Part1.pdf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13294,113.0,FHA,None,None,SURREY,4/9/2021,4/26/2021,NaN,NaN,Retail,...,NaN,No,No,No,NaN,No,14-2021,143,NaN,F22-1445 Records_Part3.pdf
13295,114.0,FHA,None,None,SURREY,4/9/2021,4/21/2021,NaN,NaN,Office,...,NaN,No,No,No,NaN,No,14-2021,143,NaN,F22-1445 Records_Part3.pdf
13296,115.0,FHA,None,None,SURREY,4/9/2021,4/23/2021,NaN,NaN,Office,...,4/12/2021,No,No,Yes,NaN,No,14-2021,143,NaN,F22-1445 Records_Part3.pdf
13297,116.0,FHA,None,None,SURREY,4/9/2021,5/29/2021,NaN,NaN,Food Processing,...,NaN,No,No,No,NaN,No,14-2021,143,NaN,F22-1445 Records_Part3.pdf


# Save final dataframe

In [320]:
filename = 'pdfs_parsed_2023-04-02'
path = r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\BC-Covid-workplace-closures\data\interim'
savepickle(df_v3, filename=filename, path=path)
save_csv(df_v3, filename=filename, path=path)

File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/BC-Covid-workplace-closures/data/interim/pdfs_parsed_2023-04-02.sav
Time completed: 2023-04-02 02:46:04.334085
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/BC-Covid-workplace-closures/data/interim/pdfs_parsed_2023-04-02.csv
Time completed: 2023-04-02 02:46:04.639010


# *End of Page*